<a href="https://colab.research.google.com/github/manzoniandrea/Geo-reconstruction_-ANN/blob/main/dataCategorization/Piemonte/Lithology_Pie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Library installation

In [ ]:
pip install geopandas

In [ ]:
pip install rasterio

In [ ]:
pip install pyvista

#Import libraries

In [ ]:
import pyvista as pv
import geopandas as gpd
import pandas as pd
from pyproj import Transformer
import numpy as np

#Import data location
data source: http://www.arpa.piemonte.it/

In [ ]:
#importo i dati geografici dal file
gdf = gpd.read_file('arpa_geotecnicageofisica.gpkg', layer='v_perforazione')
gdf = gdf[['link_stratigrafiasemplificata','geometry']]
transformer = Transformer.from_crs('epsg:32632','esri:54012',always_xy=True)
points = list(zip(gdf['geometry'].x,gdf['geometry'].y))
coordsConv = pd.DataFrame (list(transformer.itransform(points)),columns=['Easting','Northing'])
dataframe =  pd.concat([gdf['link_stratigrafiasemplificata'],coordsConv], axis=1)

In [ ]:
dataframe.link_stratigrafiasemplificata[1]

load form the website the missing information

In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
#scarico i dati dal sito riportato nei dati geolocalizzati importati sopra
dataframe['spessore_livello'] = np.zeros(dataframe.shape[0])
dataframe['l1_descr'] = np.zeros(dataframe.shape[0])
well=0
for i in range(0,dataframe.shape[0]):
  link = dataframe['link_stratigrafiasemplificata'][well]
  source = requests.get(link).text
  soup = BeautifulSoup(source, 'lxml')
  tables = soup.find_all('table')
  if len(list(tables))<=3:
    #print(tables[4].prettify())
    well+=1
  else:
    tbl = tables[4]
    data_frame = pd.read_html(str(tbl),header=0)[0]
    for layer in range(0,data_frame.shape[0]):
      if layer == 0:
        dataframe['spessore_livello'][well] = data_frame['Profondit&agrave (m)'][0]
        dataframe['l1_descr'][well] = data_frame.Descrizione[0]
      else:
        line = pd.DataFrame({'link_stratigrafiasemplificata':dataframe['link_stratigrafiasemplificata'][well],'Easting':dataframe['Easting'][well],'Northing':dataframe['Northing'][well],'spessore_livello':data_frame['Profondit&agrave (m)'][layer],'l1_descr':data_frame['Descrizione'][layer]}, index=[well+layer+1])
        dataframe = pd.concat([dataframe.iloc[:well+layer], line, dataframe.iloc[well+layer:]]).reset_index(drop=True)
    well=well+layer+1
  

In [ ]:
dataframe = pd.read_excel('piemonte.xlsx') 

#Data preprocessing

In [ ]:
wellLoc = dataframe

import rasterio
import geopandas as gpd

In [ ]:
import rasterio
import geopandas as gpd

src = rasterio.open('DEM_clip.tif')
coords = [(x,y) for x, y in zip( wellLoc.Easting, wellLoc.Northing)]

wellLoc.index = range(len(wellLoc))
wellLoc['quota'] = [x[0] for x in src.sample(coords)]


In [ ]:
#valuto il top e il bottom layer dei dati
offsetList = []
linSec = []
linVerts = []

i = 0

botLito = np.zeros(wellLoc.spessore_livello.size)
topLito = np.zeros(wellLoc.spessore_livello.size)

for layer in range(0,wellLoc.spessore_livello.size):
  x,y,z = wellLoc.loc[layer][['Easting','Northing','quota']]
  if layer == 0:
    topLito[0] = wellLoc.quota[0]
    botLito[0] = topLito[0] - wellLoc.spessore_livello[0]
  elif wellLoc.link_stratigrafiasemplificata[layer] == wellLoc.link_stratigrafiasemplificata[layer-1]:
    topLito[layer] =  botLito[layer-1]
    botLito[layer] = topLito[layer] - (float(wellLoc.spessore_livello[layer])-float(wellLoc.spessore_livello[layer-1]))
  else:
    topLito[layer] = wellLoc.quota[layer]
    botLito[layer] = topLito[layer] - wellLoc.spessore_livello[layer]
  cellVerts = [[x,y,topLito[layer]],[x,y,botLito[layer]]]
  offsetList.append(i*3)
  linSec = linSec + [2,2*i,2*i+1]
  linVerts = linVerts + cellVerts
  i+=1
  if topLito[layer] > wellLoc.quota[layer]:
    print(layer)

offsetArray = np.array(offsetList)
linArray = np.array(linSec)
cellType = np.ones([i])*3
vertArray = np.array(linVerts)

wellLoc['top_Layer'] = topLito
wellLoc['bttom_Layer'] =  botLito

#Data categorization

In [ ]:
#creo e stampo il dizionario
litoDict =  {}

i=1000

litotype = pd.DataFrame(np.zeros(len(wellLoc.l1_descr.unique())))
litotype['name'] = pd.DataFrame(np.zeros(len(wellLoc.l1_descr.unique())))
litotype = litotype.iloc[: , 1:]
litotype['name'] = wellLoc.l1_descr.unique()
litotype.name.astype(str)
litotype.name[742] ='zero'
litotype = litotype.sort_values(by=['name'],ignore_index=1)

for lito in litotype.name.unique():
  litoDict[lito] = i
  i += 1
litoDict

In [ ]:
#uso il dizionario per dare un valore numerico ai litotipi
wellLoc['l1'] = wellLoc.l1_descr
wellLoc = wellLoc.replace({'l1':litoDict})

In [ ]:
#salvo i risultati in un file excel
wellLoc.to_excel('piemonte.xlsx') 

In [ ]:
# creo un file di visualizzazione dei risultati
grid = pv.UnstructuredGrid(offsetArray, linArray, cellType, vertArray)
grid.cell_arrays["values"] = wellLoc.l1.values
grid.save('wellLito_P.vtu',binary=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: VTK 9 no longer accepts an offset array
  """Entry point for launching an IPython kernel.


In [ ]:
litoDict[1]

In [ ]:
for i in range(0,len(litoDict)):
   descr = litoDict[i,0]

   litoDict["descr"] = 2018

In [ ]:
wellLoc = pd.read_excel('piemonte.xlsx')

In [ ]:
#creo e stampo il dizionario
litoDict =  {}

i=1000

litotype = pd.DataFrame(np.zeros(len(wellLoc.l1_descr.unique())))
litotype['name'] = pd.DataFrame(np.zeros(len(wellLoc.l1_descr.unique())))
litotype = litotype.iloc[: , 1:]
litotype['name'] = wellLoc.l1_descr.unique()
litotype.name.astype(str)
litotype.name[742] ='zero'
litotype = litotype.sort_values(by=['name'],ignore_index=1)

for lito in litotype.name.unique():
  litoDict[lito] = i
  i += 1

In [ ]:
key_list = list(litoDict.keys())
val_list = list(litoDict.values())

In [ ]:
#all values containing a key word
keyW = 'matrice argillosa'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if 'scarsa matrice' in key_list[position] or 'poca matrice' in key_list[position] or 'subordinata matrice' in key_list[position]:
    a=1
  else:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

alluvione ghiaioso ciottolosa in prevalente matrice argillosa debolmente limosa
alluvioni ghiaie e ciottoli in matrice argillosa
alterazione substrato livelli alterati di noduli quarzosi frammenti in matrice argillosa
alterazione substrato noduli quarzosi frammenti in matrice argillosa
alternanza di arenaria marna siltosa ossidata e sabbia medio fine debolmente cementata con passata a matrice argillosa fratture ossidate
alternanza di marna siltosa litoide e arenaria litoide con passate fratturate ossidate in matrice argillosa fratture ossidate
alternanza di strati di marna argillosa molto consistente e di strati di sabbia e/o sabbia ghiaiosa con rari clasti ciottolosi in matrice argillosa addensata
alternanze di ghiaie a matrice sabbiosa e matrice argillosa debolmente sabbiosa clasti eterometrici rari ciottoli
ammasso roccioso disarticolato , prevalentemente blocchi fratturati di calcare marnoso in matrice argillosa debolmente limosa
ammasso roccioso serpentinitico disarticolato altera

In [ ]:
#all values containing a key word
keyW = 'matrice argilloso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if 'scarsa matrice' in key_list[position] or 'poca matrice' in key_list[position] or 'subordinata matrice' in key_list[position]:
    a=1
  else:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

abbondante matrice argilloso limoso sabbiosa inglobante clasti rocciosi
abbondante matrice argilloso limoso sabbiosa inglobante ghiaietto e clasti in subordine
abbondante matrice argilloso limoso sabbiosa inglobante rari clasti rocciosi eterometrici
alluvione ghiaioso ciottolosa con ciottoli in matrice argilloso limosa abbondante
ammasso roccioso con frammenti di calcescisti alterati argillificati in matrice argilloso sabbiosa
ammasso roccioso disarticolato blocchi in matrice argilloso limosa a volte sabbiosa
arenaria in frammenti grossolani in matrice argilloso limosa plastica
argilla limosa da mediamente compatta a compatta con livelli di ghiaia fine in matrice argilloso limosa
argilloscisti e metapeliti scistosi costituito da scagliette laminate , teneri , a struttura caotica in matrice argilloso limoso
blocchi e clasti rocciosi in matrice argilloso sabbiosa con frammenti rocciosi sparsi
blocchi e clasti rocciosi in matrice argilloso sabbiosa con frammenti rocciosi sparsi alterazion

In [ ]:
#all values containing a key word
keyW = 'matrice limosa'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if 'scarsa matrice' in key_list[position] or 'poca matrice' in key_list[position] or 'subordinata matrice' in key_list[position]:
    a=1
  else:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

Ghiaia eterometrica con ciottoli in abbondante matrice limosa, a tratti debolmente sabbiosa, molto consistente; presenza di livelli a matrice prevalente
Ghiaia eterometrica in abbondante matrice limosa
abbondanti ciottoli in matrice limosa debolmente sabbiosa
alternanza di livelli sabbiosi e sabbioso ghiaiosi, con matrice limosa abbondante
alternanza di materiale sciolto a matrice limosa e di materiale sabbioso ghiaioso compatto in matrice limoso argillosa a volte con presenza di piccoli strati del substrato roccioso
alternanze di livelli costituito da clasti e frammenti di scisti grafitici molto alterati e sfatti in matrice limosa sabbiosa debolmente argillosa e livelli con frazione prevalente lapidea ; da moderatamente a scarsamente addensata
alternzanza di limo sabbioso debolmente argilloso poco addensato plastico con livelli sabbioso ghiaiosi in matrice limosa sciolti
ammasso roccioso disarticolato blocchi di quarzite alterata e argillificata raramente litoide in matrice limosa deb

In [ ]:
#all values containing a key word
keyW = 'matrice limo'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if 'scarsa matrice' in key_list[position] or 'poca matrice' in key_list[position] or 'subordinata matrice' in key_list[position]:
    a=1
  else:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

Ghiaia eterometrica con clasti in abbondante matrice limoso sabbioso
Ghiaia eterometrica con rari ciottoli, con clasti, in abbondante matrice limoso sabbiosa, addensata
alluvione ciottolosa in matrice limoso sabbiosa
alluvione ciottoloso ghiaiosa in abbondante matrice limoso argillosa
alluvione ciottoloso ghiaiosa in matrice limoso sabbiosa
alluvione ghiaioso ciottolosa in matrice limoso argillosa abbondante
alluvione ghiaioso ciottolosa media in matrice limoso argillosa abbondante
alternanza di ghiaia con rari ciottoli in matrice limoso sabbiosa, debolmente argillosa e livelli a matrice limoso sabbioso argillosa con ghiaia subordinata, addensata; presenza di alterazione dei clasti
alternanza di ghiaia e ciottoli in matrice sabbiosa e ghiaia eterometrica in matrice limo sabbiosa consistente; preenza di livello di sabbia debolmente limosa
alternanza di ghiaia eteometrica e ciottoli in matrice limo sabbiosa
alternanza di livelli di sabbia medio fine debolmente limosa e di argilla marnosa

In [ ]:
#all values containing a key word
keyW = 'matrice limoso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if 'scarsa matrice' in key_list[position] or 'poca matrice' in key_list[position] or 'subordinata matrice' in key_list[position]:
    a=1
  else:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

In [ ]:
#all values containing a key word
keyW = 'matrice sabbiosa'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if 'scarsa matrice' in key_list[position] or 'poca matrice' in key_list[position] or 'subordinata matrice' in key_list[position]:
    a=1
  else:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

Riporto costituito da ghiaia con ciottoli e rari frammenti di laterizio in matrice sabbiosa a tratti limosa; a tratti sabbia debolmente limosa e consistente con poca ghiaia e frammenti
Terreno di riporto costituito da ghiaia in matrice sabbiosa e livello di limo debolmente argilloso consistente inglobante ghiaia; presenza di frammenti di laterizi
abbondante matrice sabbiosa micacea grossolana
abbondanti clasti in matrice sabbiosa
alluvione ciottolosa con ciottoli in matrice sabbiosa
alluvione ciottolosa in abbondante matrice sabbiosa
alluvione ciottolosa in matrice sabbiosa a tratti ghiaiosa
alluvione ciottolosa in matrice sabbiosa fine
alluvione ciottolosa in matrice sabbiosa medio fine
alluvione ciottolosa in matrice sabbiosa medio fine debolmente argilloso limosa
alluvione ghiaiosa in matrice sabbiosa debolmente argillosa
alluvione ghiaioso ciottolosa in matrice sabbiosa
alluvioni ciottoli in matrice sabbiosa
alluvioni ciottoli medio piccoli in matrice sabbiosa
alluvioni ciottoli mo

In [ ]:
#all values containing a key word
keyW = 'matrice sabbioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if 'scarsa matrice' in key_list[position] or 'poca matrice' in key_list[position] or 'subordinata matrice' in key_list[position]:
    a=1
  else:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

abbondante matrice sabbioso ghiaiosa da debolmente limosa a limosa con pochi clasti
abbondante matrice sabbioso limosa con rari ciottoli
alluvione ciottolosa in matrice sabbioso argillosa
alluvione ciottolosa in matrice sabbioso fine
alluvione ciottolosa in matrice sabbioso ghiaiosa
alluvione ciottolosa in matrice sabbioso limosa
alluvione ghiaioso ciottolosa in matrice sabbioso limosa
alternanza di ghiaie eterometriche sabbiose e sabbie ghiaiose con livelli di ciottoli e livelli a matrice sabbioso limosa debolmente coesivi e/o cementati
alternanza di livelli di ghiaia in matrice sabbioso-limosa e livelli sabbioso-limosa, intenso grado di ossidazione e alterazione, moderatamente addensata
alternanza di orizzonti di ghiaia con ciottoli in matrice sabbioso limosa e sabbia fine limosa, addensata, con ghiaia e ciottoli; presenza di trovanti di rocce scistose
alternanza di trovanti di granito e livelli di matrice sabbioso limoso argillosa
alternanze di ghiaia eterometrica in abbondante matr

In [ ]:
#all values containing a key word
#all values containing a key word
keyW = 'sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

Output streaming troncato alle ultime 5000 righe.
sabbia fine limoso argillosa con livelli laminazioni siltose da poco consistente a consistente
sabbia fine limoso argillosa con piccoli elementi ghiaiosi
sabbia fine limoso argillosa con pochi ciottoli e ghiaia
sabbia fine limoso argillosa con rara ghiaia fine media
sabbia fine limoso argillosa debolmente plastica
sabbia fine limoso argillosa e rari livelli diargilla sabbiosa
sabbia fine limoso argillosa ghiaiosa compatta con ciottoli alterati
sabbia fine limoso argillosa inglobante ghiaia con presenza di trovante e ciottoli
sabbia fine limoso argillosa inglobante piccoli livelli carboniosi e torbosi molto compressibile
sabbia fine limoso argillosa scarsamente addensata
sabbia fine limoso ghiaiosa da poco a moderatamente addensata rari ciottoli
sabbia fine limoso ghiaiosa molto addensata con rari ciottoli
sabbia fine limoso talcosa con frammenti scistosi
sabbia fine localmente debolmente cementata inglobante ghiaia e rari ciottoli
sabbi

In [ ]:
#all values containing a key word
keyW = 'limo'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

limo
limo , argilla e sabbia fine con scarsa ghiaia medio fine poco addensata plastica
limo , argilla e sabbia fine debolmente addensati
limo , con sabbia fine, rara ghiaia, rara ciottoli
limo , consistente; presenza di concrezioni nodulari
limo , limo argilloso e argilla limosa
limo , limo argilloso e argilla limosa a struttura debolmente stratificata , a tratti debolmente ossidata
limo , limo argilloso e argilla limosa debolmente marnosa a struttura caotica o debolmente stratificata , inglobante abbondanti noduli carbonatici
limo , limo argilloso e limo sabbioso con ghiaia e piccoli ciottoli di quarzite
limo , limo argilloso e limo sabbioso con ghiaia parzialmente alterata argillificata
limo , limo argilloso e sabbia limosa con presenza di clasti arenacei
limo , limo sabbioso e limo argilloso con brecce medio fini parzialmente alterate , presenza di radici
limo , limo sabbioso e limo argilloso con rara ghiaia e zolle di micascisti completamente alterati
limo , limo sabbioso e limo ar

In [ ]:
#all values containing a key word
keyW = 'argilla'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

argilla
argilla , alcuni trovanti e ghiaia
argilla , argilla limosa , argilla marnosa , limo e limo sabbioso a struttura debolmente stratificata con presenza di noduli e concrezioni carbonatiche e livelli sottili sabbiosi ossidati
argilla , ciottoli e sabbia
argilla , ghiaia e sabbia
argilla , ghiaietto e rari ciottoli
argilla , grossi ciottoli e ghiaietto
argilla , limo , sabbia e ghiaia mediamente compatto
argilla , sabbia
argilla , strati di sabbia e ciottoli
argilla a livelli debolmente limosa poco consistente plastica molto compressibile
argilla a livelli debolmente marnosa
argilla a livelli debolmente marnosa con intercalazioni sabbiose moderatamente consistente a volte plastica
argilla a livelli debolmente siltosa e/o siltoso sabbioso fine , da moderatamente consistente a consistente
argilla a livelli marnosi compatti
argilla a livelli siltosa con noduli e/o livelli calcarei consistente
argilla a strati
argilla a strati alternati molto compatta
argilla a strati con sabbia
argill

In [ ]:
#all values containing a key word
keyW = 'terreno di riporto argilloso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

terreno di riporto argilloso
terreno di riporto argilloso con resti di laterizi
terreno di riporto argilloso limoso ghiaioso
terreno di riporto argilloso limoso sabbioso
terreno di riporto argilloso sabbioso con ghiaia e resti di laterizi


In [ ]:
#all values containing a key word
keyW = 'substrato alterato , argille'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

substrato alterato , argille
substrato alterato , argille con cristalli di gesso
substrato alterato , argille con gesso
substrato alterato , argille con inclusi di frammenti di calcari cariati , resti di frammenti vegetali
substrato alterato , argille con locali frammenti vegetali
substrato alterato , argille marnose con passate di siltiti
substrato alterato , argille marnoso siltose con locali cristalli di gesso
substrato alterato , argille marnoso siltose con locali cristalli di gesso e con resti vegetali


In [ ]:
#all values containing a key word
keyW = 'substrato , argille'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

substrato , argille con inclusi calcarei e cristalli di gesso
substrato , argille con inclusi di calcari cariati e cristalli di gesso alterato
substrato , argille marnose
substrato , argille marnose a volte siltose


In [ ]:
#all values containing a key word
keyW = 'serpentinoscisto alterato argillificato'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

serpentinoscisto alterato argillificato
serpentinoscisto alterato argillificato a volte sfatto con passate e/o livelli cataclastici
serpentinoscisto alterato argillificato con intercalati livelli di serpentinite
serpentinoscisto alterato argillificato con intercalati livelli di serpentinite a volte litoide cataclastcici
serpentinoscisto alterato argillificato con subordinati livelli di serpentinite cataclastica
serpentinoscisto alterato argillificato con subordinati livelli di serpentinite cataclastica con presenza di talco
serpentinoscisto alterato argillificato con subordinati livelli di serpentinite cataclastici con presenza di talco
serpentinoscisto alterato argillificato sfatto a volte cataclastico moderatamente consistente


In [ ]:
#all values containing a key word
keyW = 'riporto di argilla'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

riporto di argilla
riporto di argilla ciottoli frammenti e clasti
riporto di argilla con sabbia
riporto di argilla con sabbia con frammenti di mattoni
riporto di argilla consistente con frammenti di laterizi
riporto di argilla e laterizi
riporto di argilla e limo con alcuni blocchi di marna
riporto di argilla mista a laterizi


In [ ]:
#all values containing a key word
keyW = 'paleosuolo argillificato'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

paleosuolo argillificato
paleosuolo argillificato , presenza di ciottoli alterati con sacche argillose
paleosuolo argillificato , sabbie e limi parzialmente argillificati
paleosuolo argillificato costituito da ghiaie con ciottoli e blocchi alterati e sfatti con presenza di pseudogleis e ossidi manganesiferi


In [ ]:
#all values containing a key word
keyW = 'cappellaccio argilloso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

cappellaccio argilloso molto plastico con rari ciottoli arenacei
cappellaccio argilloso plastico
cappellaccio argilloso plastico con ciottoli arenacei e marnosi
cappellaccio argilloso plastico con livelli marnosi molto plastici
cappellaccio argilloso plastico con rari ciottoli arenacei
cappellaccio argilloso plastico con subordinati ciottoli arenacei
cappellaccio argilloso plastico con subordinati ciottoli arenacei e marnosi
cappellaccio argilloso sabbioso plastico con ciottoli arenacei


In [ ]:
#all values containing a key word
keyW = 'argilloscisti'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

argilloscisti disarticolati
argilloscisti e argilliti scistose costituito da lamelle a struttura caotica con livelli intercalati e lenti di quarzo fratturati
argilloscisti e argilliti scistose costituito da sottili lamelle a struttura caotica con livelli intercalati e lenti di quarzo fratturati e livelli argillosi di alterazione
argilloscisti e argilliti scistose in livelli più compatti costituito da livelli laminati a tratti a struttura caotica con livelli intercalati e lenti di quarzo fratturati e livelli di alterazione argillosi
argilloscisti e metapeliti scistose alterate , laminate , a struttura caotica con livelli argilloso limosi di alterazione
argilloscisti fogliettati molto teneri
argilloscisti molto alterati
argilloscisti molto alterati con frammenti scistosi e livelli quarzosi
argilloscisti molto alterati sfatti e ossidati


In [ ]:
#all values containing a key word
keyW = 'alternanze di argilla'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

alternanze di argilla fogliettata e gesso cristallino localmente fratturata
alternanze di argilla fogliettata e gesso cristallino localmente fratturata a
alternanze di argilla limosa e limo sabbioso con tracce di marna
alternanze di argilla limoso sabbiosa e limo sabbioso
alternanze di argilla marnosa e marna argillosa con rari livelli di marna calcarea compatta a consistenza lapidea
alternanze di argilla marnosa e marna argillosa e di sabbia fine addensata, a tratti ossidati


In [ ]:
#all values containing a key word
keyW = 'alternanza diargilla'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

alternanza diargilla fogliettata con livelli digesso cristallino da fratturata a compatta
alternanza diargilla marnosa e disabbia fine
alternanza diargilla marnosa moderatamente consistente esabbia fine debolmente limosa da poco a moderatamente addensata


In [ ]:
#all values containing a key word
keyW = 'alternanza di livelli di argilla'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

alternanza di livelli di argilla e livelli di sabbia
alternanza di livelli di argilla e livelli di sabbia limosa
alternanza di livelli di argilla marnosa a volte siltosa consistente e di livelli di sabbia medio fine
alternanza di livelli di argilla marnosa a volte siltosa e sabbia fine moderatamente consistente con laminazioni ossidate
alternanza di livelli di argilla marnosa da litoide a sub-litoide e livello di arenaria litoide , presenza di fratture
alternanza di livelli di argilla marnosa e di sabbia medio fine con presenza di laminazioni ossidate e concrezioni gessose moderatamente consistente
alternanza di livelli di argilla marnosa e di sabbia medio fine da moderatamente addensato a addensato
alternanza di livelli di argilla marnosa e livelli di sabbia medio fine da poco a moderatamente consistente


In [ ]:
#all values containing a key word
keyW = 'alternanza di argilla'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

alternanza di argilla debolmente limosa e sabbiosa con ghiaia fine
alternanza di argilla e ghiaia
alternanza di argilla e ghiaia medio fine in poca matrice argillosa
alternanza di argilla e sabbia fine
alternanza di argilla e serpentinite
alternanza di argilla e/o argilla limoso siltosa e limo sabbioso da moderatamente consistente a consistente
alternanza di argilla limoso sabbiosa con ghiaia medio fine e sabbia medio grossa con poca ghiaia media
alternanza di argilla marnosa e marna
alternanza di argilla marnosa marna argillosa e di sabbia medio fine consistente
alternanza di argilla marnosa molto alterata a volte argillificata e livelli sabbioso fini moderatamente consistente
alternanza di argilla sabbiosa plastica con lenti di arenaria


In [ ]:
#all values containing a key word
keyW = 'limi'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

limi
limi , limi argillosi , limi sabbiosi e sabbie limose con ghiaie eterometriche di pietre verdi e ciottoli
limi , limi argillosi e argille limose
limi , limi argillosi e argille limose consistenti
limi , limi argillosi e limi sabbiosi
limi , limi sabbiosi e limi argillosi addensati con ghiaie eterometriche e ciottoli di roccia metamorfica ; presenza di livelli ossidati
limi , limi sabbiosi e limi argillosi con ghiaia eterometrica a tratti ossidata e alterata e rari piccoli ciottoli
limi , limi sabbiosi e sabbie
limi a tratti argillosi con ciottoli cristallini completamente alterati e argillificati
limi a tratti argillosi e argille limose
limi a tratti debolmente argillosi
limi a tratti debolmente argillosi , presenza di intercalazioni con subordinata ghiaia fine alterata
limi a tratti debolmente argillosi con inclusi alcuni clasti
limi a tratti debolmente sabbiosi
limi a tratti sabbiosi fini
limi a tratti sabbioso fini
limi a tratti sabbioso fini con abbondante ghiaia con presenza 

In [ ]:
#all values containing a key word
keyW = 'limo,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

limo, a tratti debolmente argilloso, consistente; presenza di sporadici noduli
limo, a tratti debolmente sabbioso fine
limo, a tratti debolmente sabbioso, consistente, con abbondante ghiaia medio fine
limo, a tratti sabbioso, consistente , con subordinata ghiaia medio fine
limo, a tratti sabbioso, consistente con abbondante ghiaia
limo, a tratti sabbioso, consistente con subordinata ghiaia medio fine
limo, a tratti sabbioso, consistente, con abbondante ghiaia
limo, a tratti sabbioso, consistente, con subordinata ghiaia medio fine
limo, ciottoli e trovanti
limo, ciottoli e trovanti alterati
limo, con ciottoli e ghiaia, sabbioso
limo, consistente
limo, consistente; presenza di noduli di concrezioni
limo, debolmente sabbioso, da molto consistente a consistente, con rara ghiaia, poligenica, qualche ciottolo, addensata
limo, limo argilloso e argilla limosa
limo, limo argilloso e argilla limosa,addensata, con brecce sparse e presenza di rari ciottoli quarzitici
limo, limo sabbioso e limo arg

In [ ]:
#all values containing a key word
keyW = 'limoa'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

limoa argilloso molto plastico
limoa rgilloso , molto comaptto
limoa rgilloso ,plastico,poco conssitente
limoa rgilloso ,s abbioso con intercalazioni di laminazioni e livelli con sabbia e ghiaia, poligenica
limoa rgilloso a tratti sabbioso fine, molto compatto
limoa rgilloso con ciottoli
limoa rgilloso con granuli, sabbioso fine
limoa rgilloso con inclusi calcescisti
limoa rgilloso con intercalati livelli carboniosi, molto compatto
limoa rgilloso con laminazioni sabbioso fini, ossidate, molto compatto
limoa rgilloso con rare concrezioni calcareo gessose, poco consistente
limoa rgilloso con sabbia e ghiai
limoa rgilloso con subordinate laminazioni sabbioso fine, discreta ossidazione, molto compatto
limoa rgilloso debolmente sabbioso
limoa rgilloso debolmente sabbioso fine con subordinata frazione costituita da clasti e frammenti rocciosi, addensato
limoa rgilloso debolmente sabbioso fine con subordinata frazione ghiaiosa, moderatamente consistente
limoa rgilloso debolmente sabbioso molt

In [ ]:
#all values containing a key word
keyW = 'silt'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

silt a volte argilloso con rari detriti vegetali
silt argilla con ghiaie e sabbie
silt argilla e sabbia fine con poche sparse ghiaie e rari ciottoli
silt argille
silt argille con ghiaie e ciottoli
silt argille con ghiaie medie grosse e sabbie
silt argille con sabbie da fini a grosse
silt argille con sabbie medie e fini
silt argille e sabbie con scarsa ghiaia con intercalazioni di piccoli livelli alterati
silt argille inglobante materiale granulare
silt argille inglobanti materiali grossolani
silt argille sabbie inglobanti ghiaie da fini a medie
silt argille sabbiose con ghiaie e ciottoli sparsi
silt argille sabbiose con sparsi elementi ghiaiosi
silt argille sabbiosi con ghiaie e ciottoli
silt argille sabbiosi ghiaie e ciottoli sparsi
silt argille sabbiosi inglobanti ghiaie medie a volte ciottoli
silt argilloso con livelli sabbiosi
silt argilloso molto fratturato
silt argilloso sabbioso , fossiliferi
silt con livelli di sabbie compatte
silt debolmente argilloso sabbioso fine con clasti 

In [ ]:
#all values containing a key word
keyW = 'argilla,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

argilla, argilla limosa e argilla marnosa debolmente stratificata
argilla, argilla limosa e argilla marnosa plastica, con orizzonti di sabbie ossidate
argilla, argilla limosa e argilla marnosa, debolmente stratificata; presenza di orizzonti sabbiosi e orizzonte di noduli carbonatici
argilla, argilla marnosa e siltite marnosa
argilla, compatta
argilla, compatta plastica
argilla, con ghiaietto e ciottoli, alterati, compatta
argilla, con raro ghiaietto
argilla, consistente
argilla, consistente,
argilla, consistente, passante ad argilla limosa
argilla, inglobante ghiaia e ciottoli
argilla, plastica
argilla, sabbia , ghiaia e trovanti
argilla, sabbia e conglomerato
argilla, sabbia e ghiaia argillosa


In [ ]:
#all values containing a key word
keyW = 'argille'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

argille
argille , argille limose e limi sabbiosi con ghiaie serpentinitiche
argille , marne e siltite marnosa debolmente stratificate e debolmente alterate
argille alterate e ghiaie
argille compatta
argille compatte
argille compatte con lenti sabbiose
argille compatte con presenza di alcuni livelli marnosi molto compatti a consistenza litoide
argille con abbondante ghiaietto , clasti prevalentemente quarzitici
argille con ciottoli
argille con ciottoli alterati
argille con elementi calcarei
argille con fossili
argille con frammenti di materiale di riporto
argille con ghiaia parzialmente plastiche
argille con ghiaia plastiche
argille con intercalazioni marnose
argille con interstrati sabbiosi
argille con lignite
argille con passate torbose
argille con pochi frammenti di arenaria
argille con presenza di frustoli vegetali consistenza media
argille con presenza di ghiaietto esabbie
argille con rari clasti
argille con resti vegetali e conchiglie
argille con sabbie
argille con scaglie di aren

In [ ]:
#all values containing a key word
keyW = 'argillite'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

argillite
argillite a tratti massiccia e fogliettata con intercalazioni di calcare marnoso
argillite alterata e sfatta da poco a moderatamente consistente debole plasticità
argillite alterata ossidata
argillite calcarea
argillite calcarea fogliettata
argillite calcarea fogliettata con patine da ossidazione
argillite calcarea fratturata
argillite calcarea fratturata a livelli milonitizzata
argillite cappellaccio
argillite compatta
argillite completamente alterata e ossidata a volte sfatta moderatamente consistente
argillite completamente alterata e parzialmente sfatta a livelli argillificata da moderatamente consistente a consistente
argillite completamente alterata e sfatta ossidata
argillite completamente alterata e sfatta ossidata molto consistente
argillite con grossi noduli e riempimenti di calcite con presenza di fratture
argillite con intercalazioni di livelli di calcare marnoso
argillite con intercalazioni di siltite
argillite con noduli e grosse vene di calcite con frattura
arg

In [ ]:
#all values containing a key word
keyW = 'argilliti'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

argilliti
argilliti compatte a tratti sub-litoidi , argilliti marnose con intercalazioni di livelli calcareo marnosi litoidi
argilliti compatte con livelli calcareo marnosi litoidi
argilliti compatte e strati calcareo marnosi litoidi
argilliti compatte fogliettate
argilliti compatte fogliettate a tratti sub-litoidi , argilliti marnose con intercalazioni di livelli calcareo marnosi litoidi
argilliti compatte fogliettate con livelli calcareo marnosi litoidi
argilliti con intercalazioni calcaree localmente fratturate
argilliti con intercalazioni calcaree molto fratturate
argilliti con passate di calcare marnoso
argilliti con scarse intercalazioni arenacee
argilliti con venature di quarziti carbonatiche fratturate con media alterazione
argilliti da poco a mediamente cementate a tratti scagliose con intercalazioni di argille molto consistenti
argilliti debolmente arenacee e argilliti marnose con interstrati sabbiosi
argilliti e argilliti argillificate
argilliti fogliettate con intercalazion

In [ ]:
#all values containing a key word
keyW = 'arenaria'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

arenaria
arenaria a debole cementazione
arenaria a debole cementazione debolmente fratturata con livelli marnosi e marnoso arenacei
arenaria a debole e media cementazione fratture con tracce di alterazione
arenaria a debole e media cementazione fratture con tracce di alterazione con presenza di intercalazioni marnose
arenaria a tratti decementata con fratture ossidate
arenaria a tratti fratturata con presenza di alcune intercalazioni arenaceo marnose
arenaria a volte decementata
arenaria a volte parzialmente decementata con frattura ossidata
arenaria alterata
arenaria alterata compatta
arenaria alterata con frattura
arenaria alterata con interlivelli di siltite
arenaria alterata con presenza di frattura
arenaria alterata con presenza di interlivelli marnoso arenacei
arenaria alterata decementata
arenaria alterata decementata con intercalazioni di livelli argillosi e/o argilloso siltosi fratture conriempimento argilloso
arenaria alterata e argillificata plastica
arenaria alterata e dece

In [ ]:
#all values containing a key word
keyW = 'gneiss'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

gneiss
gneiss (blocco di) fratturato mediamente alterato con quarziti in vene
gneiss , gneiss micascistosi con subordinate intercalazioni di calcescisti , vene quarzose o carbonatiche , con presenza di fratture localmente ossidate , intercalazioni di livelli cataclasati con fratture ossidate e riempimenti sabbioso limosi
gneiss , grado di alterazione e fessurazione elevato
gneiss , grado di alterazione e fratturazione basso
gneiss a tratti cariato
gneiss a tratti con fratture ossidate con riempimento argilloso
gneiss a tratti fratturato
gneiss a tratti fratturato a tratti fessurato con tracce di ossidazione
gneiss a tratti fratturato e fessurato con tracce di ossidazione
gneiss a volte fratturato a volte fessurato listato con tracce di ossidazione
gneiss abbondantemente micaceo molto alterato con presenza divuoto e/o cavita abbondante fratturazione
gneiss alterato
gneiss alterato con fratture
gneiss alterato e degradato
gneiss alterato e fratturato
gneiss alterato e parzialmente sfatto

In [ ]:
#all values containing a key word
keyW = 'granito'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

granito
granito a tratti fratturato con tracce di alterazione
granito alterato e degradato con venature sabbiose fratture ossidate a volte con riempimento di sabbia fine limosa
granito biolitico fratturato
granito biolitico molto fratturato localmente in scaglie e frammenti ossidati e alterati con sabbia medio grossa limosa
granito biotitico fratturato
granito biotitico fratturato e ossidato
granito biotitico fratturato localmente sfatto
granito biotitico molto alterato e fratturato con sabbia grossa
granito biotitico molto fratturato
granito completamente sfatto e alterato con limo sabbioso
granito con presenza di frattura
granito fratturato
granito gneissico con presenza di livelli di quarzite con fratture
granito in scaglie e frammenti
granito in scaglie e frammenti alterati e ossidati con sabbia grossa limosa
granito in scaglie e frammenti con sabbia medio grossa limosa
granito in scaglie e frammenti ossidate e alterate con sabbia limosa debolmente plastica
granito localmente fratt

In [ ]:
#all values containing a key word
keyW = 'kinzigite'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

kinzigite a tratti alterata e argillificata molto fratturata con presenza di riempimento argilloso e ossidazione
kinzigite a tratti alterata localmente argillificata con presenza di fratture
kinzigite alterata con presenza di frattura
kinzigite alterata con presenza di fratture
kinzigite con mica debolmente alterata con tracce di ossidazione e riempimento argilloso con presenza di fratture
kinzigite da fratturata a molto fratturata localmente in scaglie e frammenti
kinzigite fratturata
kinzigite gneissica a tratti alterata argillificata fratturata
kinzigite gneissica alterata a tratti molto alterata sfatta e argillificata con presenza di discontinuita'
kinzigite gneissica con presenza di vene e noduli di quarzo discontinuita' fratture alterate a volte argillificate con livelli di scaglie e frammenti
kinzigite gneissica con presenza di vene noduli di quarzo fratture con riempimento argilloso e scaglie di roccia sfatta
kinzigite gneissica con vene e noduli di quarzo a tratti alterata arg

In [ ]:
#all values containing a key word
keyW = 'ciottoli'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

ciottoli
ciottoli , a tratti blocchi e ghiaia eterometrica in matrice fine sabbiosa debolmente limosa addensata
ciottoli , blocchi , clasti e frammenti rocciosi alterati con sabbia debolmente limosa
ciottoli , ghiaia compatta con argilla
ciottoli , ghiaia compatta e sabbia
ciottoli , ghiaia e sabbia
ciottoli , ghiaia e sabbia , materiale sciolto moderatamente alterato
ciottoli , ghiaia e sabbia grossolana
ciottoli , ghiaia e sabbia localmente limosa
ciottoli , ghiaia eterometrica poligenica e sabbia limosa
ciottoli , piccoli trovanti e ghiaia
ciottoli , rari blocchi , frammenti e clasti rocciosi alternati a livelli fini
ciottoli , trovante , spezzame roccioso con livelli intercalati di ghiaia in abbondante sabbia
ciottoli , trovanti , argilla
ciottoli , trovanti abbondanti , ghiaia e ghiaietto con sabbia medio fine limosa molto addensata e parzialmente cementata
ciottoli , trovanti e ghiaia grossa sabbiosi debolmente limosi
ciottoli alterati
ciottoli alterati con ghiaia e sabbia in mat

In [ ]:
keyW = 'sabbie'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

sabbie
sabbie , addensate, con presenza di locale ghiaietto
sabbie , debolmente limose
sabbie , debolmente limose; presenza di rari clasti
sabbie , ghiaie e ciottoli , localmente debolmente limose
sabbie , ghiaietti con presenza di clasti
sabbie a tratti debolmente limose con ghiaie e piccoli ciottoli; presenza di livello di sabbie eterometriche sciolte
sabbie a tratti limose con ghiaia e sporadici ciottoli
sabbie a volte con tracce di torba
sabbie addensate debolmente limose
sabbie addensate e sabbie arenacee
sabbie addensate e sabbie arenacee poco cementate
sabbie addensate e sabbie limose con ghiaie e ciottoli di pietre verdi
sabbie arenacee in livelli cementati o disgregati
sabbie argillo limose
sabbie argillose
sabbie argillose , presenza di ciottoli
sabbie argillose con ghiaie e ciottoli localmente molto alterati
sabbie argillose parzialmente compatte
sabbie argilloso limose
sabbie argilloso limose debolmente ghiaiose compatte
sabbie argilloso limose debolmente ghiaiose con rari 

In [ ]:
keyW = 'clasti'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

clasti
clasti , blocchi e frammenti di calcescisti in matrice da limoso sabbiosa a sabbioso limosa , debole ossidazione , da discreta a debole alterazione
clasti , ciottoli e blocchi rocciosi frammentati prevalentemente di calcare marnoso , in scarsa matrice fine siltosa debolmente sabbiosa con intercalati livelli a matrice fine prevalente
clasti , ciottoli e subordinati blocchi rocciosi frammentati di calcare marnoso , scarsa matrice fine siltosa debolmente sabbiosa
clasti , ciottoli e subordinati blocchi rocciosi frammentati in matrice fine siltoso sabbiosa
clasti , ciottoli e subordinati blocchi rocciosi frammentati in scarsa matrice fine siltosa
clasti , ciottoli eterometrici e subordinati blocchi in matrice siltosa debolmente sabbioso fine a livelli prevalente
clasti , frammenti di pietre verdi e rari ciottoli con sabbia debolmente limosa
clasti , frammenti di pietre verdi e rari ciottoli con sabbia medio grossolana debolmente limosa
clasti , frammenti e ciottoli lapidei di calces

In [ ]:
keyW = 'frammenti clasti'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

frammenti clasti arenacei in matrice abbondante sabbiosa debolmente limosa molto ossidato alterato
frammenti clasti di gneiss e/o micascisti in sabbia limosa discreta alterazione ossidazione
frammenti clasti di gneiss in sabbia localmente ossidata debolmente alterata moderatamente addensata
frammenti clasti di marna siltosa in matrice abbondante limoso argillosa da poco consistente a plastica inglobante blocchi disarticolati di marna siltosa
frammenti clasti di marna siltosa in matrice abbondante limoso sabbiosa da poco consistente a plastica inglobante blocchi di marna siltosa
frammenti clasti dicalcescisti in scarsa matrice limoso sabbiosa
frammenti clasti marnoso arenacei in matrice abbondante limoso argillosa alterata ossidata moderatamente consistente
frammenti clasti marnoso arenacei in matrice abbondante sabbioso limosa alterata ossidata
frammenti clasti rocciosi con alterazione ossidazione
frammenti clasti rocciosi con limo sabbioso ciottoloso con discreta ossidazione
frammenti

In [ ]:
key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]

'livelli di'

In [ ]:
#all values containing a key word
keyW = 'arenaria'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

In [ ]:
#all values containing a key word
keyW = 'dolomia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

dolomia alterata e sfatta a tratti argillificata , elevato grado di ossidazione
dolomia alterata e sfatta con subordinate intercalazioni calcaree e passate litoidi
dolomia alterata e sfatta con subordinati livelli litoidi , discreto grado di ossidazione
dolomia alterata ossidata e argillificata
dolomia calcarea a tratti intensamente cariata , localmente brecciata cementata
dolomia calcarea compatta , con livelli di milonite , fratture ossidate con riempimento
dolomia calcarea compatta a tratti intensamente ossidata e alterata con presenza di livelli completamente milonitizzati
dolomia calcarea compatta con presenza di fratture ossidate con riempimento
dolomia calcarea con livelli milonitizzati e con presenza di fratture ossidate con riempimento
dolomia calcarea intensamente cataclasata passante a milonite , sabbia limoso argillosa inglobante elementi lapidei
dolomia con molte vene e piccole lenti calcitiche , fratture con riempimento
dolomia con vene biotitiche , roccia orientata , fra

In [ ]:
#all values containing a key word
keyW = 'marna'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

marna
marna , poco sabbiosa
marna ; presenza di fratture
marna a cemento argilloso con livelli marnoso siltosi
marna a consistenza lapidea fratturata
marna a livelli debolmente siltosa consistente
marna a livelli sabbiosi fini compatta
marna a livelli siltosa molto consistente
marna a livelli siltosa, molto consistente
marna a struttura compatta
marna a struttura fogliettata con noduli di gesso con presenza di fratture
marna a struttura fogliettata con noduli di gesso localmente con presenza di fratture con limo di riempimento
marna a struttura fogliettata poco consistente
marna a tratti argillosa
marna a tratti argillosa con presenza di alcuni livelli di argilla marnosa
marna a tratti argillosa con rare laminazioni limoso sabbiose
marna a tratti argillosa rare laminazioni sabbiose
marna a tratti argillosa rare laminazioni sabbiose frazione fine micacea
marna a tratti argillosa rare laminazioni sabbiose sabbia fine micacea
marna a tratti argillosa siltosa sabbiosa
marna a tratti compat

In [ ]:
#all values containing a key word
keyW = 'marne'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

marne
marne , marne argillose e siltiti , localmente sabbiose
marne alterate
marne alterate completamente sfatte con abbondante matrice fine
marne alterate con piccoli interlivelli arenacei calcarei
marne alterate frammentate a limo sabbioso
marne alterate sfatte con matrice fine
marne arenacea
marne arenacee
marne arenacee compatte
marne arenacee con alcune fratture
marne arenacee con frattura
marne arenacee con frattura a riempimento argilloso
marne arenacee con frattura a riempimento argilloso detritico
marne arenacee con frattura a riempimento sabbioso
marne arenacee con fratture
marne arenacee con piccole intercalazioni di arenarie
marne arenacee con presenza di alcune fratture
marne arenacee con presenza di alcune fratture a riempimento argilloso
marne arenacee con presenza di alcune fratture a riempimento sabbioso
marne arenacee debolmente siltose molto compatte
marne arenacee e marne in strati con intercalazioni arenacee con presenza di alterazione e ossidazione
marne arenacee 

In [ ]:
#all values containing a key word
keyW = 'ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

Output streaming troncato alle ultime 5000 righe.
ghiaia con ciottoli in abbondante matrice sabbiosa alternata a livelli con ghiaia e ciottoli con scarsa matrice sabbiosa
ghiaia con ciottoli in abbondante matrice, presenza di alterazione
ghiaia con ciottoli in abbondante matrice; presenza di alterazione
ghiaia con ciottoli in alternanza con livelli di sabbia grossa
ghiaia con ciottoli in amtrice limosa e limoso sabbiosa,addensata; presenza di clasti alterati
ghiaia con ciottoli in amtrice limosa sabbiosa, addensata
ghiaia con ciottoli in amtrice sabbiosa a tratti limosa,presenza di cementazione,addensata
ghiaia con ciottoli in amtrice sabbiosa addensata
ghiaia con ciottoli in amtrice sabbiosa e sabbioso limosa,addensata
ghiaia con ciottoli in amtrice sabbiosa limosa , addensata
ghiaia con ciottoli in amtrice sabbiosa limosa addensata, presenza di fenomeni di alterazione e/o ossidazione della matrice
ghiaia con ciottoli in amtrice sabbiosa limosa prevalente in intervalli debolmente lami

In [ ]:
#all values containing a key word
keyW = 'alluvione ghiaiosa'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

alluvione ghiaiosa


In [ ]:
#all values containing a key word
keyW = 'alluvione ghiaioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

alluvione ghiaioso ciottolosa
alluvione ghiaioso ciottolosa in matrice prevalentemente limosa
alluvione ghiaioso ciottolosa in scarsa matrice sabbiosa
alluvione ghiaioso sabbiosa addensata con frazione fine limoso argillosa
alluvione ghiaioso sabbiosa fine


In [ ]:
#all values containing a key word
keyW = 'alternanza di ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

alternanza di ghiaia ciottolosa sabbiosa e lenti di sabbia ghiaiosa
alternanza di ghiaia con sabbia e strati di ciottoli con ghiaia sabbiosa
alternanza di ghiaia eterometrica con sabbia e sabbia medio fine ghiaiosa con presenza di clasti di granito e quarziti alterati e ossidati
alternanza di ghiaia sabbiosa alterata e ghiaia sabbiosa limosa alterata, addensata
alternanza di ghiaia sabbiosa e sabbia grossolana


In [ ]:
#all values containing a key word
keyW = 'alternanza di livelli di ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

alternanza di livelli di ghiaia e ciottoli con sabbia limosa da sciolta a debolmente addensata e livelli sabbia limosa con ghiaia e ciottoli ; presenza di ciottoli alterati con tracce di cementazione carbonatica
alternanza di livelli di ghiaia eterometrica a tratti sfatta limosa ciottoli e blocchi


In [ ]:
#all values containing a key word
keyW = 'alternanze di ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

alternanze di ghiaia con sabbia e frammenti di mattoni argilla e limo debolmente sabbiosi plastiche addensate
alternanze di ghiaia con sabbia limosa , con strati di sabbia limoso argillosa con ghiaia eterometrica , con presenza di ciottoli sparsi
alternanze di ghiaia eterometrica con sabbia da debolmente limosa a localmente limoso argillosa e sabbia limosa con ghiaia medio fine , con presenza di alcuni ciottoli


In [ ]:
#all values containing a key word
keyW = 'deposito alluvionale costituito da ghiaie'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

deposito alluvionale costituito da ghiaie , ciottolose e debolmente sabbiose
deposito alluvionale costituito da ghiaie , sabbiose
deposito alluvionale costituito da ghiaie , sabbiose, addensate
deposito alluvionale costituito da ghiaie , sabbiose, intercalati da livelli sabbioso limosi
deposito alluvionale costituito da ghiaie a tratti ciottolose
deposito alluvionale costituito da ghiaie grossolane, ciottolose e sabbiose
deposito alluvionale costituito da ghiaie medie , sabbiose e limose
deposito alluvionale costituito da ghiaie medie e grossolane sabbiose
deposito alluvionale costituito da ghiaie sabbiose ciottolose


In [ ]:
#all values containing a key word
keyW = 'deposito alluvionale ghiaioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

deposito alluvionale ghiaioso
deposito alluvionale ghiaioso ciottoloso e sabbioso
deposito alluvionale ghiaioso ciottoloso molto addensato inglobante intercalazioni sabbioso limose
deposito alluvionale ghiaioso ciottoloso sabbioso
deposito alluvionale ghiaioso con intercalazioni di livelli limosi
deposito alluvionale ghiaioso in matrice da argillosa a argilloso limosa , localmente abbondante
deposito alluvionale ghiaioso molto addensato e cementato
deposito alluvionale ghiaioso sabbioso
deposito alluvionale ghiaioso sabbioso , ciottoloso
deposito alluvionale ghiaioso sabbioso ciottoloso
deposito alluvionale ghiaioso sabbioso ciottoloso addensato
deposito alluvionale ghiaioso sabbioso ciottoloso molto addensato , inglobante localmente blocchi
deposito alluvionale ghiaioso sabbioso con intercalazioni sabbioso limose
deposito alluvionale ghiaioso sabbioso con livelli parzialmente cementati
deposito alluvionale ghiaioso sabbioso inglobante ciottoli
deposito alluvionale ghiaioso sabbioso mo

In [ ]:
#all values containing a key word
keyW = 'deposito ghiaiosi'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

deposito ghiaiosi con ciottoli in matrice eterometrica prevalentemente sabbiosa , mediamente addensata
deposito ghiaiosi grossolani


In [ ]:
#all values containing a key word
keyW = 'deposito ghiaioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

deposito ghiaioso ciottolosi
deposito ghiaioso ciottoloso inglobante blocchi
deposito ghiaioso ciottoloso parzialmente cementato
deposito ghiaioso con ciottoli eterometrici
deposito ghiaioso con ciottoli eterometrici in matrice scarsa
deposito ghiaioso con ciottoli in matrice eterometrica prevalentemente sabbiosa , mediamente addensata
deposito ghiaioso con scarsa matrice sabbiosa grossolana , presenza di ciottoli
deposito ghiaioso sabbiosi
deposito ghiaioso sabbioso
deposito ghiaioso sabbioso ciottoloso con blocchi ; presenza di locali intercalazioni sabbioso limose
deposito ghiaioso sabbioso ciottoloso con trovanti
deposito ghiaioso sabbioso ciottoloso localmente inglobante blocchi
deposito ghiaioso sabbioso inglobante blocchi e intercalazioni sabbioso limose


In [ ]:
#all values containing a key word
keyW = 'detrito a matrice di ghiaietto'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

detrito a matrice di ghiaietto
detrito a matrice di ghiaietto e poca argilla
detrito a matrice di ghiaietto e poca sabbia
detrito a matrice di ghiaietto e sabbia


In [ ]:
#all values containing a key word
keyW = 'detrito ghiaioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

detrito ghiaioso con livelli sabbiosi e limo argillosi
detrito ghiaioso grossolano con ciottoli eterometrici
detrito ghiaioso grossolano con scarsa matrice sabbiosa
detrito ghiaioso medio grossolano con scarsa matrice , ciottoli poligenici eterometrici
detrito ghiaioso sabbioso
detrito ghiaioso sabbioso con ciottoli
detrito ghiaioso sabbioso con livelli di limo argilloso


In [ ]:
#all values containing a key word
keyW = 'distruzione ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

distruzione ghiaia ciottoli limo sabbioso e frammenti dicalcescisto
distruzione ghiaia ciottoli sabbia limosa e frammenti di calcescisto
distruzione ghiaia ciottoli sabbia limosa e frammenti dicalcescisto
distruzione ghiaia con ciottoli
distruzione ghiaia con ciottoli e blocchi a scarsa matrice fine clasti eterometrici cristallini
distruzione ghiaia con sabbia
distruzione ghiaia limo sabbioso e frammenti di calcescisto
distruzione ghiaia sabbia limosa e frammenti dicalcescisto


In [ ]:
#all values containing a key word
keyW = 'livello ghiaioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

livello ghiaioso
livello ghiaioso cementato
livello ghiaioso ciottoloso
livello ghiaioso con ciottoli con poca matrice sabbiosa
livello ghiaioso con limo
livello ghiaioso con poca matrice sabbiosa
livello ghiaioso con scarsa matrice limoso argillosa
livello ghiaioso debolmente sabbioso
livello ghiaioso grossolano con blocchi
livello ghiaioso in matrice abbondante limoso sabbiosa
livello ghiaioso in matrice marnoso argillosa
livello ghiaioso in matrice scarsa limoso sabbiosa


In [ ]:
#all values containing a key word
keyW = 'materiale di riporto ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

materiale di riporto ghiaia con sabbia a volte debolmente limosa con abbondanti elementi litoidi grossolani
materiale di riporto ghiaia con sabbia localmente debolmente limosa con scarsi piccoli ciottoli
materiale di riporto ghiaia e ciottoli raramente debolmente sabbiosi
materiale di riporto ghiaia e ghiaietto
materiale di riporto ghiaia eterometrica con sabbia e ciottoli
materiale di riporto ghiaia eterometrica con sabbia eterometrica con presenza di ciottoli
materiale di riporto ghiaia eterometrica debolmente sabbiosa
materiale di riporto ghiaia eterometrica sabbiosa con presenza di ciottoli
materiale di riporto ghiaia grossolana con ciottoli in alternanza con livelli di ghiaie medio fini debolmente sabbiose
materiale di riporto ghiaia grossolana e ciottoli
materiale di riporto ghiaia grossolana e ciottoli a volte debolmente sabbiosi
materiale di riporto ghiaia media ghiaietto e sabbia
materiale di riporto ghiaia medio grossa a volte debolmente sabbiosa con presenza di ciottoli


In [ ]:
#all values containing a key word
keyW = 'materiale di riporto ghiaioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

materiale di riporto ghiaioso
materiale di riporto ghiaioso ciottoloso
materiale di riporto ghiaioso con mattoni e laterizi misti a sabbia debolmente limosa
materiale di riporto ghiaioso sabbioso
materiale di riporto ghiaioso sabbioso con frazione limosa
materiale di riporto ghiaioso sabbioso con tracce di laterizi


In [ ]:
#all values containing a key word
keyW = 'riporto costituito prevalentemente da ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

riporto costituito prevalentemente da ghiaia con limo sabbioso e rari frammenti lateritici
riporto costituito prevalentemente da ghiaia e ciottoli in scarsa matrice limoso-sabbiosa, subordinati frammenti lateritici
riporto costituito prevalentemente da ghiaia sabbiosa con ciottoli sparsi e frammenti lateritici
riporto costituito prevalentemente da ghiaia sabbiosa con frammenti lateritici
riporto costituito prevalentemente da ghiaia sabbiosa con frammenti lateritici, metallici e plastici
riporto costituito prevalentemente da ghiaia sabbiosa con subordinati frammenti lateritici


In [ ]:
#all values containing a key word
keyW = 'riporto di ghiaietto'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

riporto di ghiaietto con limi argillosi
riporto di ghiaietto con sabbia
riporto di ghiaietto ghiaia e ciottoli con sabbia limosa
riporto di ghiaietto sabbie limose materiale fine frammenti di mattoni e scorie


In [ ]:
#all values containing a key word
keyW = 'terreno di riporto prevalentemente ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

terreno di riporto prevalentemente ghiaia sabbiosa
terreno di riporto prevalentemente ghiaia sabbiosa con subordinati frustoli vegetali frammenti di laterizi e ciottoli


In [ ]:
#all values containing a key word
keyW = 'terreno di riporto prevalentemente ghiaioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

terreno di riporto prevalentemente ghiaioso ciottoloso
terreno di riporto prevalentemente ghiaioso ciottoloso con laterizi e mattoni
terreno di riporto prevalentemente ghiaioso ciottoloso con sabbia
terreno di riporto prevalentemente ghiaioso ciottoloso con sabbia limosa
terreno di riporto prevalentemente ghiaioso ciottoloso con trovanti
terreno di riporto prevalentemente ghiaioso grossolano
terreno di riporto prevalentemente ghiaioso limoso con ciottoli e frammenti di laterizi
terreno di riporto prevalentemente ghiaioso sabbioso
terreno di riporto prevalentemente ghiaioso sabbioso con frammenti di laterizi
terreno di riporto prevalentemente ghiaioso sabbioso con laterizi
terreno di riporto prevalentemente ghiaioso sabbioso con rari frammenti di laterizi


In [ ]:
#all values containing a key word
keyW = 'terreno diriporto ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

terreno diriporto ghiaia con sabbia eciottoli
terreno diriporto ghiaia con sabbia eciottoli con intercalazioni di limo


In [ ]:
#all values containing a key word
keyW = 'terreno diriporto ghiaioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

terreno diriporto ghiaioso ciottoloso
terreno diriporto ghiaioso con frammenti di laterizi passante a limosa sabbiosa
terreno diriporto ghiaioso limoso con frammenti di laterizi
terreno diriporto ghiaioso limoso con rari ciottoli
terreno diriporto ghiaioso sabbioso
terreno diriporto ghiaioso sabbioso con rari ciottoli


In [ ]:
#all values containing a key word
keyW = 'terreno ghiaioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

terreno ghiaioso
terreno ghiaioso argilloso
terreno ghiaioso argilloso debolmente addensato
terreno ghiaioso ciottoloso
terreno ghiaioso ciottoloso grossolano, detriti
terreno ghiaioso sabbioso
terreno ghiaioso sabbioso, talora limoso con frammenti lateritici


In [ ]:
#all values containing a key word
keyW = 'terreno rimaneggiato costituito da ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

terreno rimaneggiato costituito da ghiaia ciottolosa
terreno rimaneggiato costituito da ghiaia con sabbia , scarsamente addensato , presenza di radici
terreno rimaneggiato costituito da ghiaia con sabbia debolmente limosa con resti vegetali, ciottolosa, moderatamente addensata
terreno rimaneggiato costituito da ghiaia da limosa ad argillosa, poco addensata
terreno rimaneggiato costituito da ghiaia e ciottoli
terreno rimaneggiato costituito da ghiaia limosa con frammenti laterizi
terreno rimaneggiato costituito da ghiaia sabbiosa
terreno rimaneggiato costituito da ghiaia sabbiosa con ciottoli sparsi
terreno rimaneggiato costituito da ghiaia sabbioso limosa alternata a livelli di terreno vegetale con radici moderatamente addensata


In [ ]:
#all values containing a key word
keyW = 'terreno rimaneggiato ghiaioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

terreno rimaneggiato ghiaioso ciottoloso
terreno rimaneggiato ghiaioso ciottoloso con sabbia
terreno rimaneggiato ghiaioso ciottoloso con sabbia, moderatamente addensato
terreno rimaneggiato ghiaioso sabbioso
terreno rimaneggiato ghiaioso sabbioso a tratti debolmente limoso
terreno rimaneggiato ghiaioso sabbioso limoso addensato


In [ ]:
#all values containing a key word
keyW = 'riporto di ciottoli'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

riporto di ciottoli
riporto di ciottoli con ghiaia limosa debolmente sabbiosa
riporto di ciottoli con ghiaia sabbiosa con frammenti di laterizi
riporto di ciottoli e argilla ghiaia
riporto di ciottoli e blocchi con sabbia e ghiaia sciolta
riporto di ciottoli e clasti in matrice prevalentemente sabbiosa
riporto di ciottoli e ghiaia frammista a detriti
riporto di ciottoli e ghiaia frammista a sabbia limosa
riporto di ciottoli e grossi frammenti di mattoni
riporto di ciottoli e limo sabbioso con abbondante frazione mattoni
riporto di ciottoli e sabbia con ghiaia e rari frammenti di laterizi
riporto di ciottoli e sabbia limosa
riporto di ciottoli e trovanti con ghiaia e sabbia poco addensato


In [ ]:
#all values containing a key word
keyW = 'ciottoli,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

ciottoli, detriti, limo e sabbia
ciottoli, ghiaia debolmente limoso sabbiosa
ciottoli, ghiaia e sabbia
ciottoli, ghiaia e sabbia limosa
ciottoli, ghiaie e sabbia


In [ ]:
#all values containing a key word
keyW = 'ciottolo'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

ciottolo
ciottolo alterato
ciottolo calcareo selcifero
ciottolo di calcare selcifero
ciottolo di metagabbro
ciottolo di micascisto
ciottolo di quarzite
ciottolo di quarzoscisto
ciottolo di roccia metamorfica scistosa
ciottolo di serpentinoscisto
ciottolo dicalcescisto fratturato
ciottolo diquarzite
ciottolo e ghiaia con sabbia
ciottolo e trovante
ciottolo fratturato
ciottolo gneissico
ciottolo granitico
ciottolo litoide debolmente alterato
ciottolo quarzitico
ciottolo serpentinitico
ciottolo serpentinitico fratturato


In [ ]:
#all values containing a key word
keyW = 'alluvioni ciottoli'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

alluvioni ciottoli con sabbia
alluvioni ciottoli in scarsa matrice sabbiosa


In [ ]:
#all values containing a key word
keyW = 'alluvioni ciottolose'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

alluvioni ciottolose con poca sabbia
alluvioni ciottolose con suolo di alterazione
alluvioni ciottolose con trovanti
alluvioni ciottolose ghiaiose
alluvioni ciottolose ghiaiose con alcuni livelli limosi
alluvioni ciottolose ghiaiose sabbiose
alluvioni ciottolose ghiaiose sabbiose e terreno vegetale


In [ ]:
#all values containing a key word
keyW = 'alluvione ciottolosa'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Ghiaia'

alluvione ciottolosa
alluvione ciottolosa ciottoli e trovanti in scarsa matrice sabbiosa medio fine
alluvione ciottolosa ciottoli in scarsa matrice ghiaioso sabbiosa
alluvione ciottolosa ciottoli in scarsa matrice ghiaioso sabbiosa passante a sabbioso limosa
alluvione ciottolosa ciottoli in scarsa matrice sabbioso ghiaiosa
alluvione ciottolosa con ciottoli in matrice ghiaioso argillosa
alluvione ciottolosa con frammenti in matrice ghiaiosa medio fine
alluvione ciottolosa con presenza di ghiaia
alluvione ciottolosa con scarsa matrice sabbiosa
alluvione ciottolosa grossolana in scarsa matrice sabbiosa
alluvione ciottolosa in matrice ghiaioso sabbiosa
alluvione ciottolosa in matrice ghiaioso sabbiosa scarsa
alluvione ciottolosa in matrice prevalente ghiaiosa sabbiosa
alluvione ciottolosa in matrice prevalente ghiaioso sabbiosa
alluvione ciottolosa in matrice prevalentemente ghiaiosa con presenza di frazione sabbiosa fine in subordine
alluvione ciottolosa in matrice prevalentemente ghiaios

In [ ]:
#all values containing a key word
keyW = 'ghiaia,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

ghiaia, a tratti con sabbia debolmente limosa, discreto grado di ossidazione e alterazione, addensata
ghiaia, argillificata con sabbia e limo, rari ciottoli
ghiaia, ciottoli e argilla
ghiaia, ciottoli e blocchi in amtrice limoso sabbiosa; addensato, presenza di alterazione
ghiaia, ciottoli e sabbia
ghiaia, ciottoli e sabbia a tratti debolmente limosa consistente
ghiaia, ciottoli e sabbia con limo
ghiaia, ciottoli e sabbia, limosa
ghiaia, ciottoli sparsi con sabbia limosa
ghiaia, ciottoli sparsi, sabbia con intercalazioni limose compatte
ghiaia, ciottoli, sabbia e ghiaietto con intercalazioni di limo sabbioso
ghiaia, ciottolosa, in abbondante e/o prevalente matrice fine sabbioso-siltosa, molto addensata
ghiaia, con clasti
ghiaia, con sabbia medio fine, limosa
ghiaia, eterometrica, poligenica, alterata, con sabbia da limosa a con limo
ghiaia, ghiaietto con molta sabbia e intercalazioni di livelli limosi sabbiosi
ghiaia, ghiaietto con sabbia addensata
ghiaia, ghiaietto con sabbia argillos

In [ ]:
#all values containing a key word
keyW = 'ghiaie'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

ghiaie
ghiaie , ciottolose, inglobanti subordinati livelli sabbiosi
ghiaie , debolmente sabbiose, addensate
ghiaie , ghiaietto sciolti con legno
ghiaie , sabbiose, addensate
ghiaie a elementi quarzosi e serpentinosi
ghiaie addensate con livelli sabbioso limosi
ghiaie addensate inglobante grossi blocchi
ghiaie addensate, intercalate da livelli sabbioso limosi
ghiaie alterate e sabbie
ghiaie argillose
ghiaie argillose con piccoli ciottoli
ghiaie argillose e argille ghiaiose debolmente plastiche
ghiaie argillose sciolte con grossi ciottoli parzialmente cementati
ghiaie argilloso sabbiose con ciottoli
ghiaie cementate calcareo
ghiaie cementate con ciottoli
ghiaie ciottoli e blocchi con matrice ghiaioso sabbiosa
ghiaie ciottoli e sabbie
ghiaie ciottoli poligenici e sabbia grossolana debolmente limosa
ghiaie ciottoli poligenici e sabbie
ghiaie ciottoli poligenici e sabbie debolmente limose
ghiaie ciottoli sabbie con argille silt
ghiaie ciottoli sparsi sabbie in matrice silt argillosa sabbios

In [ ]:
#all values containing a key word
keyW = 'ghiaie,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

ghiaie, ciottoli, blocchi e tovanti in poca matrice sabbiosa, debolmente limosa
ghiaie, in scarsa matrice sabbiosa fine; presenza di clasti di varia natura
ghiaie, sabbia e rari ciottoli
ghiaie, sabbiose, addensate
ghiaie, sabbioso limose intercalate da livelli limoso sabbiosi


In [ ]:
#all values containing a key word
keyW = 'ghiaietto'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

ghiaietto
ghiaietto , ghiaia
ghiaietto , ghiaia , ciottoli e sabbia
ghiaietto , ghiaia e ciottoli in amtrice sabbiosa da debolmente limosa a limosa; presenza di ghiaia e ciottoli alterati e locali elementi di conglomerato
ghiaietto , ghiaia e rari ciottoli in amtrice sabbiosa debolment elimosa
ghiaietto , sabbia con tracce di argilla , ghiaia con ciottoli
ghiaietto , sabbia e alcuni ciottoli
ghiaietto , sabbia e ghiaia
ghiaietto argilla
ghiaietto argilloso
ghiaietto argilloso compattocon strati di argilla
ghiaietto argilloso sabbioso
ghiaietto calcareo
ghiaietto ciottoli
ghiaietto ciottoli diquarzo ecalcite in livelli alterati
ghiaietto compatta con argilla
ghiaietto compatto
ghiaietto compatto con alcuni strati sciolti
ghiaietto compatto con arenaria
ghiaietto compatto con argilla
ghiaietto compatto con argilla sabbiosa
ghiaietto compatto con piccoli strati di argilla
ghiaietto compatto con poca argilla
ghiaietto compatto con trovanti
ghiaietto compatto terroso
ghiaietto con abbondant

In [ ]:
#all values containing a key word
keyW = 'ghiaione'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

ghiaione
ghiaione , ghiaia , sabbia con trovanti
ghiaione argilloso
ghiaione compatto
ghiaione compatto con argilla
ghiaione compatto con ciottoli
ghiaione compatto con trovanti
ghiaione con argilla
ghiaione con ciottoli
ghiaione con ciottoli con sabbia
ghiaione con ciottoli e trovanti
ghiaione con molta argilla
ghiaione con poca sabbia
ghiaione con sabbia
ghiaione con sabbia e ghiaietto
ghiaione con sabbia e trovanti
ghiaione con sabbione
ghiaione con trovanti
ghiaione da sciolto a compatto
ghiaione e argilla
ghiaione e conglomerati
ghiaione e ghiaia
ghiaione e ghiaietto con poco sabbione
ghiaione e sabbia
ghiaione e trovanti
ghiaione ghiaia e sabbia
ghiaione grosso con ghiaia
ghiaione misto a sabbia argillosa
ghiaione poco compatto
ghiaione sabbioso con argilla
ghiaione sciolto
ghiaione serpentinoso
ghiaione trovanti


In [ ]:
#all values containing a key word
keyW = 'gesso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=202
    key_list[position]='Gesso, anidrite'

gesso
gesso con cristalli con rari livelli marnoso siltosi
gesso con cristalli con subordinati livelli marnoso siltosi
gesso con intercalazioni marnose
gesso cristallino
gesso cristallino con intercalati livelli argilloso marnosi
gesso cristallino con intercalazioni di livelli di marna siltosa
gesso cristallino con piccoli livelli marnosi con presenza di fratture
gesso cristallino con presenza di livello marnoso
gesso cristallino con rari livelli microcristallini
gesso cristallino con subordinati livelli siltoso marnosi
gesso cristallino con subordinati livelli siltoso marnosi con livelli fratturati
gesso cristallino con venature di marna siltosa frattura con riempimento argilloso
gesso cristallino con venature di marna siltosa fratture con riempimento argilloso
gesso cristallino e argilla
gesso cristallino e compatto
gesso cristallino e compatto con rari livelli marnosi
gesso cristallino e subordinati clasti in matrice marnosa a volte abbondante rare fratture con riempimento argilloso

In [ ]:
#all values containing a key word
keyW = 'gessi'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=202
    key_list[position]='Gesso, anidrite'

gessi
gessi e subordinati livelli di anidrite con passate molto sfatte e alterate con fratture
gessi prevalenti con livelli di anidrite molto fratturati a volte sfatti fratture con riempimento di materiale argillificato
gessi prevalenti e livelli di anidriti con passate molto sfatte e fratture a volte con riempimento di materiale argillificato


In [ ]:
#all values containing a key word
keyW = 'strati dighiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

strati dighiaia in alternanza con strati dighiaia conargilla


In [ ]:
#all values containing a key word
keyW = 'riporto dighiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

riporto dighiaia eghiaietto con frammenti di mattoni
riporto dighiaia esabbia localmente debolmente limose
riporto dighiaia poligenica eterometrica ciottoli esabbia medio grossa limosa molto addensata


In [ ]:
#all values containing a key word
keyW = 'riporto dighiaie'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

riporto dighiaie esabbie con laterizi


In [ ]:
#all values containing a key word
keyW = 'riporto ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

riporto ghiaia e sabbia
riporto ghiaia e sabbia limosa
riporto ghiaia eterometrica poligenica ciottoli con sabbia medio grossa con presenza di frammenti laterizi
riporto ghiaia eterometrica poligenica con sabbia medio grossa limosa con presenza di ciottolo
riporto ghiaia fine con sabbia
riporto ghiaia ghiaietto ciottoli e limo sabbioso
riporto ghiaia ghiaietto e sabbia con presenza di trovante
riporto ghiaia grossa e ciottoli con sabbia limosa
riporto ghiaia media grossa con sabbia debolmente limosa
riporto ghiaia medio fine limo sabbioso passante asabbia limosa
riporto ghiaia medio grossa con sabbia


In [ ]:
#all values containing a key word
keyW = 'orizzonti ghiaiosi'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

orizzonti ghiaiosi con ciottoli


In [ ]:
#all values containing a key word
keyW = 'prevalente ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

prevalente ghiaia con sabbia e ciottoli sparsi intercalazioni di sabbia con ghiaia e livelli con abbondanti ciottoli a tratti con presenza di frazione limosa rari strati di argilla con ghiaia
prevalente ghiaia con sabbia e ciottoli sparsi intercalazioni di sabbia con ghiaia e livelli con abbondanti ciottoli con presenza di frazione limosa rari strati di argilla con ghiaia


In [ ]:
#all values containing a key word
keyW = 'rilevato ghiaioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

rilevato ghiaioso - ciottoloso
rilevato ghiaioso sabbioso
rilevato ghiaioso sabbioso con ciottoli
rilevato ghiaioso sabbioso con ciottoli sparsi


In [ ]:
#all values containing a key word
keyW = 'riporto ciottoli'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

In [ ]:
#all values containing a key word
keyW = 'riporto ciottoloso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

riporto ciottoloso
riporto ciottoloso ghiaioso addensato
riporto ciottoloso ghiaioso con abbondante frazione limo sabbiosa con presenza di con frammenti di mattoni con tracce di calcestruzzo
riporto ciottoloso ghiaioso e laterizi
riporto ciottoloso sabbioso
riporto ciottoloso sabbioso poco addensato


In [ ]:
#all values containing a key word
keyW = 'riporto ghiaietto'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

In [ ]:
#all values containing a key word
keyW = 'riporto ghiaioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

riporto ghiaioso
riporto ghiaioso - limoso con subordinati frammenti lateritici
riporto ghiaioso ciottoloso
riporto ghiaioso ciottoloso con detrito
riporto ghiaioso ciottoloso con sabbia
riporto ghiaioso ciottoloso consabbia e limo
riporto ghiaioso ciottoloso, poco addensato; frammenti di laterizi, i clasti sono costituiti prevalentemente da calcescisti
riporto ghiaioso con calcestruzzo
riporto ghiaioso con presenza di laterizi
riporto ghiaioso debolmente argilloso poco consistente
riporto ghiaioso e breccia sabbiosa
riporto ghiaioso e ghiaioso sabbioso con ciottoli e con abbondante presenza di frammenti laterizi
riporto ghiaioso e sabbioso
riporto ghiaioso limoso
riporto ghiaioso limoso con rari frammenti lateritici
riporto ghiaioso medio fine in scarsa matrice sabbioso limosa
riporto ghiaioso sabbioso
riporto ghiaioso sabbioso a tratti limoso con asfalto
riporto ghiaioso sabbioso addensato
riporto ghiaioso sabbioso ciottoloso inglobante un blocco di micascisto fratturato e poco alter

In [ ]:
#all values containing a key word
keyW = 'riporto , ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',3)[0]+' '+key_list[position].split(' ',3)[1]+' '+key_list[position].split(' ',3)[2]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

riporto , ghiaia grossa con ciottoli


In [ ]:
#all values containing a key word
keyW = 'rilevato , ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',3)[0]+' '+key_list[position].split(' ',3)[1]+' '+key_list[position].split(' ',3)[2]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

rilevato , ghiaia eterometrica poligenica , ciottoli e sabbia media e grossa debolmente limosa con presenza di frammenti di laterizi
rilevato , ghiaia eterometrica poligenica , ciottoli e sabbia media e grossa limosa
rilevato , ghiaia eterometrica poligenica e sabbia media e grossa debolmente limosa con alcuni ciottoli


In [ ]:
#all values containing a key word
keyW = 'riporto e ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',3)[0]+' '+key_list[position].split(' ',3)[1]+' '+key_list[position].split(' ',3)[2]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

riporto e ghiaia


In [ ]:
#all values containing a key word
keyW = 'riporto di ghiaie'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',3)[0]+' '+key_list[position].split(' ',3)[1]+' '+key_list[position].split(' ',3)[2]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

riporto di ghiaie e ciottoli sabbiosi con frammenti di laterizi
riporto di ghiaie e laterizi eterometrici
riporto di ghiaie e sabbie
riporto di ghiaie e sabbie con ciottoli
riporto di ghiaie limoso sabbiose debolmente limose con rari ciottoli
riporto di ghiaie sabbioso limose


In [ ]:
#all values containing a key word
keyW = 'strati di ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',3)[0]+' '+key_list[position].split(' ',3)[1]+' '+key_list[position].split(' ',3)[2]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

strati di ghiaia e sabbia compatta con argilla


In [ ]:
#all values containing a key word
keyW = 'riporto di ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',3)[0]+' '+key_list[position].split(' ',3)[1]+' '+key_list[position].split(' ',3)[2]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

riporto di ghiaia
riporto di ghiaia , ciottoli e limo sabbioso
riporto di ghiaia , ciottoli e sabbia
riporto di ghiaia , ghiaietto , resti lateritici
riporto di ghiaia , limo, sabbia
riporto di ghiaia , sabbia e frammenti di laterizi
riporto di ghiaia argillosa con ghiaia
riporto di ghiaia argillosa con sabbia
riporto di ghiaia ciottoli e argilla
riporto di ghiaia con ciottoli e laterizi
riporto di ghiaia con ciottoli, debolmente sabbiosa
riporto di ghiaia con detriti metallici , plastica
riporto di ghiaia con limo e sabbia
riporto di ghiaia con livelli di conglomerato bituminoso e abbondanti resti di asfalto
riporto di ghiaia con sabbia
riporto di ghiaia con sabbia , limosa e alcuni ciottoli
riporto di ghiaia con sabbia debolmente limosa con rari ciottoli
riporto di ghiaia con sabbia e ciottoli mista a frammenti di laterizi
riporto di ghiaia con sabbia frammenti di mattoni e rari ciottoli
riporto di ghiaia con sabbia limosa , presenza di resti vegetali e frammenti di mattoni
riporto d

In [ ]:
#all values containing a key word
keyW = 'riporto in ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',3)[0]+' '+key_list[position].split(' ',3)[1]+' '+key_list[position].split(' ',3)[2]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

riporto in ghiaia e ciottoli


In [ ]:
#all values containing a key word
keyW = 'riporto di ghiaia,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',3)[0]+' '+key_list[position].split(' ',3)[1]+' '+key_list[position].split(' ',3)[2]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

riporto di ghiaia, eterometrica, qualche ciottolo e sabbia limosa, localmente limo sabbioso; presenza di frammenti lateritici
riporto di ghiaia, poligenica, ciottoli e sabbia debolmente limosa; presenza di frammenti lateritici
riporto di ghiaia, sabbia e frammenti lateritici


In [ ]:
#all values containing a key word
keyW = 'riporto con ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',3)[0]+' '+key_list[position].split(' ',3)[1]+' '+key_list[position].split(' ',3)[2]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

riporto con ghiaia , ciottoli e laterizi ; presenza di sabbia
riporto con ghiaia con sabbia
riporto con ghiaia e mattoni
riporto con ghiaia e sabbia
riporto con ghiaia e sabbia, da addensato a sciolto
riporto con ghiaia mista a ciottoli
riporto con ghiaia sabbiosa


In [ ]:
#all values containing a key word
keyW = 'riporto misto ghiaioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',3)[0]+' '+key_list[position].split(' ',3)[1]+' '+key_list[position].split(' ',3)[2]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

riporto misto ghiaioso sabbioso, con ciottoli, con livelli limoso argillosi consistenti
riporto misto ghiaioso sabbioso, con ciottoli, con livelli limoso argillosi consistenti, resti di laterizi
riporto misto ghiaioso sabbioso, con livello limoso argilloso, ciottoli e resti laterizi


In [ ]:
#all values containing a key word
keyW = 'riporto prevalentemente ghiaioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',3)[0]+' '+key_list[position].split(' ',3)[1]+' '+key_list[position].split(' ',3)[2]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

riporto prevalentemente ghiaioso
riporto prevalentemente ghiaioso ciottoloso
riporto prevalentemente ghiaioso ciottoloso con frammenti dilaterizi
riporto prevalentemente ghiaioso ciottoloso con sabbia limosa emattoni in frammenti
riporto prevalentemente ghiaioso ciottoloso con subordinata sabbia e sporadici mattoni
riporto prevalentemente ghiaioso sabbioso
riporto prevalentemente ghiaioso sabbioso con rari ciottoli con presenza di mattoni in frammenti


In [ ]:
#all values containing a key word
keyW = 'rilevato costituito da ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',4)[0]+' '+key_list[position].split(' ',4)[1]+' '+key_list[position].split(' ',4)[2]+' '+key_list[position].split(' ',4)[3]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

rilevato costituito da ghiaia sabbiosa con ciottoli sparsi
rilevato costituito da ghiaia sabbiosa con frammenti di asfalto


In [ ]:
#all values containing a key word
keyW = 'riporto misto a ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',4)[0]+' '+key_list[position].split(' ',4)[1]+' '+key_list[position].split(' ',4)[2]+' '+key_list[position].split(' ',4)[3]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

riporto misto a ghiaia


In [ ]:
#all values containing a key word
keyW = 'riporto composto da ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',4)[0]+' '+key_list[position].split(' ',4)[1]+' '+key_list[position].split(' ',4)[2]+' '+key_list[position].split(' ',4)[3]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

riporto composto da ghiaia debolmente sabbiosa con frammenti di mattoni
riporto composto da ghiaia e sabbia, molto addensato


In [ ]:
#all values containing a key word
keyW = 'riporto costituito da ghiaie'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',4)[0]+' '+key_list[position].split(' ',4)[1]+' '+key_list[position].split(' ',4)[2]+' '+key_list[position].split(' ',4)[3]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

riporto costituito da ghiaie con sabbia e materiale cementizio con frammenti di laterizi
riporto costituito da ghiaie e sabbie con ciottoli


In [ ]:
#all values containing a key word
keyW = 'riporto costituito da ghiaia,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',4)[0]+' '+key_list[position].split(' ',4)[1]+' '+key_list[position].split(' ',4)[2]+' '+key_list[position].split(' ',4)[3]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

riporto costituito da ghiaia, ciottoli e frammenti lateritici
riporto costituito da ghiaia, frammenti litici e sabbia fine limosa
riporto costituito da ghiaia, poligenica, e limo sabbioso
riporto costituito da ghiaia, presenza di porzioni sabbiose
riporto costituito da ghiaia, sabbia e frammenti lateritici
riporto costituito da ghiaia, sabbia, ciottoli e laterizi


In [ ]:
#all values containing a key word
keyW = 'riporto costituito da ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',4)[0]+' '+key_list[position].split(' ',4)[1]+' '+key_list[position].split(' ',4)[2]+' '+key_list[position].split(' ',4)[3]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

riporto costituito da ghiaia
riporto costituito da ghiaia , sabbia con ciottoli
riporto costituito da ghiaia , sabbia con ciottoli e livelli di limo sabbioso con frammenti di laterizi
riporto costituito da ghiaia , sabbia e laterizio
riporto costituito da ghiaia , sabbia e limo con resti vegetali
riporto costituito da ghiaia , sabbia e limo, presenza di frammenti lateritici
riporto costituito da ghiaia ciottolosa con frammenti laterizi
riporto costituito da ghiaia ciottolosa con sabbia e frammenti lateritici
riporto costituito da ghiaia ciottolosa con sabbia medio grossolana limosa
riporto costituito da ghiaia con abbondanti ciottoli e blocchi di rocce metamorfiche
riporto costituito da ghiaia con ciottoli e sabbia
riporto costituito da ghiaia con ciottoli in amtrice sabbiosa abbondante
riporto costituito da ghiaia con clasti
riporto costituito da ghiaia con sabbia
riporto costituito da ghiaia con sabbia , limo e frammenti di laterizi , eterometrica
riporto costituito da ghiaia con sab

In [ ]:
#all values containing a key word
keyW = 'riporto costituito da ciottoli'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',4)[0]+' '+key_list[position].split(' ',4)[1]+' '+key_list[position].split(' ',4)[2]+' '+key_list[position].split(' ',4)[3]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

riporto costituito da ciottoli
riporto costituito da ciottoli , ghiaia e mattoni
riporto costituito da ciottoli e ghiaia con sabbia
riporto costituito da ciottoli e ghiaia sabbiosa sciolta in matrice
riporto costituito da ciottoli e laterizi sciolti
riporto costituito da ciottoli e sabbia
riporto costituito da ciottoli in matrice siltosa debolmente sabbiosa, addensata


In [ ]:
#all values containing a key word
keyW = 'terreno di riporto ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',4)[0]+' '+key_list[position].split(' ',4)[1]+' '+key_list[position].split(' ',4)[2]+' '+key_list[position].split(' ',4)[3]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

terreno di riporto ghiaia
terreno di riporto ghiaia ciottoli
terreno di riporto ghiaia ciottolosa con limo sabbioso
terreno di riporto ghiaia ciottolosa con limo sabbioso e frammenti di laterizi
terreno di riporto ghiaia con limo debolmente sabbioso e ciottoli in subordine con frammenti di laterizi in matrice
terreno di riporto ghiaia con livelli limosi
terreno di riporto ghiaia con sabbia grossa debolmente ciottolosa
terreno di riporto ghiaia con sabbia grossolana
terreno di riporto ghiaia e ciottoli
terreno di riporto ghiaia e ciottoli in matrice terroso argillosa
terreno di riporto ghiaia e ciottoli misti a argilla
terreno di riporto ghiaia e sabbia
terreno di riporto ghiaia e sabbia con ciottoli eterometrici molto cementata
terreno di riporto ghiaia eterometrica poligenica alcuni ciottoli sabbia limosa con presenza di frammenti laterizi e di calcestruzzo e resti vegetali sparsi
terreno di riporto ghiaia eterometrica poligenica ciottoli e sabbia grossa e media limosa
terreno di ripo

In [ ]:
#all values containing a key word
keyW = 'terreno di riporto ghiaie'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',4)[0]+' '+key_list[position].split(' ',4)[1]+' '+key_list[position].split(' ',4)[2]+' '+key_list[position].split(' ',4)[3]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

terreno di riporto ghiaie ciottolose
terreno di riporto ghiaie con ciottoli poligenici
terreno di riporto ghiaie e ciottoli a tratti debolmente sabbiose debolmente limose
terreno di riporto ghiaie e sabbie con ciottoli
terreno di riporto ghiaie e sabbie con ciottoli e blocchi
terreno di riporto ghiaie eterometrica poligenica frammenti di laterizi e scorie con sabbia debolmente limosa


In [ ]:
#all values containing a key word
keyW = 'terreno di riporto ghiaioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',4)[0]+' '+key_list[position].split(' ',4)[1]+' '+key_list[position].split(' ',4)[2]+' '+key_list[position].split(' ',4)[3]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

terreno di riporto ghiaioso - limoso con subordinati frammenti lateritici
terreno di riporto ghiaioso ciottoloso
terreno di riporto ghiaioso ciottoloso con frammenti di laterizi, da poco addensato ad addensata
terreno di riporto ghiaioso ciottoloso in matrice siltosa
terreno di riporto ghiaioso ciottoloso in scarsa matrice limosa debolmente sabbiosa
terreno di riporto ghiaioso ciottoloso limoso sabbioso, con sporadici frammenti di laterizi, poco addensato
terreno di riporto ghiaioso ciottoloso sabbioso
terreno di riporto ghiaioso ciottoloso, sabbioso, poco addensato
terreno di riporto ghiaioso con frammenti di laterizi, poco addensato
terreno di riporto ghiaioso con laterizi
terreno di riporto ghiaioso grossolano
terreno di riporto ghiaioso limoso
terreno di riporto ghiaioso limoso con frammenti laterizi, pochi ciottoli, poco addensato
terreno di riporto ghiaioso limoso sabbioso addensato
terreno di riporto ghiaioso sabbiosa
terreno di riporto ghiaioso sabbioso
terreno di riporto ghiai

In [ ]:
#all values containing a key word
keyW = 'terreno di riporto ciottoli'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',4)[0]+' '+key_list[position].split(' ',4)[1]+' '+key_list[position].split(' ',4)[2]+' '+key_list[position].split(' ',4)[3]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

terreno di riporto ciottoli
terreno di riporto ciottoli e blocchi in scarsa matrice sabbiosa passante a clasti rocciosi e subordinati ciottoli in matrice da sabbioso limosa a limoso sabbiosa con rari frammenti di laterizi
terreno di riporto ciottoli e ghiaia con scarsa matrice sabbiosa
terreno di riporto ciottoli e ghiaia eterometrica poligenica con sabbia grossa e media limosa
terreno di riporto ciottoli e ghiaia sabbiosa poco addensata con frammenti di laterizi
terreno di riporto ciottoli e ghiaie grosse debolmente sabbioso limose
terreno di riporto ciottoli e trovanti con sabbia e frammenti di legno
terreno di riporto ciottoli eterometrici ghiaia e sabbia poco addensata mista a resti di laterizi
terreno di riporto ciottoli marnosi e sabbia medio fine


In [ ]:
#all values containing a key word
keyW = 'terreno di riporto ciottoloso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',4)[0]+' '+key_list[position].split(' ',4)[1]+' '+key_list[position].split(' ',4)[2]+' '+key_list[position].split(' ',4)[3]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

terreno di riporto ciottoloso
terreno di riporto ciottoloso ghiaioso sabbioso
terreno di riporto ciottoloso in matrice terroso sabbiosa
terreno di riporto ciottoloso in matrice terroso sabbioso
terreno di riporto ciottoloso in matrice terroso sabbioso con tracce di mattoni e scorie
terreno di riporto ciottoloso sabbioso, con frammenti laterizi, poco consistente


In [ ]:
#all values containing a key word
keyW = 'terreno con poca ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',4)[0]+' '+key_list[position].split(' ',4)[1]+' '+key_list[position].split(' ',4)[2]+' '+key_list[position].split(' ',4)[3]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

terreno con poca ghiaia


In [ ]:
#all values containing a key word
keyW = 'terreno di riporto di ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',5)[0]+' '+key_list[position].split(' ',5)[1]+' '+key_list[position].split(' ',5)[2]+' '+key_list[position].split(' ',5)[3]+' '+key_list[position].split(' ',5)[4]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

terreno di riporto di ghiaia e sabbia fine
terreno di riporto di ghiaia eterometrica poligenica alcuni ciottoli con sabbia fine limosa localmente con limo con presenza di frammenti di laterizi
terreno di riporto di ghiaia eterometrica poligenica alcuni ciottoli con sabbia limosa e/o limo sabbioso
terreno di riporto di ghiaia eterometrica poligenica con sabbia medio fine da limosa a con limo con presenza di frammenti laterizi
terreno di riporto di ghiaia eterometrica poligenica con sabbia medio fine da limosa a debolmente limosa
terreno di riporto di ghiaia eterometrica poligenica con sabbia medio fine limosa con presenza di mattoni
terreno di riporto di ghiaia eterometrica poligenica e sabbia fine limosa con presenza di frammenti laterizi
terreno di riporto di ghiaia eterometrica poligenica e sabbia limosa e/o limo sabbioso
terreno di riporto di ghiaia eterometrica poligenica frammenti di laterizi con limo sabbioso
terreno di riporto di ghiaia grossa trovanti con intercalazioni limose


In [ ]:
#all values containing a key word
keyW = 'terreno di riporto con ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',5)[0]+' '+key_list[position].split(' ',5)[1]+' '+key_list[position].split(' ',5)[2]+' '+key_list[position].split(' ',5)[3]+' '+key_list[position].split(' ',5)[4]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

terreno di riporto con ghiaia e sabbia mediamente addensata


In [ ]:
#all values containing a key word
keyW = 'terreno di riporto con ciottoli'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',5)[0]+' '+key_list[position].split(' ',5)[1]+' '+key_list[position].split(' ',5)[2]+' '+key_list[position].split(' ',5)[3]+' '+key_list[position].split(' ',5)[4]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

terreno di riporto con ciottoli
terreno di riporto con ciottoli e abbondante frazione fine
terreno di riporto con ciottoli limoso sabbioso


In [ ]:
#all values containing a key word
keyW = 'terreno di riporto , ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',5)[0]+' '+key_list[position].split(' ',5)[1]+' '+key_list[position].split(' ',5)[2]+' '+key_list[position].split(' ',5)[3]+' '+key_list[position].split(' ',5)[4]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

terreno di riporto , ghiaia e frammenti laterizi con sabbia fine limosa
terreno di riporto , ghiaia eterometrica poligenica , ciottoli e sabbia grossa e media limosa
terreno di riporto , ghiaia eterometrica poligenica , ciottoli e sabbia grossa e media molto addensata con presenza di frammenti di laterizi
terreno di riporto , ghiaia eterometrica poligenica , ciottoli e sabbia media e fine , da limosa a con limo , con presenza di frammenti di laterizi
terreno di riporto , ghiaia eterometrica poligenica e sabbia media e fine limosa
terreno di riporto , ghiaia eterometrica poligenica e sabbia medio fine limosa
terreno di riporto , ghiaia sabbiosa
terreno di riporto , ghiaia sabbiosa con ciottoli e frammenti di laterizi sparsi
terreno di riporto , ghiaia sabbiosa con clasti lapidei sparsi e subordinati frammenti di asfalto
terreno di riporto , ghiaia sabbiosa con frammenti di laterizi sparsi
terreno di riporto , ghiaia sabbiosa con subordinati frammenti di laterizi


In [ ]:
#all values containing a key word
keyW = 'terreno con presenza di ciottoli'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',5)[0]+' '+key_list[position].split(' ',5)[1]+' '+key_list[position].split(' ',5)[2]+' '+key_list[position].split(' ',5)[3]+' '+key_list[position].split(' ',5)[4]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

In [ ]:
#all values containing a key word
keyW = 'terreno di riporto costituito da ghiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',6)[0]+' '+key_list[position].split(' ',6)[1]+' '+key_list[position].split(' ',6)[2]+' '+key_list[position].split(' ',6)[3]+' '+key_list[position].split(' ',6)[4]+' '+key_list[position].split(' ',6)[5]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

terreno di riporto costituito da ghiaia
terreno di riporto costituito da ghiaia , poligenica e sabbia grossa e media limosa, con locali ciottoli
terreno di riporto costituito da ghiaia , poligenica, ciottoli e sabbia, limosa
terreno di riporto costituito da ghiaia ,poligenica, e sabbia medio grossa, limosa; presenza di frammewnti laterizi
terreno di riporto costituito da ghiaia ; ciottoli e sabbia medio grossa, limosa
terreno di riporto costituito da ghiaia ciottolosa con rari frammenti laterizi
terreno di riporto costituito da ghiaia ciottolosa con trovanti , blocchi e mattoni
terreno di riporto costituito da ghiaia con sabbia debolmente limosa e frammenti lateritici
terreno di riporto costituito da ghiaia con sabbia debolmente limosa e frammenti, addensata
terreno di riporto costituito da ghiaia con sabbia debolmente limosa, rari frammenti lateritici, moderatamente addensata
terreno di riporto costituito da ghiaia con sabbia e ciottoli; a tratti passaggi limosi argillosi
terreno di r

In [ ]:
#all values containing a key word
keyW = 'terreno di riporto costituito da ciottoli'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',6)[0]+' '+key_list[position].split(' ',6)[1]+' '+key_list[position].split(' ',6)[2]+' '+key_list[position].split(' ',6)[3]+' '+key_list[position].split(' ',6)[4]+' '+key_list[position].split(' ',6)[5]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

terreno di riporto costituito da ciottoli con subordinati clasti in matrice siltoso sabbiosa
terreno di riporto costituito da ciottoli e blocchi lapidei
terreno di riporto costituito da ciottoli e ghiaia eterometrica in matrice siltoso sabbiosa


In [ ]:
keyW = 'terra'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=602
    key_list[position]='Materiale organico'

terra
terra , limo , ciottoli
terra , limo e ciottoli
terra argillosa
terra argillosa compatta
terra argillosa con noduli di conglomerato
terra argilloso sabbiosa
terra ciottoli e sabbia
terra con ciottoli
terra con ghiaia
terra con ghiaietto
terra con limo e ciottoli
terra con pietre
terra con presenza di ciottoli
terra di riporto
terra e argilla
terra e ciottoli
terra e clasti
terra e ghiaia
terra e massi
terra e sabbia
terra e sabbia molto compatta con ciottoli
terra e trovanti
terra fangosa
terra gialla
terra limo sabbiose , sabbie argillose , limi
terra limo sabbioso , sabbie argillose , limi
terra mista a ghiaia
terra mista a ghiaia compatta
terra mista a ghiaia e sabbia compatta
terra mista a sabbia
terra mista a tufo
terra mista ghiaia
terra sabbia e massi
terra sabbiosa


In [ ]:
keyW = 'terre'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=602
    key_list[position]='Materiale organico'

terre argillose
terre argillose compatte miste a ghiaia
terre argillose compatte miste a tufo
terre argillose mista a ghiaia
terre argillose miste a ghiaia
terre sabbiose


In [ ]:
keyW = 'torba'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=602
    key_list[position]='Materiale organico'

torba
torba a livelli debolmente argillosa
torba argillosa
torba compatta
torba compressibile
torba compressibile con legno
torba con argilla
torba con intercalati livelli argilloso siltosi
torba con lignite
torba con limo
torba con piccoli livelli limosi
torba con presenza di frammentati di legno
torba con resti vegetali
torba con sabbia fine
torba e argilla torbosa sabbiosa fine
torba e argilla torboso sabbiosa
torba e ghiaietto
torba e lignite
torba e limi torbosi
torba e resti vegetali
torba e sabbia
torba e/o limo argilloso torboso a tratti sabbioso poco consistente plastico e compressibile
torba e/o limo sabbioso torboso a tratti argilloso poco consistente plastico e compressibile
torba limo sabbioso
torba mista a argilla
torba passante a limo argilloso


In [ ]:
keyW = 'tereno'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=602
    key_list[position]='Materiale organico'

tereno misto a ghiaia
tereno vegetale
tereno vegetale, sabbia limosa


In [ ]:
keyW = 'terreno vegetale,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=602
    key_list[position]='Materiale organico'

In [ ]:
#all values containing a key word
keyW = 'substrato roccioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

substrato roccioso
substrato roccioso , alternanza di livelli da consistente a livelli deboli conglomeratici , con ciottoli eterogeneo , ofiolitici
substrato roccioso , alternanze di micascisti e paragneiss
substrato roccioso , marne sabbiose da fratturate a frantumate , presenza di patine ossidazione
substrato roccioso alterato , ciottoli eterometrici a matrice ghiaioso sabbiosa , clasti ofiolitici con presenza di livelli alterati
substrato roccioso alterato a elementi argillificati localmente riempimento di argilla
substrato roccioso alterato di kinzigite cataclastico alterata con riempimento argilloso e scaglie di roccia alterata
substrato roccioso alterato di micascisto ossidato con presenza di riempimento argilloso
substrato roccioso alterato gneiss cloritico micaceo con presenza di fratture e livelli molto fratturati
substrato roccioso alterato gneiss micaceo e cloritico con fratture
substrato roccioso alterato molto fratturato con ossidazione
substrato roccioso alternanze di mar

In [ ]:
#all values containing a key word
keyW = 'substrato siltiti'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

substrato siltiti argillose
substrato siltiti argillose molto stratificate
substrato siltiti e livelli sabbiosi
substrato siltiti marnose stratificate
substrato siltiti molto stratificate
substrato siltiti sabbiose
substrato siltiti stratificate


In [ ]:
#all values containing a key word
keyW = 'substrato marnoso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

substrato marnoso
substrato marnoso alterato
substrato marnoso arenaceo e conglomeratico
substrato marnoso compatto
substrato marnoso con livelli siltosi
substrato marnoso consistente
substrato marnoso e arenaceo
substrato marnoso e arenaceo compatto
substrato marnoso siltoso compatto


In [ ]:
#all values containing a key word
keyW = 'substrato marne'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

substrato marne arenaceo sabbiose
substrato marne argillose
substrato marne argillose e arenaceo sabbiose
substrato marne argillose e arenarie
substrato marne argillose e arenarie sabbiose
substrato marne e arenarie con intercalazioni sabbiose
substrato marne e arenarie sabbiose
substrato marne e sabbie
substrato marne e sabbie arenacee
substrato marne e/o sabbie arenacee


In [ ]:
#all values containing a key word
keyW = 'substrato , marna'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',3)[0]+' '+key_list[position].split(' ',2)[1]+' '+key_list[position].split(' ',3)[2]==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

substrato , marna argillosa
substrato , marna argillosa , argilla marnosa
substrato , marna argillosa alterata
substrato , marna argillosa con livelli argillosi
substrato , marna calcarea
substrato , marna calcarea arenacea
substrato , marna siltosa stratificata con presenza di fratture


In [ ]:
#all values containing a key word
keyW = 'sabbia,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

sabbia, addensata
sabbia, addensata con assate debolmente cementate
sabbia, con ghiaia eterometrica, limosa, localmente con limo
sabbia, con ghiaia fine, poligenica, limosa
sabbia, con ghiaia grossa,poligenica, limosa
sabbia, con ghiaia medio fine, poligenica, debolmente limosa
sabbia, con ghiaia, limosa, localmente con limo
sabbia, con ghiaia, poligenica, limosa
sabbia, con ghiaia, poligenica, limosa, localmente con limo
sabbia, con ghiaia, poligenica, pochi ciottoli, limosa
sabbia, da limosa a con limo e ghiaia eterometrica, da alterata a intensamente alterata
sabbia, da sciolta a debolmente addensata, a tratti debolmente limosa
sabbia, debolmente limosa
sabbia, debolmente limosa con ghiaia , rari ciottoli, molto addensata con livelli cementati
sabbia, debolmente limosa e ghiaia, qualche ciottolo
sabbia, debolmente limosa, addensata
sabbia, debolmente limosa, con ghiaia eterometrica, poligenica, localmente con rara ghiaia
sabbia, debolmente limosa, micacea
sabbia, frammista a ghiaiet

In [ ]:
#all values containing a key word
keyW = 'sabbiamedio'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

sabbiamedio grossa, con ghiaia eterometrica, poligenica, localmente alterata


In [ ]:
#all values containing a key word
keyW = 'sabbie,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

sabbie, con frazione limosa
sabbie, debolmente limose
sabbie, ghiaietti e ghiaie argillose


In [ ]:
#all values containing a key word
keyW = 'sabbione'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

sabbione
sabbione , ghiaietto
sabbione , quarzo
sabbione , quarzo e pietre verdi
sabbione argillosa
sabbione argilloso
sabbione argilloso compatto
sabbione argilloso e ghiaia
sabbione cementato con strati di ghiaietto
sabbione compatto
sabbione compatto con argilla
sabbione compatto e argilla
sabbione con argilla
sabbione con ciottoli
sabbione con ciottoli sparsi
sabbione con ghiaia
sabbione con ghiaia e ghiaietto
sabbione con ghiaia e grossi ciottoli
sabbione con ghiaietto
sabbione con ghiaietto compatto
sabbione con ghiaietto con strati di argilla
sabbione con lenti di argilla
sabbione con poca argilla
sabbione con poco ghiaietto
sabbione con tracce di argilla
sabbione con tracce di ghiaietto
sabbione con tracce di lignite
sabbione con trovanti
sabbione costituito da quarzo e pietre verdi
sabbione da detrito
sabbione da detrito con limi
sabbione e argilla
sabbione e ciottoli
sabbione e ciottoli quarzosi
sabbione e ghiaia
sabbione e ghiaia con ciottoli
sabbione e ghiaia con strati di 

In [ ]:
#all values containing a key word
keyW = 'terreno vegetale'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=602
    key_list[position]='Materiale organico'

terreno vegetale
terreno vegetale , ghiaia e sabbia con ciottoli
terreno vegetale , limo
terreno vegetale , limo debolmente sabbioso
terreno vegetale , limo e sabbia
terreno vegetale , limo e sabbia fine con vegetali
terreno vegetale , limo sabbioso
terreno vegetale , limo sabbioso argilloso con presenza di resti vegetali
terreno vegetale , limo sabbioso con ghiaia e vegetali
terreno vegetale , limo sabbioso con ghiaia eterometrica e frammenti di laterizi
terreno vegetale , limo sabbioso con presenza di resti vegetali
terreno vegetale , limo sabbioso con radici
terreno vegetale , limo sabbioso con rari ciottoli e con presenza di radici
terreno vegetale , limo sabbioso con vegetali e ghiaia
terreno vegetale , limo sabbioso con vegetali e rara ghiaia
terreno vegetale , limo sabbioso passante a sabbia limosa
terreno vegetale , limo sabbioso plastico
terreno vegetale , limo sabbioso poco consistente
terreno vegetale , limoso argilloso
terreno vegetale , materiali di riporto e argilla
terre

In [ ]:
#all values containing a key word
keyW = 'trovante calcareo'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

trovante calcareo
trovante calcareo compatto
trovante calcareo fratturato
trovante calcareo intensamente fratturato
trovante calcareo localmente fratturato
trovante calcareo mediamente alterato e localmente fratturato


In [ ]:
#all values containing a key word
keyW = 'trovante di calcare'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',3)[0]+' '+key_list[position].split(' ',2)[1]+' '+key_list[position].split(' ',3)[2]==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

trovante di calcare
trovante di calcare compatto
trovante di calcare mediamente alterato e fratturato


In [ ]:
#all values containing a key word
keyW = 'trovanti di roccia calcarea'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',4)[0]+' '+key_list[position].split(' ',2)[1]+' '+key_list[position].split(' ',3)[2]+' '+key_list[position].split(' ',4)[3]==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

trovanti di roccia calcarea


In [ ]:
#all values containing a key word
keyW = 'trovante'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=306
    key_list[position]='Blocchi, trovanti, erratici'

trovante
trovante , gneiss occhiadino molto fratturato con riempimento di materiali fini e patine di alterazione
trovante calcescisto
trovante ciottoloso
trovante compatto
trovante con arenaria
trovante con arenaria e conchiglie
trovante con ciottoli
trovante con fratture
trovante con grossi ciottoli e ghiaia grossolana
trovante con molto quarzo
trovante cristallino
trovante di
trovante di anfibolite
trovante di arenaria alterata
trovante di calcescisti
trovante di calcescisti molto compatto
trovante di calcescisto
trovante di calcescisto compatto
trovante di calcescisto compatto, poco alterato e fratturato
trovante di calcescisto con molta calcite e noduli quarzosi
trovante di calcescisto con vena quarzitica
trovante di calcescisto filladico
trovante di calcescisto fratturato
trovante di calcescisto in limo sabbioso
trovante di calcescisto litoide alterato
trovante di calcescisto micaceo
trovante di calcescisto molto fratturato
trovante di calcescisto parzialmente alterato
trovante di

In [ ]:
#all values containing a key word
keyW = 'trovanti'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=306
    key_list[position]='Blocchi, trovanti, erratici'

trovanti
trovanti , blocchi e ciottoli alterati in matrice scarsa sabbiosa con limo parzialmente argillificato
trovanti , blocchi e ciottoli alterati in scarsa matrice sabbiosa debolmente limosa argillosa
trovanti , blocchi e ciottoloni compatti
trovanti , ciottoli , blocchi con patine di alterazione in matrice scarsa sabbiosa limosa argillificata alterata
trovanti , ciottoli , blocchi con patine di alterazione in matrice scarsa sabbioso limosa alterata
trovanti , ciottoli , blocchi con patine di alterazione in matrice scarsa sabbioso limosa argillificata alterata
trovanti , ciottoli con sabbia
trovanti , ciottoli e blocchi con patine di alterazione in matrice scarsa sabbiosa limosa alterata
trovanti , ciottoli e ghiaia
trovanti , ciottoli e ghiaia con poca sabbia
trovanti , ciottoli e ghiaia eterometrica
trovanti , ghiaia e sabbia
trovanti , litoide
trovanti blocchi e ciottoli in sabbia
trovanti ciottoli e ghiaia
trovanti ciottoli e ghiaia con sabbia
trovanti ciottoli e ghiaia con sab

In [ ]:
#all values containing a key word
keyW = 'siltite'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

siltite
siltite a debole cementazione
siltite a debole cementazione con interlivelli dimarna debolmente argillosa
siltite a volte alterata e disgregata con intercalazioni di argillite
siltite a volte arenacea ben cementata
siltite a volte debolmente argillosa
siltite alterata
siltite alterata , da debolmente marnosa a marnosa con subordinate laminazioni sabbioso fini
siltite arenacea ben cementata con interstrati argillite scagliosa rari
siltite arenacea compatta
siltite arenacea in livelli con intercalazioni di limi argillosi
siltite arenaceo marnosa
siltite argillosa , intenso grado di alterazione e ossidazione con subordinati livelli sabbioso fini intensamente ossidati, moderatmante consistente
siltite argillosa a volte marnosa consistente con rari livelli sabbiosi e/o sabbioso arenacei
siltite argillosa a volte sabbiosa fine ossidata moderatamente consistente debolmente plastica con rare concrezioni calcaree e intercalati livelli di argilla plastica con abbondante ossidazione
silti

In [ ]:
#all values containing a key word
keyW = 'tufo'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

tufo
tufo argilloso
tufo argilloso con ghiaia
tufo compatto
tufo con ghiaia
tufo con lignite
tufo con trovanti calcarei
tufo con trovanti in lignite
tufo con venature sabbiose
tufo duro
tufo e arenarie compatte
tufo e calcare
tufo e ghiaia
tufo e ghiaietto
tufo limoso
tufo limoso coesivo con sabbia
tufo limoso sabbioso debolmente argilloso
tufo misto a ghiaia
tufo sabbioso
tufo sabbioso limoso con ghiaietto


In [ ]:
#all values containing a key word
keyW = 'terreno sabbioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

terreno sabbioso
terreno sabbioso argilloso
terreno sabbioso argilloso con frammenti serpentinitici
terreno sabbioso argilloso con laterizi
terreno sabbioso argilloso con resti vegetali elaterizi
terreno sabbioso argilloso e ciottoli marnosi compatti
terreno sabbioso argilloso rari clasti
terreno sabbioso ciottoloso
terreno sabbioso con ciottoli
terreno sabbioso con ciottoli e blocchi alterati
terreno sabbioso con elementi ghiaiosi
terreno sabbioso con frazione fine limosa argillosa con ciottoli
terreno sabbioso con frazione fine limosa argillosa con rari ciottoli alterati
terreno sabbioso con frazione fine limoso argillosa con rari ciottoli alterati
terreno sabbioso con ghiaie , con subordinati resti di laterizi
terreno sabbioso con scarsa ghiaia
terreno sabbioso debolmente limoso con breccia scistosa e resti vegetali
terreno sabbioso debolmente limoso con rari clasti di calcescisti
terreno sabbioso debolmente limoso con resti vegetali e pietre
terreno sabbioso debolmente limoso con r

In [ ]:
#all values containing a key word
keyW = 'terreno limoso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

terreno limoso
terreno limoso alterato con resti vegetali
terreno limoso argillosa
terreno limoso argilloso
terreno limoso argilloso con clasti ; coltre eluvio-colluviale
terreno limoso argilloso con frazione vegetale
terreno limoso argilloso con ghiaietto e ciottoli
terreno limoso argilloso con rari clasti
terreno limoso argilloso di alterazione con frammenti di marna
terreno limoso con ciottoli
terreno limoso con ciottoli micacei e resti vegetali
terreno limoso con ghiaia
terreno limoso con ghiaia e ciottoli
terreno limoso debolmente sabbioso
terreno limoso inglobante piccoli ciottoli di gneiss
terreno limoso sabbiosa
terreno limoso sabbiosa debolmente argillosa
terreno limoso sabbiosa debolmente argillosa poco rimaneggiato
terreno limoso sabbiosa debolmente argillosa rimaneggiato
terreno limoso sabbioso
terreno limoso sabbioso argilloso
terreno limoso sabbioso con ciottoli
terreno limoso sabbioso con presenza di blocchi
terreno limoso sabbioso con presenza di materiale argilloso con

In [ ]:
#all values containing a key word
keyW = 'ghaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

ghaia frammista a sabbia


In [ ]:
#all values containing a key word
keyW = 'ghiai'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

ghiai con sabbia
ghiai con sabbia limosa, tracce di argilla e locali ciottoli
ghiai con sabbia; preenza di blocchi
ghiai eterometrica e ciottoli molto densa
ghiai media e grossolana sabbiosa


In [ ]:
#all values containing a key word
keyW = 'ghiaia,poligenica,rari'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

ghiaia,poligenica,rari ciottoli e sabbia, limosa


In [ ]:
#all values containing a key word
keyW = 'ghiaiacon'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

ghiaiacon intercalazioni limose


In [ ]:
#all values containing a key word
keyW = 'ghiaiaeterometrica,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

ghiaiaeterometrica, alterata, ciottoli e sabbia media e fine da limosa a con limo


In [ ]:
#all values containing a key word
keyW = 'ghiaietoo'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

In [ ]:
#all values containing a key word
keyW = 'ghiaietta'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

ghiaietta
ghiaietta fine


In [ ]:
#all values containing a key word
keyW = 'ghiaietto,'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

ghiaietto, ghiaia e ciottoli in amtrice sabbiosa debolmente limosa
ghiaietto, sabbia fine con ciottoli


In [ ]:
#all values containing a key word
keyW = 'ghiaiettto'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

ghiaiettto con argilla calcarea


In [ ]:
#all values containing a key word
keyW = 'ghiia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

ghiia con sabbia
ghiia e sabbia debolmente limosa con clasti
ghiia e sabbia fine addensata
ghiia frammista a sabbia limosa mediamente compatta, con intercalazioni di limi sabbiosi


In [ ]:
#all values containing a key word
keyW = 'ghiiaia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

ghiiaia e sabbia debolmente limoso ciottolosa


In [ ]:
#all values containing a key word
keyW = 'grossi ciottoli'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

grossi ciottoli
grossi ciottoli con ghiaia
grossi ciottoli con ghiaia , molta sabbia e lenti di argilla
grossi ciottoli con ghiaia e sabbia
grossi ciottoli con ghiaia e sabbia limosa
grossi ciottoli con ghiaia e trovanti
grossi ciottoli con poca sabbia
grossi ciottoli con sabbia
grossi ciottoli con sabbia terrosa
grossi ciottoli con trovanti
grossi ciottoli e blocchi debolmente ghiaioso sabbiosi
grossi ciottoli e ghiaia grossa con poca sabbia limosa
grossi ciottoli e ghiaia grossa con sabbia
grossi ciottoli e ghiaia grossa con sabbia fine
grossi ciottoli e ghiaia in sabbia a tratti limosa
grossi ciottoli e ghiaia in sabbia sciolta
grossi ciottoli e ghiaia parzialmente cementata con sabbia fine
grossi ciottoli e ghiaietto con sabbia limosa
grossi ciottoli e sabbia mista a sabbione
grossi ciottoli e sabbia terrosa
grossi ciottoli etrovanti con ghiaia grossolana
grossi ciottoli etrovanti con scarsa frazione sabbiosa
grossi ciottoli etrovanti diroccia cristallina
grossi ciottoli trovanti c

In [ ]:
#all values containing a key word
keyW = 'grosso ciottolo'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

grosso ciottolo
grosso ciottolo di metabasite


In [ ]:
#all values containing a key word
keyW = 'piccoli blocchi'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

piccoli blocchi ciottoli e ghiaia da media a grossa
piccoli blocchi detritici e frammenti rocciosi con abbondante matrice fine
piccoli blocchi di gneiss fratturati e alterati con riempimento di piccole scaglie e sabbia limosa
piccoli blocchi molto alterati e fratturati con abbondante materiale sciolto


In [ ]:
#all values containing a key word
keyW = 'piccoli ciottoli'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

piccoli ciottoli di ghiaietto medio fine in matrice di terreno sciolto debolmente limoso
piccoli ciottoli e clasti in matrice di sabbia ghiaiosa molto micacea
piccoli ciottoli e clasti in sabbia ghiaiosa con presenza di limo
piccoli ciottoli e micascisti in frammenti alterati
piccoli ciottoli fratturati con scaglie frammenti e limo sabbioso
piccoli ciottoli ghiaia e calcestruzzo
piccoli ciottoli ghiaie medie e fini in matrice siltoso sabbiosa


In [ ]:
#all values containing a key word
keyW = 'piccoli clasti'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=300
    key_list[position]='Ghiaia'

piccoli clasti in matrice di limo sabbioso debolmente ghiaioso


In [ ]:
#all values containing a key word
keyW = 'grossi trovanti'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=306
    key_list[position]='Blocchi, trovanti, erratici'

grossi trovanti
grossi trovanti , ciottoli e ghiaia con sabbia
grossi trovanti con ghiaia
grossi trovanti con ghiaia e sabbia
grossi trovanti con poca sabbia
grossi trovanti e ghiaia


In [ ]:
#all values containing a key word
keyW = 'grosso trovante'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=306
    key_list[position]='Blocchi, trovanti, erratici'

grosso trovante
grosso trovante di calcare compatto
grosso trovante di calcare marnoso
grosso trovante gneissico


In [ ]:
#all values containing a key word
keyW = 'grosso blocco'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=306
    key_list[position]='Blocchi, trovanti, erratici'

grosso blocco


In [ ]:
#all values containing a key word
keyW = 'piccoli trovanti'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=306
    key_list[position]='Blocchi, trovanti, erratici'

piccoli trovanti
piccoli trovanti , ciottoli e ghiaia
piccoli trovanti alterati
piccoli trovanti ciottoli ghiaia con sabbia
piccoli trovanti con elementi lapidei esabbia
piccoli trovanti con poca sabbia
piccoli trovanti di calcescisto con tracce di sabbia limosa
piccoli trovanti di calcescisto inglobanti in limo sabbioso
piccoli trovanti digneiss a tratti fratturati con fratture ossidate
piccoli trovanti digneiss con detriti
piccoli trovanti digneiss con sabbia argillosa
piccoli trovanti e ciottoli
piccoli trovanti e ciottoli gneissici
piccoli trovanti e ciottoli poligenici
piccoli trovanti eciottoli
piccoli trovanti fratturati e ossidati
piccoli trovanti intercalati con ghiaia grossolana e ciottoli


In [ ]:
#all values containing a key word
keyW = 'piccolo trovante'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',2)[0]+' '+key_list[position].split(' ',2)[1]==keyW:
    print(key_list[position])
    val_list[position]=306
    key_list[position]='Blocchi, trovanti, erratici'

piccolo trovante
piccolo trovante di roccia
piccolo trovante di roccia metamorfica fratturata


In [ ]:
#all values containing a key word
keyW = 'massi'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=306
    key_list[position]='Blocchi, trovanti, erratici'

massi
massi con argilla
massi con torba
massi dicalcescisto quarzoso con sabbia limosa
massi didiorite eeclogite
massi digneiss eanfibolite
massi e ciottoli
massi e frammenti con limo esabbia
massi eciottoli
massi frammenti eciottoli con limo sabbioso
massi gneissici con breccia medio grossolana e ciottoli
massi grossi
massi grossi con ghiaia


In [ ]:
#all values containing a key word
keyW = 'massi'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=306
    key_list[position]='Blocchi, trovanti, erratici'

In [ ]:
#all values containing a key word
keyW = 'laterizi'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=1001
    key_list[position]='Mattoni, laterizi'

laterizi
laterizi e ghiaia
laterizi e mattoni
laterizi frantumati rari
laterizi in frammenti


In [ ]:
#all values containing a key word
keyW = 'macerie'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=1001
    key_list[position]='Mattoni, laterizi'

macerie e laterizi


In [ ]:
#all values containing a key word
keyW = 'mattoni'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=1001
    key_list[position]='Mattoni, laterizi'

mattoni
mattoni e arenaria
mattoni e calcestruzzo
mattoni e sabbia fine
mattoni eterreno diriporto
mattoni riporto di clasti rocciosi consabbia


In [ ]:
#all values containing a key word
keyW = 'legno'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=602
    key_list[position]='Materiale organico'

legno
legno argilloso con frazione sabbiosa rari ciottoli alterati
legno e sabbia fine limosa
legno e sabbia medio fine


In [ ]:
#all values containing a key word
keyW = 'lim'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

lim o a r g i l l o s o p a s s a n t e a l i m o s a b b i o s o , c o n s u b o r d i n a t i l i v e l l i s a b b i o s o f i n i , m o l t o c o n s i s t e n t e / T D > / T R > T R a l i g n = ' c e n t e r ' b g c o l o r = ' # E 6 E E F F ' > T D > 9 9 9 5 1 4 6 / T D > T D > 1 3 9 . 1 0 / T D > T D a l i g n = ' l e f t ' > s a b b i a m e d i o f i n e d e b o l m e n t e l i m o s a / T D > / T R > T R a l i g n = ' c e n t e r ' b g c o l o r = ' # C D D E F F ' > T D > 9 9 9 5 1 4 6 / T D > T D > 1 4 0 . 7 0 / T D > T D a l i g n = ' l e f t ' > l i m o a r g i l l o s o , m o l t o c o n s i s t e n t e , c o m p a t t o ; l i v e l l o c a r b o n i o s o / T D > / T R > T R a l i g n = ' c e n t e r ' b g c o l o r = ' # E 6 E E F F ' > T D > 9 9 9 5 1 4 6 / T D > T D > 1 4 3 . 9 0 / T D > T D a l i g n = ' l e f t ' > s a b b i a m o l t o f i n e l i m o s a , c o n s p o r a d i c i l i v e l l i l i m o s o a r g i l l o s i / T D > / T R > T R a l i g n = ' c e n 

In [ ]:
#all values containing a key word
keyW = 'limo,ciottoli'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

limo,ciottoli e trovanti cementati


In [ ]:
#all values containing a key word
keyW = 'limo-sabbiosa'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

limo-sabbiosa


In [ ]:
#all values containing a key word
keyW = 'limos'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

limos abbioso ciottoloso con frazione argilloso limosa
limos abbioso debolmente plastico
limos abbioso inglobante ghiaia


In [ ]:
#all values containing a key word
keyW = 'marmi'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

marmi dolomitici , roccia da compatta a orientata , fratture ossidate con riempimento , con presenza di alcuni livelli cataclasati e livelli molto fratturati
marmi listati a flogopite
marmi listati a flogopite intensamente fratturati lungo piani sub-verticali , presenza di abbondante calcite e livelli di piccoli solfuri
marmi micacei molto fratturati con livelli di calcescisti molto alterati


In [ ]:
#all values containing a key word
keyW = 'marmo'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

marmo
marmo a flogopite intensamente fratturato lungo piani sub-verticali
marmo a tratti cariato
marmo calcareo con fratture
marmo cloritico muscovitico molto alterato e fratturato a volte argillificato
marmo dolomitico , fratture moderatamente ossidate con riempimento , con presenza di alcuni livelli di roccia da molto fogliettata a cataclasata
marmo dolomitico , fratture ossidate con riempimento , con livelli di roccia da molto fratturata a cataclasata
marmo dolomitico con livelli da molto fratturati a cataclasati
marmo dolomitico intensamente fratturato
marmo e dolomia mediamente fratturata con sottili patine di aragonite
marmo fratturato localmente intensamente alterato


In [ ]:
#all values containing a key word
keyW = 'micascisti'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

micascisti
micascisti a quarzo e mica molto fogliettati e fratturati
micascisti a tratti compatti a tratti fratturati
micascisti a tratti grafitici e/o granatiferi
micascisti alterati con livelli limoso argillosi compatti
micascisti alterati e argillificati
micascisti alterati e argillificati , presenza di porzioni litoidi e inalterate
micascisti alterati e degradati limo di degradazione micacea
micascisti alterati e degradati localmente con presenza di vene di quarzo
micascisti alterati e sfatti
micascisti alterati e sfatti con tracce di ossidazione
micascisti alterati e sfatti parzialmente argillificati con livelli ossidati
micascisti alterati fratturati
micascisti alterati fratturati e localmente degradati in limo sabbioso
micascisti alterati localmente degradati
micascisti alterati molto fratturati
micascisti alterati ossidati e argillificati
micascisti biolitici con lenti e noduli di quarzo , roccia compatta con locali livelli fratturati
micascisti biolitici con locali lenti di qu

In [ ]:
#all values containing a key word
keyW = 'micascisto'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

micascisto
micascisto , intensamente fratturato in clasti con patine di ossidazione
micascisto , molto fratturato , ossidato e alterato da consistente a molto consistente con passate litoidi
micascisto , prevalentemente integro fortemente scistoso; presenza di fratture con patine di ossidazione
micascisto a granati localmente debolmente fratturato lungo piani di scistosità suborizzontale
micascisto a tratti gneissico con vene di quarzo molto alterato
micascisto a volte gneissico , intensa alterazione e ossidazione , a livelli argillificato , a volte arcosico , da sublitoide a litoide
micascisto a volte gneissico , intensa ossidazione e alterazione , argillificato e sfatto , arcosico , da sublitoide a litoide
micascisto alterato
micascisto alterato e argillificato da poco consistente a consistente con livelli debolmente litoidi intercalati
micascisto alterato e degradato in limo con presenza di lenti di quarzo
micascisto alterato e fratturato
micascisto alterato e sfatto argillificato c

In [ ]:
#all values containing a key word
keyW = 'milonite'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

milonite
milonite , cappellaccio di alterazione , sabbia limosa con elementi lapidei di micascisto
milonite , sabbia debolmente limosa con elementi lapidei dolomitici
milonite con fratture
milonite con lenti di quarzo , compatto
milonite di serpentinoscisti e talcoscisti da intensamente laminati a sfatti
milonite fogliettata
milonite fogliettata con patine lucide, ossidate
milonite fogliettata e sfatta a volte argillificata con rari livelli litoidi
milonite micascistosa con intercalazioni di breccia calcarea molto cariata e ossidata


In [ ]:
#all values containing a key word
keyW = 'miloniti'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

miloniti


In [ ]:
#all values containing a key word
keyW = 'oficalce'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

oficalce
oficalce alterata argillificata
oficalce alterata argillificata con tracce di calcite
oficalce alterata e argillificata
oficalce alterata e argillificata con rari livelli sublitoidi
oficalce cataclastica
oficalce cataclastica con intercalati livelli di serpentinite
oficalce cataclastica con intercalati livelli di serpentinoscisto in subordine
oficalce cataclastica con livelli mineralizzati a talco e asbesto
oficalce con argillificazione
oficalce con calcite e intercalati livelli di talco
oficalce con calcite e livelli di serpentinoscisto in subordine ossidazione alterazione
oficalce con calcite e livelli di talco in subordine con passata di serpentinite
oficalce con calcite e livelli intercalati di talco con presenza di fratture
oficalce con calcite e rari livelli intercalati di talco
oficalce con calcite rari livelli di talco con presenza di fratture
oficalce con frattura
oficalce con livelli talcosi
oficalce con passate di livelli cataclastcici
oficalce con subordinati livel

In [ ]:
#all values containing a key word
keyW = 'ortogneiss'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

ortogneiss compatto , duro
ortogneiss duro e compatto
ortogneiss poco fratturato


In [ ]:
#all values containing a key word
keyW = 'pietra'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

pietra


In [ ]:
#all values containing a key word
keyW = 'pietre'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

pietre
pietre , mattoni in scarsa matrice sabbiosa localmente cementata
pietre compatte
pietre con ghiaia grossolana
pietre e ghiaione
pietre friabile
pietre friabili
pietre grosse e terra
pietre sciolte


In [ ]:
#all values containing a key word
keyW = 'porfidi'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

porfidi
porfidi da molto fratturato a fratturato con riempimento limoso


In [ ]:
#all values containing a key word
keyW = 'prasinite'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

prasinite
prasinite brecciata
prasinite brecciata fratturata e frantumata
prasinite compatta
prasinite compatta o poco fratturata con alcune venature quarzitiche fratture con tracce di ossidazione
prasinite compatta passante a serpentino con interlivello di serpentinite fratturata
prasinite completamente frantumata e ossidata
prasinite con fratture con livelli quarzosi
prasinite con fratture in lenti quarzose
prasinite con fratture in lenti quarzose e di calcite
prasinite con fratture ossidate in lenti quarzose
prasinite con inclusi a venature di quarzo
prasinite con lenti feldspatiche , fratture ossidate con riempimento , con livelli di roccia molto fratturati
prasinite con livelli di calcare e fratture
prasinite con livelli di calcare fratturata
prasinite con piccole lenti feldspatico calcitici , fratture ossidate con riempimento , con presenza di livelli di roccia molto fratturata e rari livelli di cataclasiti
prasinite da compatta a fratturata fratture con presenza di materiale fin

In [ ]:
#all values containing a key word
keyW = 'prasiniti'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

prasiniti
prasiniti fratturate
prasiniti grafitiche
prasiniti in frammenti
prasiniti in livelli a volte compatti a volte fessurati ossidati con presenza di cloritoscisti


In [ ]:
#all values containing a key word
keyW = 'quarzite'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

quarzite
quarzite , rare fratture debolmente ossidate con riempimento
quarzite alterata da consistente a sublitoide
quarzite alterata ossidata con fratture
quarzite compatta passante a paragneiss
quarzite completamente alterata e sfatta a tratti argillificata con ossidazione
quarzite con fratture
quarzite con fratture ossidate
quarzite con livelli quarzose e fratture
quarzite con mica
quarzite con presenza di discontinuita'
quarzite con rare fratture debolmente ossidate con riempimento
quarzite conglomeratica
quarzite da compatta a debolmente orientata
quarzite da compatta a debolmente orientata con livelli di roccia molto fratturati
quarzite da intensamente alterata e ossidata a litoide
quarzite e cristalli di mica
quarzite fratturazione con riempimento argilloso ossidato
quarzite in roccia calcescistosa
quarzite intensamente alterata ossidata
quarzite intensamente alterata ossidata da molto consistente a sub-litoide
quarzite intensamente alterata sub-litoide
quarzite litoide
quarzite

In [ ]:
#all values containing a key word
keyW = 'puddinga'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

puddinga
puddinga fratturata
puddinga molto fine
puddinga stratificata


In [ ]:
#all values containing a key word
keyW = 'quarzo'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

quarzo micascisto fratturato
quarzo micascisto fratturato con ossidazione
quarzo micascisto fratturato ossidato


In [ ]:
#all values containing a key word
keyW = 'quarzomicascisti'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

quarzomicascisti compatti
quarzomicascisti e calcemicascisti compatti a intensa stratificazione e foliazione con rare discontinuita' , presenza di livello argillificato
quarzomicascisti e calcemicascisti compatti con intensa stratificazione e foliazione , presenza di rare discontinuita'
quarzomicascisti intensamente alterati e argillificati


In [ ]:
#all values containing a key word
keyW = 'gabbro'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  if key_list[position].split(' ',1)[0]==keyW:
    print(key_list[position])
    val_list[position]=200
    key_list[position]='Materiale roccioso del substrato (escluso rocce clastiche)'

gabbro compatto con poche fratture
gabbro fratturato


In [ ]:
#all values containing a key word
keyW = 'matrice sabbioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

In [ ]:
#all values containing a key word
keyW = 'materiale sabbioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

materiale sabbioso
materiale sabbioso argilloso
materiale sabbioso con ciottoli
materiale sabbioso con grossi ciottoli
materiale sabbioso con presenza di limo argilla frammenti micacei e quarzosi
materiale sabbioso grossolano con rari clasti
materiale sabbioso limoso
materiale sabbioso limoso con clasti
materiale sabbioso limoso con passate di abbondante argilla
materiale sabbioso limoso dicalcescisto equarzo in frammenti
materiale sabbioso sciolto con grossi ciottoli


In [ ]:
#all values containing a key word
keyW = 'loam sabbioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

loam sabbioso
loam sabbioso con livello di loam limoso
loam sabbioso passante a sabbia fine limosa


In [ ]:
#all values containing a key word
keyW = 'livello sabbioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

livello sabbioso
livello sabbioso argilloso compatto alterazione di elementi grossolani
livello sabbioso con ciottoli
livello sabbioso con ciottoli in subordine
livello sabbioso con ghiaietto
livello sabbioso con subordinati clasti
livello sabbioso debolmente limoso
livello sabbioso e limoso
livello sabbioso fine
livello sabbioso fine limoso
livello sabbioso fine sciolto
livello sabbioso ghiaioso
livello sabbioso limoso
livello sabbioso limoso argilloso
livello sabbioso limoso compatto
livello sabbioso limoso con ghiaia
livello sabbioso limoso con ghiaietto
livello sabbioso limoso con scarso ghiaietto
livello sabbioso limoso molto plastico
livello sabbioso limoso plastico
livello sabbioso limoso plastico poco consistente
livello sabbioso limoso poco addensato
livello sabbioso molto plastico
livello sabbioso sabbioso limoso compatto
livello sabbioso sciolto
livello sabbioso torboso
livello sabbioso torboso alterato


In [ ]:
#all values containing a key word
keyW = 'livelli sabbiosi'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

livelli sabbiosi compatti in marna compatta
livelli sabbiosi e sabbioso limosi
livelli sabbiosi e sabbioso limosi , localmente blocchi


In [ ]:
#all values containing a key word
keyW = 'livelli disabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

livelli disabbia fine debolmente limosa e livelli argilloso marnosi
livelli disabbia fine debolmente limosa in alternanza con livelli argilloso marnosi debolmente plastici
livelli disabbia fine debolmente limosa in alternanza con livelli dimarna siltosa da poco a moderatamente consistente
livelli disabbia fine in alternanza con livelli siltoso marnosi
livelli disabbia fine limosa discreta alterazione in alternanza con livelli dimarna siltosa fratturata consistente
livelli disabbia fine ossidata in alternanza con livelli diargilla marnosa moderatamente consistente


In [ ]:
#all values containing a key word
keyW = 'litotipi sabbiosi'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

litotipi sabbiosi
litotipi sabbiosi a tratti cementati
litotipi sabbiosi con ghiaia
litotipi sabbiosi con ghiaietto
litotipi sabbiosi con raro ghiaietto


In [ ]:
#all values containing a key word
keyW = 'distruzione sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

distruzione sabbia con ghiaia e ciottoli
distruzione sabbia e ghiaia


In [ ]:
#all values containing a key word
keyW = 'detrito sabbioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

detrito sabbioso
detrito sabbioso con clasti brecciosi
detrito sabbioso con clasti con presenza di blocco di quarzite
detrito sabbioso con livelli ghiaiosi e rari ciottoli
detrito sabbioso ghiaioso con ciottoli e resti vegetali
detrito sabbioso ghiaioso con livelli di limo argillosi
detrito sabbioso limoso con breccia da fine a grossolana
detrito sabbioso limoso con ghiaia e breccia fine media inglobante abbondanti ciottoli e ghiaia grossolana


In [ ]:
#all values containing a key word
keyW = 'deposito sabbioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

deposito sabbioso con ciottoli e locali resti vegetali
deposito sabbioso e limoso con strati di argilla e marne
deposito sabbioso ghiaioso ciottoloso addensato
deposito sabbioso ghiaioso ciottoloso addensato con intercalazioni locali sabbiose limose
deposito sabbioso ghiaioso con intercalazioni ciottolose
deposito sabbioso ghiaioso con intercalazioni sabbioso limose
deposito sabbioso limosa
deposito sabbioso limosi
deposito sabbioso limoso
deposito sabbioso limoso con intercalazioni ghiaiose sabbiose ciottolose
deposito sabbioso limoso con passate ghiaioso sabbiose


In [ ]:
#all values containing a key word
keyW = 'da sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

da sabbia a marna sabbiosa
da sabbia argillosa medio fine a argilla sabbiosa
da sabbia e clasti a clasti e sabbia
da sabbia fine limosa a limo sabbioso
da sabbia limosa a limo sabbioso
da sabbia limosa con ghiaietto a limo argilloso


In [ ]:
#all values containing a key word
keyW = 'livello di sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

livello di sabbia
livello di sabbia debolmente ghiaiosa
livello di sabbia debolmente limosa
livello di sabbia fine
livello di sabbia fine limosa debolmente addensata
livello di sabbia limosa


In [ ]:
#all values containing a key word
keyW = 'materiale prevalentemente sabbioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

materiale prevalentemente sabbioso ciottoloso
materiale prevalentemente sabbioso con ciottoli e trovanti
materiale prevalentemente sabbioso con clasti e passate debolmente limose
materiale prevalentemente sabbioso con clasti e passate limose
materiale prevalentemente sabbioso con rari clasti e ciottoli di calcescisti alterati
materiale prevalentemente sabbioso grossolano con ciottoli e trovanti
materiale prevalentemente sabbioso limoso con ciottoli di calcescisti alterati
materiale prevalentemente sabbioso limoso con clasti
materiale prevalentemente sabbioso limoso con rari ciottoli e clasti
materiale prevalentemente sabbioso limoso con rari clasti
materiale prevalentemente sabbioso limoso con rari clasti con presenza di trovante di calcescisti
materiale prevalentemente sabbioso limoso con rari clasti di calcescisti
materiale prevalentemente sabbioso limoso con rari clasti e trovanti e passate di limo sabbioso
materiale prevalentemente sabbioso limoso con rari clasti eterometrici


In [ ]:
#all values containing a key word
keyW = 'strati di sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

strati di sabbia compatta con conchiglie e strati di pietra
strati di sabbia grossa


In [ ]:
#all values containing a key word
keyW = 'strati di sabbie'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

strati di sabbie eterometriche , addensate e localmente debolmente cementate , alternate a livelli di arenarie
strati di sabbie finissime alternati a strati di limo con lenti di argille e conglomerati di calce


In [ ]:
#all values containing a key word
keyW = 'suolo con sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

suolo con sabbia


In [ ]:
#all values containing a key word
keyW = 'substrato alterato sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

substrato alterato sabbia arenacee


In [ ]:
#all values containing a key word
keyW = 'terreno alterato sabbioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

terreno alterato sabbioso ghiaioso


In [ ]:
#all values containing a key word
keyW = 'terreno agrario sabbioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

terreno agrario sabbioso
terreno agrario sabbioso poco argilloso


In [ ]:
#all values containing a key word
keyW = 'siltiti e/o sabbie'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

siltiti e/o sabbie medio fini debolmente cementate


In [ ]:
#all values containing a key word
keyW = 'scorie e sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

scorie e sabbia media debolmente limosa con frammenti di laterizi


In [ ]:
#all values containing a key word
keyW = 'riporto prevalentemente sabbioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

riporto prevalentemente sabbioso ciottoloso con laterizi
riporto prevalentemente sabbioso con rara ghiaia e frammenti di laterizi
riporto prevalentemente sabbioso con rara ghiaia e frammenti laterizi
riporto prevalentemente sabbioso con subordinati clasti marnoso arenacei
riporto prevalentemente sabbioso ghiaioso
riporto prevalentemente sabbioso ghiaioso con abbondanti macerie e frammenti di laterizi
riporto prevalentemente sabbioso ghiaioso con limo e laterizi
riporto prevalentemente sabbioso ghiaioso con mattoni
riporto prevalentemente sabbioso limoso
riporto prevalentemente sabbioso limoso con abbondanti laterizi
riporto prevalentemente sabbioso limoso con frammenti di laterizi
riporto prevalentemente sabbioso limoso con ghiaia ciottoli e rari frammenti dilaterizi
riporto prevalentemente sabbioso sciolto con ghiaia ciottoli e frammenti dilaterizi


In [ ]:
#all values containing a key word
keyW = 'riporto di sabbie'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

riporto di sabbie
riporto di sabbie alterate
riporto di sabbie con abbondanti frammenti di mattoni
riporto di sabbie con ghiaia con frammenti di laterizi
riporto di sabbie e ghiaie
riporto di sabbie ghiaie argillose con frammenti di mattoni
riporto di sabbie limose con ghiaia eterometrica e mattoni frammenti di mattoni con presenza di alcuni livelli limoso sabbiosi
riporto di sabbie limose fini


In [ ]:
#all values containing a key word
keyW = 'riporto di sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

riporto di sabbia , ghiaia , ciottoli con frazione limosa
riporto di sabbia , ghiaia e ciottoli
riporto di sabbia a livelli con blocchi di cemento
riporto di sabbia a livelli ghiaiosa
riporto di sabbia argillosa con ghiaia
riporto di sabbia con breccia e resti
riporto di sabbia con ghiaia
riporto di sabbia con ghiaia , ghiaietto e rari ciottoli
riporto di sabbia con ghiaia con frammenti di calcestruzzo e laterizi
riporto di sabbia con ghiaia con frazione limosa
riporto di sabbia con ghiaia e ghiaietto
riporto di sabbia con ghiaia e rara frazione limosa
riporto di sabbia con ghiaia e rari ciottoli e frammenti di laterizi
riporto di sabbia con ghiaietto e rara ghiaia
riporto di sabbia con limo , ghiaiosa
riporto di sabbia debolmente argillosa con frammenti di laterizi
riporto di sabbia debolmente ghiaiosa con ciottoli
riporto di sabbia debolmente ghiaiosa con frammenti di calcestruzzo
riporto di sabbia debolmente ghiaiosa con rari ciottoli
riporto di sabbia debolmente ghiaiosa con rari c

In [ ]:
#all values containing a key word
keyW = 'riporto con sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

riporto con sabbia con ghiaia e ciottoli limosa
riporto con sabbia limosa , con ghiaia
riporto con sabbia limosa inglobante ghiaia , resti lateritici e rari ciottoli


In [ ]:
#all values containing a key word
keyW = 'riporto , sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

riporto , sabbia argillosa con ghiaia , ciottoli e resti laterizi
riporto , sabbia ghiaiosa con ciottoli , debolmente limosa
riporto , sabbia media limosa inglobante ghiaia


In [ ]:
#all values containing a key word
keyW = 'livello prevalentemente sabbioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

livello prevalentemente sabbioso
livello prevalentemente sabbioso con elevato grado di addensamento , poco coesivo
livello prevalentemente sabbioso con frammenti e resti legnosi , metallici
livello prevalentemente sabbioso con sporadici blocchi
livello prevalentemente sabbioso con sporadici blocchi ; presenza di detrito e resti
livello prevalentemente sabbioso con strati ghiaiosi
livello prevalentemente sabbioso molto addensato , poca coesivo


In [ ]:
#all values containing a key word
keyW = 'livelli di sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

livelli di sabbia debolmente limosa in alternanza con livelli di siltite
livelli di sabbia e argilla
livelli di sabbia e limo
livelli di sabbia fine debolmente limosa ossidata in alternanza con livelli limoso argillosi moderatamente consistente
livelli di sabbia fine in alternanza con livelli di argilla limosa ossidati con frammenti clasti sparsi di arenaria
livelli di sabbia in alternanza con livelli sabbioso limosi con rari ciottoli
livelli di sabbia limosa in alternanza con livelli marnoso argillosi moderatamente consistente
livelli di sabbia medio grossa limosa in alternanza con ciottoli poligenici rara ghiaia eterometrica poligenica


In [ ]:
#all values containing a key word
keyW = 'detrito misto a sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

detrito misto a sabbia
detrito misto a sabbia e limo
detrito misto a sabbia limosa


In [ ]:
#all values containing a key word
keyW = 'terreno di riporto sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

terreno di riporto sabbia con ciottoli e abbondanti laterizi
terreno di riporto sabbia con clasti rocciosi in matrice fine limosa e subordinati frammenti di laterizi
terreno di riporto sabbia con ghiaia
terreno di riporto sabbia con ghiaia e ciottoli
terreno di riporto sabbia con resti vegetali
terreno di riporto sabbia da limosa a con limo debolmente plastica poco consistente con ghiaia eterometrica poligenica parzialmente alterata
terreno di riporto sabbia debolmente limosa e ghiaia
terreno di riporto sabbia e ghiaia
terreno di riporto sabbia e ghiaia debolmente limosa mista a laterizi
terreno di riporto sabbia ghiaia blocchi e resti di laterizi
terreno di riporto sabbia ghiaia medio grossolana e scorie
terreno di riporto sabbia ghiaiosa con ciottoli eterometrici con matrice localmente limosa
terreno di riporto sabbia ghiaiosa con tracce di ciottoli
terreno di riporto sabbia ghiaiosa e ghiaia sabbiosa
terreno di riporto sabbia grossa e media da limosa a con limo e ghiaia eterometrica

In [ ]:
#all values containing a key word
keyW = 'terreno di riporto sabbioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

terreno di riporto sabbioso
terreno di riporto sabbioso con ciottoli
terreno di riporto sabbioso con detriti di laterizi
terreno di riporto sabbioso con ghiaia ciottoli mattoni laterizi e legno
terreno di riporto sabbioso con ghiaia, ciottoli, frammenti lateritici
terreno di riporto sabbioso con grossi ciottoli e trovanti
terreno di riporto sabbioso con rara ghiaia e piccoli frammenti di laterizi
terreno di riporto sabbioso debolmente limoso con clasti sparsi
terreno di riporto sabbioso debolmente limoso con frustoli vegetali frammenti di laterizi
terreno di riporto sabbioso debolmente limoso con ghiaia eterometrica
terreno di riporto sabbioso ghiaioso
terreno di riporto sabbioso ghiaioso a tratti limoso
terreno di riporto sabbioso ghiaioso argilloso con frammenti bituminosi e di laterizi
terreno di riporto sabbioso ghiaioso con abbondanti frammenti bituminosi
terreno di riporto sabbioso ghiaioso con ciottoli
terreno di riporto sabbioso ghiaioso con ciottoli di calcescisti moderatament

In [ ]:
#all values containing a key word
keyW = 'terreno di copertura sabbiosa'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

terreno di copertura sabbiosa limoso con frammenti lapidei poligenici e alcuni ciottoli


In [ ]:
#all values containing a key word
keyW = 'terreno di copertura sabbioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

terreno di copertura sabbioso limoso argilloso ghiaioso mediamente consistente


In [ ]:
#all values containing a key word
keyW = 'abbondante sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

abbondante sabbia argillosa con ciottoli e ghiaia
abbondante sabbia con ghiaia sparsa
abbondante sabbia fine con ghiaia medio piccola
abbondante sabbia fine con poca ghiaia sparsa


In [ ]:
#all values containing a key word
keyW = 'alternanza di livelli di sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

alternanza di livelli di sabbia addensata e di marna siltosa litoide frattura ossidata
alternanza di livelli di sabbia argillosa e di livelli di argilla marnoso scagliosa moderatamente consistente a volte plastica a tratti compressibile rare intercalazioni arenacee
alternanza di livelli di sabbia con livelli siltosi, moderatmaente consistente
alternanza di livelli di sabbia discretamente cementata e di marna siltosa
alternanza di livelli di sabbia e di argilla marnosa
alternanza di livelli di sabbia e livelli siltosi , poco consistente
alternanza di livelli di sabbia e livelli siltosi, moderatamente consistente
alternanza di livelli di sabbia fine con limo sabbioso rara ghiaia
alternanza di livelli di sabbia fine debolmente limosa ossidata e livelli limoso argillosi da poco a moderatamente consistente
alternanza di livelli di sabbia fine e di limo argilloso torba e subordinatamente ghiaia fine poco consistente debolmente plastico
alternanza di livelli di sabbia fine e di limo siltoso d

In [ ]:
#all values containing a key word
keyW = 'alternanza di livelli sabbiosi'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

alternanza di livelli sabbiosi con livelli molto argillosi
alternanza di livelli sabbiosi e arenacei
alternanza di livelli sabbiosi e arenacei con subordinati livelli marnosi
alternanza di livelli sabbiosi e livelli limoso argillosi
alternanza di livelli sabbiosi e marnoso siltosi
alternanza di livelli sabbiosi e sabbioso ghiaiosi, in matrice fine abbondante
alternanza di livelli sabbiosi e sabbioso limosi poco addensati


In [ ]:
#all values containing a key word
keyW = 'alternanza di passate di sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

alternanza di passate di sabbia addensata con laminazioni siltose e di marna compatta
alternanza di passate di sabbia addensata e di marna litoide con livelli arenacei con presenza di livelli fratturati
alternanza di passate di sabbia addensata e di marna litoide con rare intercalazioni arenacee
alternanza di passate di sabbia addensata e di marna litoide intercalazioni di arenaria molto fratturata
alternanza di passate di sabbia addensata e di marna siltoso litoide
alternanza di passate di sabbia fine limosa e limo sabbioso moderatamente consistente
alternanza di passate di sabbia fine ossidata e di argilla marnosa ossidata alterata moderatamente consistente
alternanza di passate di sabbia medio fine a volte grossolana debolmente cementata e di arenaria con subordinata marna siltosa , litoide presenza di fratture
alternanza di passate di sabbia medio fine debolmente cementata , arenaria e marna siltosa , litoide
alternanza di passate di sabbia medio fine debolmente cementata e di marn

In [ ]:
#all values containing a key word
keyW = 'alternanza di sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

alternanza di sabbia addensata arenaria e marna siltosa litoidi fratture ossidate
alternanza di sabbia addensata e arenaria litoide con livelli marnosi in subordine
alternanza di sabbia addensata e arenaria litoide con passate di marna siltosa fratture ossidate
alternanza di sabbia addensata e arenaria litoide con subordinata marna fratture ossidate
alternanza di sabbia addensata e argilla
alternanza di sabbia addensata e argilla marnosa
alternanza di sabbia addensata e di marna siltosa litoide
alternanza di sabbia addensata e di marna siltosa litoide con presenza di frattura
alternanza di sabbia addensata e marna siltosa litoide con passate di arenaria litoide in subordine e fratture
alternanza di sabbia addensata e marna siltosa litoide fratture con ossidazione
alternanza di sabbia arenaria e marna fratture con ossidazione
alternanza di sabbia cementata con livelli di microconglomerato marna litoide e arenaria in subordine
alternanza di sabbia cementata con subordinati livelli arenac

In [ ]:
#all values containing a key word
keyW = 'alternanza di sabbie'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

alternanza di sabbie con ghiaia e ciottoli e sabbie grossolane con ghiaietto
alternanza di sabbie con ghiaie e di sabbie limose debolmente argillose
alternanza di sabbie debolmente ghiaiose e sabbie con ghiaia
alternanza di sabbie debolmente ghiaiose e sabbie con ghiaia eterometrica
alternanza di sabbie debolmente ghiaiose e sabbie con ghiaie eterometriche e ciottoli
alternanza di sabbie e arenarie
alternanza di sabbie e sabbie ghiaiose con rari ciottoli con presenza di livelli con clasti alterati e/o debolmente cementati
alternanza di sabbie fini limose e argilla
alternanza di sabbie fini limose e sabbie ghiaiose
alternanza di sabbie ghiaiose e presenza di matrice fine limosa con sostanza organica limosa plastica
alternanza di sabbie limose e di ghiaie
alternanza di sabbie poco addensate


In [ ]:
#all values containing a key word
keyW = 'alternanza disabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

alternanza disabbia debolmente cementata emarna debolmente siltosa con arenaria litoide fratture ossidate
alternanza disabbia debolmente limosa moderatamente addensata con passate di conglomerato
alternanza disabbia fine debolmente limosa moderatamente addensata e dilimo sabbioso moderatamente consistente livelli ossidati
alternanza disabbia fine e dimarna siltosa con subordinati livelli diarenaria litoide con passate fratturate
alternanza disabbia fine emarna debolmente siltosa con arenaria litoide in subordine con presenza di fratture
alternanza disabbia fine emarna siltosa a volte argillosa litoide
alternanza disabbia fine ossidata e livelli limoso argillosi rari frammenti carboniosi poco consistente
alternanza disabbia fine ossidata e sabbia fine limosa moderatamente addensata
alternanza disabbia medio fine addensata e dimarna siltosa litoide con passate fratturate
alternanza disabbia medio fine debolmente cementata earenaria con livelli marnoso argillosi alterati ossidati in subor

In [ ]:
#all values containing a key word
keyW = 'alternanze di sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

alternanze di sabbia addensata e argilla ed argilla limosa
alternanze di sabbia con ghiaia debolmente limosa e ghiaia con sabbia debolmente limosa
alternanze di sabbia con ghiaia e ghiaia con sabbia con subordinata frazione limosa
alternanze di sabbia e argilla
alternanze di sabbia e marna siltosa compatta
alternanze di sabbia e sabbione
alternanze di sabbia eterometrica debolmente limosa e piccoli trovanti con ciottoli
alternanze di sabbia fine limosa e di limo sabbioso
alternanze di sabbia fine limosa limo argilloso e alcuni rari livelli di torba
alternanze di sabbia fine media e limi sabbiosi
alternanze di sabbia ghiaiosa e ghiaia sabbiosa , presenza di rari ciottoli
alternanze di sabbia ghiaiosa e ghiaia sabbiosa ; presenza di rari ciottoli
alternanze di sabbia ghiaiose e ghiaia sabbiosa ; presenza di rari ciottoli
alternanze di sabbia grossa con ghiaia eterometrica e sabbia medio fine debolmente limosa inglobante ghiaia fine
alternanze di sabbia limosa passante a limo sabbioso ing

In [ ]:
#all values containing a key word
keyW = 'alternanze di sabbie'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

alternanze di sabbie e argille
alternanze di sabbie e ghiaie medie e grossolane con ciottoli
alternanze di sabbie fini , sabbia fine limosa , limo argilloso e argilla limosa molto addensati , presenza di arenaria debolmente cementata
alternanze di sabbie fini e di argille limose plastiche
alternanze di sabbie fini e limi , a tratti sabbioso fini , addensate
alternanze di sabbie fini limose e limi
alternanze di sabbie fini limose e limi argillosi e/o argille limose
alternanze di sabbie fini limose e limi da sabbiosi a debolmente argillosi
alternanze di sabbie limose , limi e limi argillosi con ghiaie serpentinitiche e ciottoli
alternanze di sabbie limose addensata , limi e limi argillosi
alternanze di sabbie limose e limi sabbiosi con ciottoli
alternanze di sabbie marnose e di arenarie cementate
alternanze di sabbie medio fini sabbie limose e limi sabbiosi


In [ ]:
#all values containing a key word
keyW = 'cappellaccio sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

cappellaccio sabbia fine limoso argillosa inglobante ciottoli poco addensata
cappellaccio sabbia fine limoso argillosa molto addensata


In [ ]:
#all values containing a key word
keyW = 'conglomerato di sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

conglomerato di sabbia
conglomerato di sabbia e argilla compatta
conglomerato di sabbia e ciottoli
conglomerato di sabbia e ghiaia


In [ ]:
#all values containing a key word
keyW = 'conglomerato sabbioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

conglomerato sabbioso
conglomerato sabbioso a piccoli strati di ghiaia
conglomerato sabbioso cementato
conglomerato sabbioso ghiaioso
conglomerato sabbioso in alternanza a strati con sabbia
conglomerato sabbioso limoso con ghiaia e ciottoli


In [ ]:
#all values containing a key word
keyW = 'riporto costituito da sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

riporto costituito da sabbia
riporto costituito da sabbia a livelli ghiaiosa
riporto costituito da sabbia ciottolosa debolmente ghiaiosa sciolta , con frazioni sabbiosa fine
riporto costituito da sabbia con ghiaia
riporto costituito da sabbia con ghiaia , ciottoli , limosa
riporto costituito da sabbia con ghiaia e mattoni
riporto costituito da sabbia debolmente limosa addensata con ghiaia , ghiaietto , pietre e resti lateritici
riporto costituito da sabbia debolmente limosa con ciottoli
riporto costituito da sabbia debolmente limosa con raro ghiaietto sparso e sporadici frammenti lateritici, poco addensato
riporto costituito da sabbia debolmente limosa con raro ghiaietto sparso, poco addensato
riporto costituito da sabbia debolmente limosa e consistente con poca ghiaia e frammenti di laterizi, ferro
riporto costituito da sabbia debolmente limosa, raro ghiaietto sparso, poco consistente
riporto costituito da sabbia e clasti con limo
riporto costituito da sabbia e ghiaia
riporto costitui

In [ ]:
#all values containing a key word
keyW = 'riporto sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

riporto sabbia con ciottoli e resti di laterizi
riporto sabbia con ciottoli poligenici con presenza di resti di laterizi
riporto sabbia con ghiaia
riporto sabbia con resti di laterizi e vegetali
riporto sabbia debolmente limosa con ghiaia ghiaietto e ciottoli
riporto sabbia debolmente limosa con ghiaia ghiaietto resti di laterizi
riporto sabbia debolmente limosa inglobante ghiaia ghiaietto e resti vegetali
riporto sabbia debolmente limosa inglobante ghiaietto ghiaia e ciottoli
riporto sabbia debolmente limosa inglobante ghiaietto rara ciottoli mattoni
riporto sabbia debolmente limosa inglobante ghiaietto resti vegetali mattoni e rara ghiaia
riporto sabbia debolmente limosa limo argilloso rara ghiaia con resti di mattoni e laterizi
riporto sabbia e ghiaia
riporto sabbia fine debolmente limosa con breccia e ciottoli
riporto sabbia fine debolmente limosa inglobante ghiaia ghiaietto e rari ciottoli
riporto sabbia fine debolmente limosa inglobante ghiaietto e ghiaia
riporto sabbia fine limo

In [ ]:
#all values containing a key word
keyW = 'riporto sabbioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

riporto sabbioso
riporto sabbioso , presenza di locale detrito di laterizi e resti
riporto sabbioso argilloso con presenza di ghiaie, pietre
riporto sabbioso ciottoloso
riporto sabbioso ciottoloso con presenza di ciottoli
riporto sabbioso ciottoloso con presenza di ciottoli e laterizi
riporto sabbioso ciottoloso con tracce di mattoni
riporto sabbioso con breccia
riporto sabbioso con ciottoli
riporto sabbioso con ghiaia
riporto sabbioso con ghiaia ciottolosa e frammenti di mattoni
riporto sabbioso con limo ghiaia e calcestruzzo in frammenti moderatamente addensato
riporto sabbioso con mattoni e ciottoli
riporto sabbioso con rara ghiaia e ciottoli
riporto sabbioso con rari frammenti di laterizio
riporto sabbioso debolmente ghiaioso
riporto sabbioso debolmente limoso con ghiaietto
riporto sabbioso e subordinatamente a tratti argilloso con abbondante scheletro clastica con rara presenza di laterizi
riporto sabbioso fine
riporto sabbioso fine argilloso e argilloso limoso
riporto sabbioso fi

In [ ]:
#all values containing a key word
keyW = 'strato sabbioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

strato sabbioso
strato sabbioso debolmente limoso
strato sabbioso limoso argilloso con subordinato scheletro ghiaioso medio fine
strato sabbioso limoso con poca ghiaia
strato sabbioso limoso con rari ciottoli e resti vegetali
strato sabbioso micaceo con livello di alterazione livelli limosi olimo sabbiosi


In [ ]:
#all values containing a key word
keyW = 'suolo sabbioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

suolo sabbioso a tratti limoso con resti vegetali
suolo sabbioso con presenza di resti vegetali
suolo sabbioso e abbondanti frammenti vegetali
suolo sabbioso elementi lapidei medi con resti vegetali
suolo sabbioso ghiaioso
suolo sabbioso limoso con clasti di serpentiniti alterate e frammenti di laterizi
suolo sabbioso limoso con rari ciottoli
suolo sabbioso limoso con resti vegetali


In [ ]:
#all values containing a key word
keyW = 'terreno di riporto sabbie'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

terreno di riporto sabbie limose
terreno di riporto sabbie limose e ciottoli granitici
terreno di riporto sabbie limose ghiaie detriti e frammenti di laterizi


In [ ]:
#all values containing a key word
keyW = 'terreno diriporto sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

terreno diriporto sabbia limosa con frammenti di laterizi ciottoli e livelli sabbioso ghiaiosi
terreno diriporto sabbia limosa localmente limo sabbioso eghiaia eterometrica poligenica


In [ ]:
#all values containing a key word
keyW = 'terreno diriporto sabbioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

terreno diriporto sabbioso argilloso con resti vegetali elaterizi
terreno diriporto sabbioso ciottoloso poco addensato con frammenti
terreno diriporto sabbioso ghiaioso
terreno diriporto sabbioso ghiaioso con mattoni
terreno diriporto sabbioso ghiaioso limoso
terreno diriporto sabbioso limoso addensato con sparsi ciottoli
terreno diriporto sabbioso limoso con ciottoli e frammenti dilaterizi
terreno diriporto sabbioso limoso con frammenti di mattoni poco addensato
terreno diriporto sabbioso limoso con frustoli vegetali e frammenti dilaterizi
terreno diriporto sabbioso limoso poco addensato
terreno diriporto sabbioso terroso con presenza di mattoni e livelli plastici
terreno diriporto sabbioso terroso con tracce di mattoni


In [ ]:
#all values containing a key word
keyW = 'terreno rimaneggiato prevalentemente sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

terreno rimaneggiato prevalentemente sabbia eterometrica con rari clasti di ghiaia
terreno rimaneggiato prevalentemente sabbia eterometrica e ghiaia


In [ ]:
#all values containing a key word
keyW = 'terreno rimaneggiato prevalentemente sabbioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

terreno rimaneggiato prevalentemente sabbioso con ciottoli
terreno rimaneggiato prevalentemente sabbioso con clasti e blocchi arenacei
terreno rimaneggiato prevalentemente sabbioso debolmente limoso
terreno rimaneggiato prevalentemente sabbioso limoso
terreno rimaneggiato prevalentemente sabbioso limoso con clasti marnoso arenacei moderatamente consistente
terreno rimaneggiato prevalentemente sabbioso limoso con frammenti di laterizi , clasti e radici


In [ ]:
#all values containing a key word
keyW = 'riporto costituito da sabbie'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

riporto costituito da sabbie con ghiaietto e ghiaie e limi
riporto costituito da sabbie ghiaiose debolmente limose con ciottoli
riporto costituito da sabbie ghiaiose limose con frammenti lateritici
riporto costituito da sabbie limose
riporto costituito da sabbie limose con ghiaia e ciottoli ; presenza di rari ciottoli


In [ ]:
#all values containing a key word
keyW = 'materiale di riporto sabbioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

materiale di riporto sabbioso argilloso con abbondanti frammenti di laterizi
materiale di riporto sabbioso ciottoloso con alcuni mattoni
materiale di riporto sabbioso ghiaioso ciottoloso con presenza di legno
materiale di riporto sabbioso ghiaioso con ciottoli e abbondanti scorie
materiale di riporto sabbioso ghiaioso limoso argilloso
materiale di riporto sabbioso limoso con ciottoli


In [ ]:
#all values containing a key word
keyW = 'materiale di riporto sabbie'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

materiale di riporto sabbie debolmente limose inglobanti rare ghiaie e ciottoli misto a rifiuti
materiale di riporto sabbie medio fini debolmente ghiaiose miste a frazioni limose debolmente argillose moderatamente consistenti con presenza di frammenti di mattoni


In [ ]:
#all values containing a key word
keyW = 'materiale di riporto sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

materiale di riporto sabbia eterometrica limosa con frammenti di laterizi
materiale di riporto sabbia ghiaiosa eterometrica passante a ghiaia eterometrica sabbiosa passante a ciottoli poligenici


In [ ]:
#all values containing a key word
keyW = 'detrito con matrice sabbiosa'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

In [ ]:
#all values containing a key word
keyW = 'detrito con matrice sabbiosa'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

In [ ]:
#all values containing a key word
keyW = 'terreno di riporto , sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

terreno di riporto , sabbia argillosa
terreno di riporto , sabbia argillosa con resti lateritici e cementizio
terreno di riporto , sabbia e limo con ciottoli laterizi e materiale plastico
terreno di riporto , sabbia fine da sabbiosa a con limo , debolmente plastico , poco addensata , poco consistente con ghiaia eterometrica e ciottoli
terreno di riporto , sabbia fine limosa
terreno di riporto , sabbia limosa con elementi litoidi
terreno di riporto , sabbia media , da limosa a con limo , poco addensato con ghiaia eterometrica poligenica , ciottoli
terreno di riporto , sabbia media e fine e ghiaia eterometrica poligenica con presenza di frammenti di laterizi
terreno di riporto , sabbia media e fine limosa , poco addensato con ghiaia eterometrica e ciottoli


In [ ]:
#all values containing a key word
keyW = 'terreno di riporto prevalentemente sabbioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

terreno di riporto prevalentemente sabbioso
terreno di riporto prevalentemente sabbioso con clasti di calcare e frammenti di laterizi
terreno di riporto prevalentemente sabbioso con clasti eterometrici di calcare e frammenti di laterizi
terreno di riporto prevalentemente sabbioso con clasti frammenti di laterizi
terreno di riporto prevalentemente sabbioso con frammenti di laterizi
terreno di riporto prevalentemente sabbioso con ghiaia ciottoli e frammenti di laterizi
terreno di riporto prevalentemente sabbioso con ghiaia e frammenti di laterizi
terreno di riporto prevalentemente sabbioso fine con laterizi molto alterato e scarsamente addensato con presenza di resti vegetali
terreno di riporto prevalentemente sabbioso ghiaioso
terreno di riporto prevalentemente sabbioso ghiaioso con ciottoli
terreno di riporto prevalentemente sabbioso ghiaioso con ciottoli e frammenti di laterizi
terreno di riporto prevalentemente sabbioso ghiaioso con frammenti di laterizi
terreno di riporto prevalente

In [ ]:
#all values containing a key word
keyW = 'terreno di riporto prevalentemente sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

terreno di riporto prevalentemente sabbia limosa


In [ ]:
#all values containing a key word
keyW = 'terreno di riporto di sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

terreno di riporto di sabbia con ghiaia
terreno di riporto di sabbia limoso e/o limo sabbioso con frammenti di mattoni sparsi
terreno di riporto di sabbia medio fine con ghiaia eterometrica poligenica con presenza di calcestruzzo


In [ ]:
#all values containing a key word
keyW = 'terreno di riporto da sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

terreno di riporto da sabbia fine con raro ghiaietto sparso, poco addensato


In [ ]:
#all values containing a key word
keyW = 'riporto grossolano in matrice sabbiosa'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

In [ ]:
#all values containing a key word
keyW = 'riporto grossolano con matrice sabbioso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

In [ ]:
#all values containing a key word
keyW = 'terreno di riporto costituito da sabbie'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

terreno di riporto costituito da sabbie con ghiaietto e ghiaie e limi
terreno di riporto costituito da sabbie limose con rari ciottoli


In [ ]:
#all values containing a key word
keyW = 'terreno di riporto costituito da sabbia'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=700
    key_list[position]='Sabbia'

terreno di riporto costituito da sabbia , da limosa a con limo, e ghiaia, poligenica; alterata;presenza di frammenti laterizi
terreno di riporto costituito da sabbia con clasti e subordinati frammenti lateritici
terreno di riporto costituito da sabbia con ghiaia ciottolosa, con rari frammenti laterizi
terreno di riporto costituito da sabbia con limo e ghiaia
terreno di riporto costituito da sabbia con limo, ghiaia e frammenti lateritici
terreno di riporto costituito da sabbia con rara ghiaia, poligenica, qualche ciottolo; presenza di frammenti laterizi
terreno di riporto costituito da sabbia debolmente ghiaiosa con rari frammenti laterizi
terreno di riporto costituito da sabbia debolmente limosa con frazione ghiaiosa, poco addensato
terreno di riporto costituito da sabbia debolmente limosa con subordinata frazione ghiaiosa, poco addensata
terreno di riporto costituito da sabbia e frammenti rocciosi
terreno di riporto costituito da sabbia e ghiaia con frammenti di asfalto
terreno di rip

In [ ]:
#all values containing a key word
keyW = 'coltre argilloso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

coltre argilloso sabbiosa fine


In [ ]:
#all values containing a key word
keyW = 'coltre diargilla'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

coltre diargilla con sabbia fine
coltre diargilla sabbiosa


In [ ]:
#all values containing a key word
keyW = 'conglomerato argilloso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

conglomerato argilloso
conglomerato argilloso compatto
conglomerato argilloso compatto con frammenti eterometrici addensati
conglomerato argilloso con ciottoli


In [ ]:
#all values containing a key word
keyW = 'copertura argilloso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

copertura argilloso limosa
copertura argilloso limosa con ciottoli
copertura argilloso limosa debolmente sabbiosa
copertura argilloso limoso sabbiosa


In [ ]:
#all values containing a key word
keyW = 'da argilla'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

da argilla a argilla marnosa
da argilla marnosa a marna argillosa
da argilla tenere a compatta


In [ ]:
#all values containing a key word
keyW = 'deposito alluvionale argilloso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

deposito alluvionale argilloso e argilloso limosa
deposito alluvionale argilloso e argilloso limoso
deposito alluvionale argilloso e argilloso limoso con ciottoli
deposito alluvionale argilloso limoso con scarsi ciottoli
deposito alluvionale argilloso siltoso


In [ ]:
#all values containing a key word
keyW = 'deposito argilloso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

deposito argilloso
deposito argilloso compatto
deposito argilloso compatto con presenza di alcuni elementi ciottolosi
deposito argilloso e argilloso limoso
deposito argilloso e argilloso limoso con ciottoli
deposito argilloso limoso
deposito argilloso plastico con presenza di frazione sabbiosa
deposito argilloso sabbiosi


In [ ]:
#all values containing a key word
keyW = 'detrito limoso argilloso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

detrito limoso argilloso
detrito limoso argilloso con ciottoli e ghiaia
detrito limoso argilloso con ciottoli in livelli sabbiosi e ghiaiosi


In [ ]:
#all values containing a key word
keyW = 'distruzione argilla'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

distruzione argilla
distruzione argilla debolmente limosa
distruzione argilla debolmente marnosa
distruzione argilla e/o argilla siltosa
distruzione argilla marnosa emarna argillosa


In [ ]:
#all values containing a key word
keyW = 'litotipi argillosi'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

litotipi argillosi
litotipi argillosi a tratti sabbiosi ; presenza di torba
litotipi argillosi e argilloso sabbiosi


In [ ]:
#all values containing a key word
keyW = 'livello argilloso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

livello argilloso
livello argilloso con intercalazioni di livelli ghiaiosi
livello argilloso con presenza difossili
livello argilloso con sabbie
livello argilloso detritico e scaglioso
livello argilloso detritico e scaglioso con ciottoli e ghiaie debolmente plastico
livello argilloso detritico e scaglioso debolmente plastico
livello argilloso e argilloso limoso
livello argilloso e limoso argilloso
livello argilloso e limoso argilloso con piccoli ciottoli
livello argilloso e limoso argilloso molto plastico con piccoli ciottoli
livello argilloso grafitico con rari clasti eterometrici sparsi di calcescisti poco consistente da debolmente plastico a plastico e compressibile
livello argilloso limoso mediamente plastico
livello argilloso marnoso compatto
livello argilloso marnoso ossidato alterato
livello argilloso marnoso plastico
livello argilloso marnoso poco sabbioso plastico
livello argilloso plastico
livello argilloso plastico con scarsi frammenti di calcescisto


In [ ]:
#all values containing a key word
keyW = 'livello di argilla'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

livello di argilla
livello di argilla compata
livello di argilla compatta
livello di argilla con inclusi frammenti lapidei di metargilliti
livello di argilla consistente


In [ ]:
#all values containing a key word
keyW = 'materiale argilloso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

materiale argilloso
materiale argilloso con ghiaia eciottoli
materiale argilloso con ghiaie fini eciottoli quarzosi
materiale argilloso con presenza di ciottoli
materiale argilloso con presenza di strati diciottoli in alternanza conmateriale argilloso con presenza di trovanti quarzosi
materiale argilloso di scisto alterato
materiale argilloso limoso
materiale argilloso limoso con presenza di piccoli strati dicalcescisti emicascisti e dighiaia ciottoli
materiale argilloso limoso con rari clasti
materiale argilloso limoso plastico compatto
materiale argilloso molto plastico con frammenti di calcescisti eciottoli
materiale argilloso sabbioso


In [ ]:
#all values containing a key word
keyW = 'matrice argillosa'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

In [ ]:
#all values containing a key word
keyW = 'matrice silt argillosa'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

matrice silt argillosa inglobante abbondante sabbie grossolane ghiaie medie fini a volte ciottoli
matrice silt argillosa inglobante ghiaia prevalentemente sabbie a volte ciottoli
matrice silt argillosa inglobante ghiaie sabbie a volte ciottoli
matrice silt argillosa inglobante sabbie grosse e ciottoli sparsi
matrice silt argillosa inglobante sabbie grossolane e medie con presenza di ghiaie medie e fini e ciottoli sparsi
matrice silt argillosa sabbiosa inglobante ghiaia medie grosse e ciottoli
matrice silt argillosa sabbiosa inglobante ghiaie da medie a grosse


In [ ]:
#all values containing a key word
keyW = 'metargilliti'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

metargilliti compatte con intercalazioni di calcari cristallini e livelli di quarziti carbonatiche fratture a riempimento argilloso con tracce di ossidazione
metargilliti compatte con intercalazioni di calcari cristallini fratture con tracce di ossidazione e di riempimenti argillosi
metargilliti compatte con interstrati di calcari cristallini e livelli di quarziti carbonatiche fratture a riempimento argilloso con tracce di ossidazione
metargilliti compatte e poco fratturate con presenza di quarziti carbonatici in piccoli strati fratture a riempimento argilloso
metargilliti con presenza di abbondante argilla e ciottoli molto alterati
metargilliti frantumate
metargilliti molto alterate passanti a fratturate
metargilliti molto alterato con presenza di argilla


In [ ]:
#all values containing a key word
keyW = 'riporto argilla'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

riporto argilla con noduli di calcare
riporto argilla limosa sabbiosa con presenza di frammenti di laterizi
riporto argilla limoso sabbiosa con ghiaia medio grossa e resti di mattoni
riporto argilla plastica sabbiosa con ciottoli


In [ ]:
#all values containing a key word
keyW = 'riporto prevalentemente argilloso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

riporto prevalentemente argilloso
riporto prevalentemente argilloso compatto con tracce di mattoni
riporto prevalentemente argilloso sabbioso con frammenti di mattoni
riporto prevalentemente argilloso sabbioso con ghiaia elaterizi con presenza disabbia fine con tracce di alterazione


In [ ]:
#all values containing a key word
keyW = 'strato argilloso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

strato argilloso
strato argilloso limoso inglobante livelli ghiaiosi
strato argilloso plastico e poco consistente


In [ ]:
#all values containing a key word
keyW = 'strato di argilla'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

strato di argilla
strato di argilla sabbiosa compatta


In [ ]:
#all values containing a key word
keyW = 'substrato , argilla'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

substrato , argilla marnosa


In [ ]:
#all values containing a key word
keyW = 'substrato alterato , argilla'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

substrato alterato , argilla
substrato alterato , argilla con calcari cariati e marne siltose in piccoli livelli
substrato alterato , argilla con gesso alterato
substrato alterato , argilla con inclusi alterati e/o sfatti con locali ciottoli calcarei
substrato alterato , argilla con livelli di calcare cariato sfatto
substrato alterato , argilla con livelli di calcari cariati sfatti e con presenza di resti vegetali
substrato alterato , argilla con rari frammenti calcarei
substrato alterato , argilla limosa
substrato alterato , argilla marnosa
substrato alterato , argilla marnosa con ghiaia e ciottoli calcarei


In [ ]:
#all values containing a key word
keyW = 'substrato argilloso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

substrato argilloso
substrato argilloso marnoso
substrato argilloso marnoso e sabbioso


In [ ]:
#all values containing a key word
keyW = 'substrato siltite argillosa'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

substrato siltite argillosa
substrato siltite argillosa molto stratificata


In [ ]:
#all values containing a key word
keyW = 'suolo argilloso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

suolo argilloso
suolo argilloso sabbioso
suolo argilloso sabbioso con resti vegetali


In [ ]:
#all values containing a key word
keyW = 'terreno argilloso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

terreno argilloso
terreno argilloso compatto
terreno argilloso con frammenti litoidi
terreno argilloso con ghiaia
terreno argilloso con materiale vegetale
terreno argilloso con pietre
terreno argilloso debolmente sabbioso fine poco compatto con resti vegetali
terreno argilloso ghiaioso
terreno argilloso limoso
terreno argilloso limoso compatto
terreno argilloso limoso compatto con alcuni elementi ghiaiosi
terreno argilloso limoso con alcuni trovanti di scisto sericitico molto alterati
terreno argilloso limoso con trovanti di scisto molto alterato
terreno argilloso limoso con trovanti di scisto sericitico poco compatto
terreno argilloso limoso e sabbioso
terreno argilloso limoso plastico
terreno argilloso misto a ghiaia
terreno argilloso sabbioso
terreno argilloso sabbioso inglobante ciottoli e ghiaie
terreno argilloso sabbioso limoso con raro ghiaietto resti vegetali molto alterato con tracce di ossidazione poco compatto
terreno argilloso sciolto
terreno argilloso vegetale


In [ ]:
#all values containing a key word
keyW = 'terreno di riporto argilla'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

terreno di riporto argilla limosa con poco ghiaietto e alcuni frammenti di mattoni molto plastico
terreno di riporto argilla limosa molto sabbiosa


In [ ]:
#all values containing a key word
keyW = 'terreno di riporto prevalentemente argilloso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

terreno di riporto prevalentemente argilloso
terreno di riporto prevalentemente argilloso limoso con presenza di scorie
terreno di riporto prevalentemente argilloso sabbioso


In [ ]:
#all values containing a key word
keyW = 'terreno diriporto argilloso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

terreno diriporto argilloso
terreno diriporto argilloso debolmente sabbioso con frammenti di laterizi
terreno diriporto argilloso limoso
terreno diriporto argilloso limoso con ghiaia


In [ ]:
#all values containing a key word
keyW = 'alternanza di marna'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

alternanza di marna alterata argillificata a livelli arenacei
alternanza di marna arenacea compatta stratificata e di arenaria cementata
alternanza di marna arenacea compatta stratificata e di arenaria fine marnosa mediamente cementata debolmente fratturata
alternanza di marna arenaria e sabbia
alternanza di marna argillosa consistente e arenaria litoide frattura ossidata
alternanza di marna argillosa e argilla marnosa con intercalazioni di sabbia e arenaria carbonatica compatta localmente molto fratturate
alternanza di marna argillosa e di marna debolmente sabbiosa con presenza di livelli di arenaria
alternanza di marna argillosa e di marna siltosa
alternanza di marna argillosa e di sabbia fine a volte ossidata da moderatamente consistente a molto consistente
alternanza di marna argillosa e di sabbia limosa poco consistente
alternanza di marna argillosa e di sabbia medio fine consistente
alternanza di marna argillosa e di sabbia medio fine limosa con subordinata arenaria da moderatame

In [ ]:
#all values containing a key word
keyW = 'alternanza di strati di marna'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

alternanza di strati di marna siltosa litoide e di sabbia addensata raramente cementata


In [ ]:
#all values containing a key word
keyW = 'calcescisti'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

calcescisti
calcescisti , fratture debolmente ossidate con locali riempimenti sabbiosi
calcescisti , fratture moderatamente ossidate con presenza di riempimento , rari livelli da molto fratturati a cataclasati
calcescisti a frammenti ghiaiosi con patine di alterazione
calcescisti a tratti fratturati
calcescisti a tratti molto fratturati
calcescisti a volte con lenti quarzo albitici debolmente grafitici fratturato fratture con riempimento
calcescisti a volte con vene albitico quarzose
calcescisti a volte degradati ossidati con alcuni interstrati argillosi con vene di calcite alcuni noduli quarzo
calcescisti alterate scagliettati con vene quarzose
calcescisti alterati
calcescisti alterati argillificati
calcescisti alterati con lenti fratturate vene di quarzo
calcescisti alterati e/o sfatti a volte argillificati
calcescisti alterati fessurati cappellaccio
calcescisti alterati fratturati
calcescisti alterati fratturati cappellaccio
calcescisti alterati molto fratturati con livelli parzialm

In [ ]:
#all values containing a key word
keyW = 'calcescisto'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

calcescisto
calcescisto , fratture con locali riempimenti sabbiosi e argillosi , locali vene di calcite
calcescisto , locali noduli e vene di calcite , con presenza di fratture da debolmente ossidate a ossidate , alcune fratture a riempimento limoso argilloso
calcescisto , roccia orientata , fratture moderatamente alterate con riempimento , con presenza di alcuni livelli molto fratturati
calcescisto a livelli grafitico molto alterato a tratti cataclastico con subordinati livelli litoidi
calcescisto a livelli grafitico molto alterato e ossidato cataclastico
calcescisto a tratti da debolmente grafitico a grafitico intensa alterato e argillificato da moderatamente consistente a molto consistente
calcescisto a tratti fratturato
calcescisto a tratti grafitico , litoide , debole grado di ossidazione
calcescisto a tratti grafitico con intercalati livelli quarzitici litoide
calcescisto a tratti grafitico in alternanza con livelli quarzosi litoide fratture con riempimento argilloso
calcescisto 

In [ ]:
#all values containing a key word
keyW = 'calcare'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

calcare
calcare alterato
calcare arenaceo
calcare arenaceo litoide con vene di calcite
calcare ben cementato
calcare compatto
calcare compatto con abbondanti concrezioni calcaree
calcare compatto con concrezioni calcaree
calcare con fratturazione
calcare con fratture ossidate
calcare con vene di calcite con presenza di fratture
calcare cristallino
calcare cristallino con livelli grafitico scistosi
calcare cristallino molto fratturato
calcare da mediamente a poco fratturato
calcare da molto fratturato a compatto
calcare debolmente fratturato
calcare debolmente fratturato con presenza di calcare marnoso
calcare debolmente marnoso con subordinati livelli a matrice più marnosa e con passate lapidee intensamente fratturate, sub-litoide
calcare debolmente marnoso intensamente fratturato
calcare debolmente marnoso intensamente fratturato con sporadici livelli di marna argilloso-siltosa sub-litoide
calcare debolmente marnoso, litoide , intensamente fratturato
calcare debolmente marnoso, litoid

In [ ]:
#all values containing a key word
keyW = 'blocchi'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=306
    key_list[position]='Blocchi, trovanti, erratici'

blocchi
blocchi arenacei e/o marnoso arenacei con limo argilloso debolmente sabbioso , poco consistente
blocchi calcarei
blocchi calcarei e strati in scarsa matrice argillosa
blocchi ciottoli e ghiaia con abbondante sabbia
blocchi ciottoli e sabbie
blocchi ciottoli eghiaia dicalcescisti
blocchi clasti di calcescisti eterometrici in matrice abbondante limoso sabbiosa
blocchi clasti di calcescisti in scarsa matrice fine argillificazione con presenza di livelli argillificati
blocchi clasti eterometrici litoidi insabbia limosa
blocchi clasti rocciosi eterometrici prevalentemente di calcescisti scarsa matrice limoso sabbiosa debole ossidazione
blocchi clasti rocciosi insabbia limosa addensata
blocchi con fratture
blocchi con fratture con alcuni livelli di ghiaia debolmente sabbioso limosa
blocchi con fratture e debole alterazione
blocchi con fratture e debole alterazione con intercalazioni di livelli ghiaiosi ciottolosi
blocchi con ghiaia in scarsa matrice sabbioso-limosa, addensata
blocchi

In [ ]:
#all values containing a key word
keyW = 'blocco di marna'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

In [ ]:
#all values containing a key word
keyW = 'blocco di calcare'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

In [ ]:
#all values containing a key word
keyW = 'blocco di calcescisto '
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=201
    key_list[position]='Calcare'

blocco
blocco arenaceo
blocco arenaceo fratturato con tracce di ossidazione
blocco calcareo
blocco calcareo e spezzoni
blocco con fratture
blocco con ghiaia e sabbia
blocco debolmente fratturato
blocco di arenaria
blocco di arenaria lapideo
blocco di calcare marnoso
blocco di calcare marnoso con fratture a riempimento argilloso
blocco di calcare marnoso lapideo
blocco di calcare marnoso litoide
blocco di calcare marnoso molto fratturato a volte sfatto
blocco di calcare molto fratturato con vene di calcite
blocco di calcescisto
blocco di calcescisto , massiccio , fratturato
blocco di calcescisto a tratti alterato e fratturato
blocco di calcescisto a vene di quarzo
blocco di calcescisto alterato argillificato e ossidato
blocco di calcescisto alterato e fratturato
blocco di calcescisto alterato fratturato
blocco di calcescisto compatto
blocco di calcescisto completamente alterato e argillificato
blocco di calcescisto con abbondante quarzo
blocco di calcescisto con albitico quarzose vene a

In [ ]:
#all values containing a key word
keyW = 'blocco'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=306
    key_list[position]='Blocchi, trovanti, erratici'

In [ ]:
#all values containing a key word
keyW = 'arenarie'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=100
    key_list[position]='Argilla'

arenarie
arenarie , clasti medio fine
arenarie alterate
arenarie alterate in sabbia ghiaiosa
arenarie argillose con presenza di alcune fratture
arenarie argillose con presenza di alcune fratture a riempimento sabbioso
arenarie ben cementate
arenarie ben cementate a moderatamente cementate
arenarie ben cementate con intercalazioni di marne
arenarie ben cementate con livelli di marne limose
arenarie ben cementate fratturate
arenarie ben cementate in alternanza conmarne poco consistenti
arenarie ben cementate passanti amarne
arenarie ben cementate passanti amarne limose con livelli sabbiosi
arenarie calcaree ben cementate
arenarie cementate
arenarie cementate con intercalazioni di livelli sabbiosi
arenarie cementate con presenza di livelli dimarne poco consistenti
arenarie cementate con presenza di piccoli livelli marnosi
arenarie compatte con strati di limo
arenarie compatte fessurate
arenarie con fratture ossidate
arenarie con intercalazioni di livelli dimarne limose
arenarie con interc

In [ ]:
#all values containing a key word
keyW = 'terreno diriporto limo'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

terreno diriporto limo con frammenti di scisti
terreno diriporto limo con ghiaia frammenti discisti alterati
terreno diriporto limo inglobante frammenti di scisti alterati
terreno diriporto limo sabbioso con ciottoli frammentati


In [ ]:
#all values containing a key word
keyW = 'terreno diriporto limoso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

terreno diriporto limoso argilloso con mattoni
terreno diriporto limoso argilloso debolmente ghiaioso con frammenti di laterizi
terreno diriporto limoso sabbioso
terreno diriporto limoso sabbioso clasti emattoni frammentati
terreno diriporto limoso sabbioso con clasti frammentati
terreno diriporto limoso sabbioso con frammenti di laterizi
terreno diriporto limoso sabbioso con frammenti di scisti alterati
terreno diriporto limoso sabbioso con frammenti rocciosi medio grossolani
terreno diriporto limoso sabbioso con frammenti rocciosi prevalentemente grossolani
terreno diriporto limoso sabbioso con frustoli vegetali frammenti dilaterizi


In [ ]:
#all values containing a key word
keyW = 'terreno di riporto prevalentemente limoso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

terreno di riporto prevalentemente limoso argilloso
terreno di riporto prevalentemente limoso argilloso con clasti e frammenti di laterizi poco consistente
terreno di riporto prevalentemente limoso argilloso con clasti rocciosi e frammenti di laterizi
terreno di riporto prevalentemente limoso argilloso con clasti sparsi e frammenti di laterizi
terreno di riporto prevalentemente limoso argilloso con frustoli vegetali frammenti di laterizi con subordinata ghiaia poco consistente plastico
terreno di riporto prevalentemente limoso argilloso con ghiaia ciottoli e frammenti di laterizi
terreno di riporto prevalentemente limoso argilloso con ghiaia ciottoli laterizi legno
terreno di riporto prevalentemente limoso argilloso con ghiaietto sparso e frammenti di laterizi
terreno di riporto prevalentemente limoso argilloso con subordinata ghiaia ciottoli e frammenti di laterizi poco consistente e compressibile
terreno di riporto prevalentemente limoso con raro ghiaietto e frammenti di laterizi spa

In [ ]:
#all values containing a key word
keyW = 'terreno di riporto limoso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

terreno di riporto limoso argilloso
terreno di riporto limoso argilloso , presenza di locali clasti eterometrici e laterizi
terreno di riporto limoso argilloso con clasti frustoli vegetali
terreno di riporto limoso argilloso con clasti sparsi
terreno di riporto limoso argilloso con clasti sparsi e frammenti di laterizi
terreno di riporto limoso argilloso con ghiaia fine
terreno di riporto limoso argilloso con ghiaie
terreno di riporto limoso argilloso con piccoli clasti sparsi
terreno di riporto limoso argilloso e frustoli vegetali
terreno di riporto limoso argilloso sabbioso con abbondante ghiaia, Frammenti di macerie e laterizi
terreno di riporto limoso argilloso sabbioso con ciottoli mattoni frammenti di bitume raro ghiaietto passante a limo con rari frammenti di mattoni
terreno di riporto limoso argilloso sabbioso, plastico e inconsistente, con frammenti sparsi laterizi, ghiaia e residui carboniosi
terreno di riporto limoso argilloso torboso con frammenti lapidei e di laterizi con 

In [ ]:
#all values containing a key word
keyW = 'terreno di riporto limo'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

terreno di riporto limo argilloso con frammenti di laterizi
terreno di riporto limo argilloso con mattoni e laterizi
terreno di riporto limo argilloso sabbioso plastico con rari ciottoli eterometrici
terreno di riporto limo sabbioso
terreno di riporto limo sabbioso abbondanti frammenti di laterizi e ghiaia eterometrica poligenica alterata
terreno di riporto limo sabbioso argilloso clasti sparsi
terreno di riporto limo sabbioso con abbondanti frammenti di laterizi
terreno di riporto limo sabbioso con ciottoli
terreno di riporto limo sabbioso con frammenti di laterizi e rara ghiaia eterometrica
terreno di riporto limo sabbioso con frammenti di mattoni
terreno di riporto limo sabbioso con frammenti litoidi e resti vegetali
terreno di riporto limo sabbioso con ghiaia eterometrica poligenica
terreno di riporto limo sabbioso con ghiaia poligenica e frammenti di laterizi
terreno di riporto limo sabbioso con rara ghiaia poligenica
terreno di riporto limo sabbioso con rari frammenti rocciosi pa

In [ ]:
#all values containing a key word
keyW = 'terreno di riporto di limo'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

terreno di riporto di limo argilloso con frammenti di laterizi da poco consistente a moderatamente consistente
terreno di riporto di limo molto consistente con presenza di rara ghiaia silicea
terreno di riporto di limo sabbioso con resti di mattoni passante a sabbia limosa con resti di mattoni
terreno di riporto di limo sabbioso e/o sabbia limosa con rara ghiaia con presenza di frammenti di laterizi
terreno di riporto di limo sabbioso e/o sabbia limosa e ghiaia eterometrica poligenica con presenza di mattoni


In [ ]:
#all values containing a key word
keyW = 'terreno di riporto costituito da limo'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

terreno di riporto costituito da limo
terreno di riporto costituito da limo argilloso con clasti e frammenti lateritici
terreno di riporto costituito da limo argilloso con clasti sparsi
terreno di riporto costituito da limo argilloso con clasti sparsi e frammenti lateritici, poco consistente
terreno di riporto costituito da limo argilloso con clasti sparsi e rari frammenti lateritici
terreno di riporto costituito da limo argilloso con clasti sparsi e subordianti frammenti lateritici
terreno di riporto costituito da limo argilloso con subordinata frazione ghiaiosa e rari frammenti lateritici, poco consistente, debolmente plastico
terreno di riporto costituito da limo argilloso con subordinati clasti
terreno di riporto costituito da limo argilloso con subordinati frammenti lateritici
terreno di riporto costituito da limo argilloso debolmente ghiaioso , poco consistente
terreno di riporto costituito da limo argilloso debolmente ghiaioso, poco consistente
terreno di riporto costituito da l

In [ ]:
#all values containing a key word
keyW = 'terreno di riporto , limo'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

terreno di riporto , limo argilloso sabbioso plastico , inconsistente
terreno di riporto , limo sabbioso
terreno di riporto , limo sabbioso con clasti sparsi
terreno di riporto , limo sabbioso con clasti sparsi , ciottoli e subordinati frammenti di laterizi
terreno di riporto , limo sabbioso con clasti sparsi e frammenti di laterizi
terreno di riporto , limo sabbioso con ghiaia eterometrica poligenica con presenza di frammenti di laterizi
terreno di riporto , limo sabbioso e sabbia media limosa con ghiaia eterometrica poligenica , rari frammenti di laterizi
terreno di riporto , limo sabbioso moderatamente consistente


In [ ]:
#all values containing a key word
keyW = 'suolo limoso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

suolo limoso
suolo limoso argilloso
suolo limoso argilloso compatto con abbondanti resti vegetali
suolo limoso con resti vegetali
suolo limoso debolmente argilloso poco consistente
suolo limoso sabbioso
suolo limoso sabbioso con inclusi resti vegetali
suolo limoso sabbioso con laterizi
suolo limoso sabbioso debolmente argilloso, consistente


In [ ]:
#all values containing a key word
keyW = 'riporto limoso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

riporto limoso argilloso
riporto limoso argilloso a trati sabbioso, con clasti sparsi, poco consistente
riporto limoso argilloso con clasti e rari frammenti di mattoni
riporto limoso argilloso con rara ghiaia , abbondanti frammenti vegetali e mattoni
riporto limoso argilloso con rara ghiaia , frammenti di radici e mattoni
riporto limoso compatto con ghiaia eterometrica e laterizi
riporto limoso con frammenti di mattoni
riporto limoso con ghiaia
riporto limoso con presenza di mattoni inconsistente e plastico
riporto limoso e mattoni inconsistente e plastico
riporto limoso ghiaioso
riporto limoso ghiaioso a tratti debolmente argilloso
riporto limoso ghiaioso con frammenti di asfalto
riporto limoso sabbiosa con frazione argillosa e mattoni in frammenti
riporto limoso sabbioso
riporto limoso sabbioso argilloso con ghiaia
riporto limoso sabbioso con ciottoli e frammenti di mattoni
riporto limoso sabbioso con detrito
riporto limoso sabbioso con frammenti lateritici
riporto limoso sabbioso co

In [ ]:
#all values containing a key word
keyW = 'riporto limo'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

riporto limo argilloso
riporto limo argilloso consistente inglobante rari frammenti litoidi con tracce di ossidazione
riporto limo argilloso localmente sabbioso con presenza di frammenti di laterizi
riporto limo con ghiaia
riporto limo sabbia ghiaia resti laterizi
riporto limo sabbiosi
riporto limo sabbioso clasti ghiaiosi sabbia fine limosa
riporto limo sabbioso con abbondante frazione argillosa rari ciottoli con frammenti di mattoni
riporto limo sabbioso con abbondanti resti di laterizi
riporto limo sabbioso con breccia medio grossolana e piccoli trovanti
riporto limo sabbioso con frammenti lateritici
riporto limo sabbioso con ghiaia
riporto limo sabbioso con lenti di torba
riporto limo sabbioso con rari ciottoli
riporto limo sabbioso con resti di laterizi
riporto limo sabbioso debolmente argilloso inglobante resti di laterizi
riporto limo sabbioso inglobante ghiaietto ghiaia e resti di mattoni
riporto limo sabbioso inglobante rara ghiaia ghiaietto e rari ciottoli con presenza di res

In [ ]:
#all values containing a key word
keyW = 'riporto di limo'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

riporto di limo argilloso compatto scarsi frammenti di laterizi
riporto di limo argilloso con frammenti di mattoni
riporto di limo argilloso con ghiaia eterometrica e resti alteritici
riporto di limo argilloso con rari elementi ghiaiosi e frammenti di mattoni
riporto di limo argilloso con raro ghiaietto frammenti di mattoni e vegetali
riporto di limo argilloso poco consistente con frammenti di calcestruzzo
riporto di limo argilloso poco consistente inglobante elementi ghiaiosi e frammenti di mattoni in alternanza con sabbia con ghiaia e frammenti di laterizi
riporto di limo argilloso sabbioso compatto con frammenti di laterizi
riporto di limo debolmente argilloso con frammenti di mattoni
riporto di limo e mattoni
riporto di limo laterizi e ghiaia passante a limo sabbioso
riporto di limo sabbioso
riporto di limo sabbioso a tratti ciottoloso con tracce di mattoni
riporto di limo sabbioso argilloso compatto con rari frammenti di laterizi
riporto di limo sabbioso argilloso inglobante brecc

In [ ]:
#all values containing a key word
keyW = 'riporto costituito da limo'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

riporto costituito da limo argilloso
riporto costituito da limo argilloso con clasti sparsi
riporto costituito da limo argilloso con clasti sparsi e frammenti lateritici
riporto costituito da limo argilloso con rari frammenti laterizi
riporto costituito da limo argilloso con sabbia fine consistente inglobante ghiaia e ciottoli
riporto costituito da limo argilloso con subordinati clasti
riporto costituito da limo argilloso sabbioso con rari ciottoli e abbondanti mattoni con calcestruzzo
riporto costituito da limo argilloso sabbioso con rari ciottoli e frammenti di mattoni
riporto costituito da limo argilloso sabbioso e ghiaia
riporto costituito da limo argilloso sabbioso e ghiaia eterometrica parzialmente sfatta , presenza di resti vegetali , consistente
riporto costituito da limo argilloso, presenza di laterizi
riporto costituito da limo con ghiaia
riporto costituito da limo con ghiaia fine sparsa , sabbia e frammenti lateritici , misto a terreno vegetale
riporto costituito da limo con

In [ ]:
#all values containing a key word
keyW = 'materiale limoso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

materiale limoso addensato
materiale limoso compatto
materiale limoso di riporto
materiale limoso sabbioso
materiale limoso sabbioso compatto , plastico
materiale limoso sabbioso con clasti di calcescisti alterati


In [ ]:
#all values containing a key word
keyW = 'livello limoso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

livello limoso
livello limoso argilloso
livello limoso argilloso con presenza di ghiaia poco consistente
livello limoso argilloso plastico
livello limoso con presenza di sporadiche concrezioni gessose
livello limoso debolmente argilloso con intercalazioni di ghiaia
livello limoso debolmente sabbioso
livello limoso e argilloso limoso
livello limoso sabbioso
livello limoso sabbioso con argilla e clasti di scisto e quarzo


In [ ]:
#all values containing a key word
keyW = 'livello di limo'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

livello di limo
livello di limo molto consistente
livello di limo sabbioso
livello di limo sabbioso debolmente consistente


In [ ]:
#all values containing a key word
keyW = 'deposito limoso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

deposito limoso
deposito limoso argillosi
deposito limoso argillosi con ciottoli e ghiaia
deposito limoso argillosi con poca sabbia
deposito limoso argillosi molto addensati
deposito limoso argilloso
deposito limoso sabbiosi con rari ciottoli
deposito limoso sabbiosi debolmente argillosi con passate locali di ghiaietto
deposito limoso sabbioso debolmente argilloso con passate locali di ghiaietto


In [ ]:
#all values containing a key word
keyW = 'da limo'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

da limo argilloso a argilla limosa con presenza di frustoli carboniosi
da limo argilloso a sabbia fine debolmente limosa
da limo argilloso ad argilla limosa
da limo argilloso inglobante rara ghiaia media a limo sabbioso
da limo con ghiaia poligenica e sabbia limosa passante a limo con rara ghiaia
da limo sabbioso a argilla limosa
da limo sabbioso a sabbia fine limosa
da limo sabbioso a sabbia fine limosa inglobante resti vegetali e torba
da limo sabbioso a sabbia media addensata debolmente limosa
da limo sabbioso a sabbia media debolmente limosa con ghiaia eterometrica poligenica
da limo sabbioso a sabbia medio fine


In [ ]:
#all values containing a key word
keyW = 'coltre limoso'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

coltre limoso sabbiosa
coltre limoso sabbiosa a tratti compatta con abbondanti ciottoli etrovanti
coltre limoso sabbiosa argillosa parzialmente rimaneggiata con frammenti di laterizi
coltre limoso sabbiosa argillosa parzialmente rimaneggiata con sporadici frammenti di laterizi
coltre limoso sabbiosa con brecce e piccoli trovanti di calcescisti
coltre limoso sabbiosa con brecce eterometriche
coltre limoso sabbiosa con brecce fini di micascisti alterati piccoli strati ossidati e resti vegetali
coltre limoso sabbiosa con ghiaia e rari ciottoli
coltre limoso sabbiosa con ghiaie e rari ciottoli
coltre limoso sabbiosa con presenza di radici
coltre limoso sabbiosa con rari ciottoli poco addensata
coltre limoso sabbiosa mediamente alterata con presenza di clasti di serpentinoscisti molto alterati e fratturati


In [ ]:
#all values containing a key word
keyW = 'cappellaccio limo'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

cappellaccio limo a volte argilloso discretamente addensato
cappellaccio limo argilloso ben addensato
cappellaccio limo argilloso ben addensato con intercalazioni sabbiose
cappellaccio limo argilloso inglobante grossi ciottoli alterati


In [ ]:
#all values containing a key word
keyW = 'alternanze di limo'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

alternanze di limo , argilla e sabbia fine limosa addensati
alternanze di limo argilloso , argilla limosa e limo sabbioso
alternanze di limo argilloso con limo sabbioso consistente
alternanze di limo argilloso e limo sabbioso
alternanze di limo argilloso e limo sabbioso molto consistente
alternanze di limo argilloso e marna debolmente cementata
alternanze di limo debolmente sabbioso e sabbie fini limose
alternanze di limo e sabbia rari ciottoli
alternanze di limo sabbioso argilloso e sabbia limosa con piccoli blocchi gneissici alterati
alternanze di limo sabbioso e di sabbia limosa
alternanze di limo sabbioso e sabbia fine limosa con ghiaia fine
alternanze di limo sabbioso scarsamente plastico debolmente consistente e argille limose scarsamente plastiche mediamente consistenti con presenza di frustoli vegetali


In [ ]:
#all values containing a key word
keyW = 'alternanza di livelli di limo'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

alternanza di livelli di limo argilloso e di sabbia fine da poco a moderatamente consistente
alternanza di livelli di limo argilloso e livelli di marna e arenaria con presenza di piccoli vuoti
alternanza di livelli di limo argilloso e livelli sabbioso-limosi, consistente
alternanza di livelli di limo argilloso sabbia limosa marna e arenaria
alternanza di livelli di limo debolmente sabbioso e di sabbia limosa
alternanza di livelli di limo debolmente sabbioso e limo con sabbia consistente
alternanza di livelli di limo e di arenaria
alternanza di livelli di limo e ghiaietto poco cementato
alternanza di livelli di limo e livelli di marna e di arenaria
alternanza di livelli di limo e livelli di marna e di arenaria con presenza di piccoli vuoti
alternanza di livelli di limo e sabbia fine da moderatamente consistente a consistente e livelli di sabbia fine e limo da moderatamente addensata a addensata a volte con presenza di frustoli organici
alternanza di livelli di limo e sabbia fine da mode

In [ ]:
#all values containing a key word
keyW = 'alternanza di limo'
index = [idx for idx, s in enumerate(key_list) if keyW in s]
for i in range(len(index)):
  position = index[i]
  keys= key_list[position].split(' ')
  key =  keys[0]
  for j in range(len(keyW.split(' '))-1):
    key = key+' '+keys[j+1]
  if key==keyW:
    print(key_list[position])
    val_list[position]=400
    key_list[position]='Limo'

alternanza di limo argilloso a argilla limosa molto consistente con frustoli e rari fossili
alternanza di limo argilloso a argilla limosa molto consistente con presenza di frustoli e frammenti carboniosi e rari fossili
alternanza di limo argilloso a volte sabbioso e di argilla debolmente marnosa
alternanza di limo argilloso e limo sabbioso inglobante rara ghiaia con presenza di piccoli livelli di sabbia medio fine limosa
alternanza di limo argilloso o argilla limosa consistente con presenza di rari frammenti di fossili e frustoli organici
alternanza di limo argilloso o argilla limosa consistente con presenza di rari frammenti fossili
alternanza di limo argilloso o argilla limosa consistente con presenza di rari frammenti fossili e di frustoli
alternanza di limo argilloso o argilla limosa da moderatamente consistente a consistente con presenza di fossili
alternanza di limo argilloso o argilla limosa da moderatamente consistente a consistente con presenza di fossili in frammenti
alternan

In [ ]:
val_list = list(np.array(pd.read_excel('val_list.xlsx',usecols='B')))
key_list = list(np.array(pd.read_excel('key_list.xlsx',usecols='B')))
for i in range(len(key_list)):
  key_list[i] = key_list[i][0]
  val_list[i] = val_list[i][0]

In [ ]:
#all values containing a key word
c = 100002
i =  41451


while c != 0:
  if (val_list[i]==100 or val_list[i]==200 or val_list[i]==201 or val_list[i]==202 or
      val_list[i]==300 or val_list[i]==306 or val_list[i]==400 or val_list[i]==602 or
      val_list[i]==700 or val_list[i]==1001):
    i=i+1
  else:
    print(i)
    print(key_list[i])
    print('1=Argilla  2=Materiale roccioso  3=calcare 4=gesso 5=ghiaia 6=Blocchi,trovanti 7=Limo 8=materiale organico 9=sabba s=skip')
    inp=input('lito:')
    if inp=='s':
        i=i+1
    else:
      c=int(inp)
      if c==1:
        val_list[i]=100
        key_list[i]='Argilla'
        print(key_list[i])
        i=i+1
      elif c==2:
        val_list[i]=200
        key_list[i]='Materiale roccioso del substrato (escluso rocce clastiche)'
        print(key_list[i])
        i=i+1
      elif c==3:
        val_list[i]=201
        key_list[i]='Calcare'
        print(key_list[i])
        i=i+1
      elif c==4:
        val_list[i]=202
        key_list[i]='Gesso, anidrite'
        print(key_list[i])
        i=i+1
      elif c==5:
        val_list[i]=300
        key_list[i]='Ghiaia'
        print(key_list[i])
        i=i+1
      elif c==6:
        val_list[i]=306
        key_list[i]='Blocchi, trovanti, erratici'
        print(key_list[i])
        i=i+1
      elif c==7:
        val_list[i]=400
        key_list[i]='Limo'
        print(key_list[i])
        i=i+1
      elif c==8:
        val_list[i]=602
        key_list[i]='Materiale organico'
        print(key_list[i])
        i=i+1        
      elif c==9:
        val_list[i]=700
        key_list[i]='Sabbia'
        print(key_list[i])
        i=i+1
      else:
        print('ERRORE!!')

41451
['terreno di riporto costituito da cemento e sabbia debolmente limosa']
1=Argilla  2=Materiale roccioso  3=calcare 4=gesso 5=ghiaia 6=Blocchi,trovanti 7=Limo 8=materiale organico 9=sabba s=skip
lito:9
Sabbia
41460
['terreno di riporto costituito da clasti con subordinati frammenti lateritici e plastica, poco addensato']
1=Argilla  2=Materiale roccioso  3=calcare 4=gesso 5=ghiaia 6=Blocchi,trovanti 7=Limo 8=materiale organico 9=sabba s=skip
lito:s
41461
['terreno di riporto costituito da clasti e frammenti in abbondante matrice fine sabbiosa debolmente limosa, poco addensato']
1=Argilla  2=Materiale roccioso  3=calcare 4=gesso 5=ghiaia 6=Blocchi,trovanti 7=Limo 8=materiale organico 9=sabba s=skip
lito:s
41462
['terreno di riporto costituito da clasti e frammenti lateritici']
1=Argilla  2=Materiale roccioso  3=calcare 4=gesso 5=ghiaia 6=Blocchi,trovanti 7=Limo 8=materiale organico 9=sabba s=skip
lito:s
41463
['terreno di riporto costituito da clasti e frammenti lateritici, poco add

IndexError: ignored

In [ ]:
#all values containing a key word
c = 100002
i =  41451


while c != 0:
  if (val_list[i]==100 or val_list[i]==200 or val_list[i]==201 or val_list[i]==202 or
      val_list[i]==300 or val_list[i]==306 or val_list[i]==400 or val_list[i]==602 or
      val_list[i]==700 or val_list[i]==1001):
    i=i+1
  else:
    print(i)
    print(key_list[i])
    print('1=Argilla  2=Materiale roccioso  3=calcare 4=gesso 5=ghiaia 6=Blocchi,trovanti 7=Limo 8=materiale organico 9=sabba s=skip')
    inp=input('lito:')
    if inp=='s':
        i=i+1
    else:
      c=int(inp)
      if c==1:
        val_list[i]=100
        key_list[i]='Argilla'
        print(key_list[i])
        i=i+1
      elif c==2:
        val_list[i]=200
        key_list[i]='Materiale roccioso del substrato (escluso rocce clastiche)'
        print(key_list[i])
        i=i+1
      elif c==3:
        val_list[i]=201
        key_list[i]='Calcare'
        print(key_list[i])
        i=i+1
      elif c==4:
        val_list[i]=202
        key_list[i]='Gesso, anidrite'
        print(key_list[i])
        i=i+1
      elif c==5:
        val_list[i]=300
        key_list[i]='Ghiaia'
        print(key_list[i])
        i=i+1
      elif c==6:
        val_list[i]=306
        key_list[i]='Blocchi, trovanti, erratici'
        print(key_list[i])
        i=i+1
      elif c==7:
        val_list[i]=400
        key_list[i]='Limo'
        print(key_list[i])
        i=i+1
      elif c==8:
        val_list[i]=602
        key_list[i]='Materiale organico'
        print(key_list[i])
        i=i+1        
      elif c==9:
        val_list[i]=700
        key_list[i]='Sabbia'
        print(key_list[i])
        i=i+1
      else:
        print('ERRORE!!')

In [ ]:
pd.DataFrame(list(val_list_val)).to_excel('val_list.xlsx')
pd.DataFrame(key_list).to_excel('key_list.xlsx')

In [ ]:
for i in range(len(val_list)):  
  if (key_list[i] == 'A' or key_list[i] == 'Mattoni, laterizi' or key_list[i] == 'L' or
        key_list[i] =='S' or key_list[i] =='M' or key_list[i] =='G'or 
        key_list[i] == 'B' or key_list[i] =='C' or key_list[i] =='(blocco di) gneiss'):
      print(key_list[i])
      print(i )
      val_list[i]=900
      key_list[i]='Altro'
      

In [ ]:
for i in range(len(val_list)):  
  if key_list[i] == 'Altro':
    print(key_list[i])

In [ ]:
val_list_val = val_list
for i in range(len(val_list)):
  if list(val_list)[i] ==2:
    val_list_val[i] = int(list(val_list)[i][0])
    val_list_val[i] = val_list_val[i][0]
  elif type(val_list_val[i]) == type(np.array([1])):
    val_list_val[i] = val_list_val[i][0]    
  elif type(val_list_val[i])==type('1'):
    print(i)
    val_list_val[i] = int((val_list_val)[i][0].split('[')[1].split(']')[0])

In [ ]:
for i in range(len(key_list)):
  if type(key_list[i]) == type(np.array([1])):
    key_list[i] = key_list[i][0]    

In [ ]:
type(val_list_val[10000]) == type(np.array([1]))

True

In [ ]:
print(pd.DataFrame(key_list)[0].unique())
print(pd.DataFrame(val_list)[0].unique())
print(len(pd.DataFrame(key_list)[0].unique()))
print(len(pd.DataFrame(val_list)[0].unique()))

['Blocchi, trovanti, erratici' 'Altro' 'Limo' 'Argilla' 'Calcare' 'Ghiaia'
 'Sabbia' 'Materiale organico'
 'Materiale roccioso del substrato (escluso rocce clastiche)'
 'Gesso, anidrite']
[306 900 400 100 201 300 700 602 200 202]
10
10


In [ ]:
#creo e stampo il dizionario
litoDict =  {}

i=0

litotype = pd.DataFrame(np.zeros(len(pd.DataFrame(key_list)[0].unique())))
litotype['name'] = pd.DataFrame(np.zeros(len(pd.DataFrame(key_list)[0].unique())))
litotype = litotype.iloc[: , 1:]
litotype['name'] = pd.DataFrame(key_list)[0].unique()
litotype.name.astype(str)
#litotype.name[742] ='zero'


for lito in litotype.name.unique():
  litoDict[lito] = pd.DataFrame(val_list_val)[0].unique()[i]
  i += 1
litoDict

{'Altro': 900,
 'Argilla': 100,
 'Blocchi, trovanti, erratici': 306,
 'Calcare': 201,
 'Gesso, anidrite': 202,
 'Ghiaia': 300,
 'Limo': 400,
 'Materiale organico': 602,
 'Materiale roccioso del substrato (escluso rocce clastiche)': 200,
 'Sabbia': 700}

In [ ]:
DFU = pd.DataFrame(key_list)[0].unique()
dim =len(pd.DataFrame(key_list)[0].unique())
for i in range(dim):  
  print(DFU[i])

Blocchi, trovanti, erratici
Altro
Limo
Argilla
Calcare
Ghiaia
Sabbia
Materiale organico
Materiale roccioso del substrato (escluso rocce clastiche)
Gesso, anidrite


In [ ]:
litoDict =  {}

i=0

litotype = pd.DataFrame(np.zeros(len(wellLoc.l1_descr.unique())))
litotype['name'] = pd.DataFrame(np.zeros(len(wellLoc.l1_descr.unique())))
litotype = litotype.iloc[: , 1:]
litotype['name'] = wellLoc.l1_descr.unique()
litotype.name.astype(str)
litotype.name[742] ='zero'
litotype = litotype.sort_values(by=['name'],ignore_index=1)


for lito in litotype.name.unique():
  Dict[lito] = key_list[i]
  i += 1
Dict

{'(blocco di) calcescisto litoide': 'Blocchi, trovanti, erratici',
 '(blocco di) gneiss': 'Altro',
 '(blocco di) serpentini litoide alterato': 'Blocchi, trovanti, erratici',
 '100limo': 'Limo',
 'Argilla con limo sabbiosa': 'Argilla',
 'Calcare marnoso': 'Calcare',
 'Ghiaia eterometrica con ciottoli in abbondante matrice limosa, a tratti debolmente sabbiosa, molto consistente; presenza di livelli a matrice prevalente': 'Limo',
 'Ghiaia eterometrica con clasti in abbondante matrice limoso sabbioso': 'Limo',
 'Ghiaia eterometrica con rari ciottoli, con clasti, in abbondante matrice limoso sabbiosa, addensata': 'Limo',
 'Ghiaia eterometrica in abbondante matrice limosa': 'Limo',
 'Ghiaia, poligenica, qualche ciottolo e sabbia limosa; presenza di intercalazione limosa': 'Ghiaia',
 'Limo debolmente argilloso': 'Limo',
 'Limo sabbioso argilloso, molto consistente': 'Limo',
 'Riporto argilloso sabbioso con ciottoli e resti di conglomerato e laterizi': 'Argilla',
 'Riporto costituito da ghiaia

In [ ]:
wellLoc['category_name'] = wellLoc['l1_descr']
wellLoc = wellLoc.replace({"category_name": Dict})

In [ ]:
wellLoc['category_name'].unique()

array(['Limo', 'Sabbia', 'Argilla', 'Ghiaia', 'Materiale organico',
       'Blocchi, trovanti, erratici', 'Altro', 'Calcare', 0,
       'Materiale roccioso del substrato (escluso rocce clastiche)',
       'Gesso, anidrite'], dtype=object)

In [ ]:
 wellLoc['category'] = wellLoc['category_name']
 wellLoc = wellLoc.replace({"category": litoDict})

#Save ouput

In [ ]:
wellLoc.to_csv('wellLito_P.csv')

In [ ]:
# creo un file di visualizzazione dei risultati
grid = pv.UnstructuredGrid(offsetArray, linArray, cellType, vertArray)
grid.cell_arrays["values"] = wellLoc.category.values
grid.save('wellLito_P.vtu',binary=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: VTK 9 no longer accepts an offset array
  
/usr/local/lib/python3.7/dist-packages/pyvista/core/dataset.py:1335: PyvistaDeprecationWarning: Use of `cell_arrays` is deprecated. Use `cell_data` instead.
  PyvistaDeprecationWarning
